# Define Library

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.
# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.cloud import storage
import os
import tempfile
import time
from datetime import datetime
import uuid
import joblib
import uuid

import gcsfs
import duckdb as dd
import pickle
import joblib
from typing import Union
import io

path = r'C:\Users\Dwaipayan\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')
os.environ["GOOGLE_CLOUD_PROJECT"] = "prj-prod-dataplatform"
# %% [markdown]
## Configure Settings
# Set options or configurations as needed
pd.set_option('display.max_columns', None)
pd.set_option("Display.max_rows", 100)

# Function

## expand_calc_features

In [2]:
import pandas as pd
import json

def expand_calc_features(df):
    """
    Expand the calcFeatures JSON column into separate columns and return the complete DataFrame.
    
    Parameters:
    df (pd.DataFrame): Input DataFrame with calcFeatures column containing JSON data
    
    Returns:
    pd.DataFrame: Expanded DataFrame with all original columns plus JSON features as separate columns
    """
    
    # Make a copy to avoid modifying the original DataFrame
    df_expanded = df.copy()
    
    # Parse the calcFeatures JSON column
    calc_features_list = []
    
    for idx, calc_features_str in enumerate(df['calcFeatures']):
        try:
            # Parse the JSON string
            features_dict = json.loads(calc_features_str.replace("'", '"'))  # Replace single quotes with double quotes for valid JSON
            calc_features_list.append(features_dict)
        except (json.JSONDecodeError, AttributeError) as e:
            # If parsing fails, create an empty dict and print warning
            print(f"Warning: Could not parse calcFeatures at index {idx}: {e}")
            calc_features_list.append({})
    
    # Create DataFrame from the parsed JSON data
    calc_features_df = pd.DataFrame(calc_features_list)
    
    # Add prefix to JSON-derived columns to avoid conflicts
    calc_features_df = calc_features_df.add_prefix('calc_')
    
    # Reset index to ensure proper alignment
    df_expanded = df_expanded.reset_index(drop=True)
    calc_features_df = calc_features_df.reset_index(drop=True)
    
    # Combine original DataFrame with expanded calcFeatures
    result_df = pd.concat([df_expanded, calc_features_df], axis=1)
    
    return result_df


## expand_calc_features_robust

In [3]:
import pandas as pd
import json

def expand_calc_features_robust(df):
    """
    Expand the calcFeatures JSON column into separate columns with better error handling.
    
    Parameters:
    df (pd.DataFrame): Input DataFrame with calcFeatures column containing JSON data
    
    Returns:
    pd.DataFrame: Expanded DataFrame with all original columns plus JSON features as separate columns
    """
    
    # Make a copy to avoid modifying the original DataFrame
    df_expanded = df.copy()
    
    # Parse the calcFeatures JSON column
    calc_features_data = []
    
    for idx, row in df.iterrows():
        calc_features_str = row['calcFeatures']
        
        if pd.isna(calc_features_str) or calc_features_str == '':
            calc_features_data.append({})
            continue
            
        try:
            # Clean the string and parse JSON
            cleaned_str = calc_features_str.replace("'", '"').replace('None', 'null').replace('True', 'true').replace('False', 'false')
            features_dict = json.loads(cleaned_str)
            calc_features_data.append(features_dict)
        except Exception as e:
            print(f"Warning: Could not parse calcFeatures at index {idx}: {e}")
            print(f"Problematic string: {calc_features_str[:100]}...")  # Print first 100 chars
            calc_features_data.append({})
    
    # Create DataFrame from the parsed JSON data
    calc_features_df = pd.DataFrame(calc_features_data)
    
    # Add prefix to JSON-derived columns to avoid conflicts with existing columns
    calc_features_df = calc_features_df.add_prefix('feat_')
    
    # Combine DataFrames
    result_df = pd.concat([df_expanded, calc_features_df], axis=1)
    
    print(f"Original DataFrame shape: {df.shape}")
    print(f"Expanded DataFrame shape: {result_df.shape}")
    print(f"Added {len(calc_features_df.columns)} new columns from calcFeatures")
    
    return result_df

## PSI Functions new

In [4]:
# import pandas as pd
# import numpy as np
# from typing import List, Dict, Tuple
# import warnings
# warnings.filterwarnings('ignore')

# def identify_feature_types(df: pd.DataFrame, feature_list: List[str]) -> Dict[str, List[str]]:
#     """
#     Identify categorical and numerical features from the feature list.
    
#     Parameters:
#     -----------
#     df : pd.DataFrame
#         Input dataframe
#     feature_list : List[str]
#         List of features to classify
    
#     Returns:
#     --------
#     Dict with 'categorical' and 'numerical' keys containing respective feature lists
#     """
#     categorical_features = []
#     numerical_features = []
    
#     for feature in feature_list:
#         if feature not in df.columns:
#             print(f"Warning: Feature '{feature}' not found in dataframe")
#             continue
            
#         # Check if feature is numeric
#         if pd.api.types.is_numeric_dtype(df[feature]):
#             # If unique values are less than 15 and all integers, treat as categorical
#             unique_vals = df[feature].nunique()
#             if unique_vals < 15 and df[feature].dropna().apply(lambda x: x == int(x) if isinstance(x, (int, float)) else False).all():
#                 categorical_features.append(feature)
#             else:
#                 numerical_features.append(feature)
#         else:
#             categorical_features.append(feature)
    
#     return {
#         'categorical': categorical_features,
#         'numerical': numerical_features
#     }


# def create_bins_for_features(df: pd.DataFrame, 
#                              numerical_features: List[str],
#                              categorical_features: List[str],
#                              base_month: str,
#                              month_col: str = 'Application_month') -> Dict:
#     """
#     Create bins for numerical features (deciles) and categorical features (top 6 + others).
    
#     Parameters:
#     -----------
#     df : pd.DataFrame
#         Input dataframe
#     numerical_features : List[str]
#         List of numerical features
#     categorical_features : List[str]
#         List of categorical features
#     base_month : str
#         Base month to determine binning strategy
#     month_col : str
#         Name of month column
    
#     Returns:
#     --------
#     Dictionary containing binning information for each feature
#     """
#     base_df = df[df[month_col] == base_month].copy()
#     binning_info = {}
    
#     # Create bins for numerical features (deciles)
#     for feature in numerical_features:
#         valid_data = base_df[feature].dropna()
        
#         if len(valid_data) == 0:
#             binning_info[feature] = {'type': 'numerical', 'bins': None}
#             continue
        
#         # Create decile bins
#         try:
#             bins = np.percentile(valid_data, np.arange(0, 101, 10))
#             # Remove duplicates and sort
#             bins = np.unique(bins)
#             binning_info[feature] = {
#                 'type': 'numerical',
#                 'bins': bins
#             }
#         except:
#             binning_info[feature] = {'type': 'numerical', 'bins': None}
    
#     # Create bins for categorical features (top 6 + others)
#     for feature in categorical_features:
#         value_counts = base_df[feature].value_counts()
#         top_6 = value_counts.nlargest(6).index.tolist()
        
#         binning_info[feature] = {
#             'type': 'categorical',
#             'top_categories': top_6
#         }
    
#     return binning_info


# def apply_binning(df: pd.DataFrame, 
#                   feature: str, 
#                   binning_info: Dict) -> pd.Series:
#     """
#     Apply binning to a feature based on binning information.
    
#     Parameters:
#     -----------
#     df : pd.DataFrame
#         Input dataframe
#     feature : str
#         Feature name
#     binning_info : Dict
#         Binning information for the feature
    
#     Returns:
#     --------
#     pd.Series with binned values
#     """
#     if binning_info['type'] == 'numerical':
#         if binning_info['bins'] is None:
#             return pd.Series(['Missing'] * len(df), index=df.index)
        
#         bins = binning_info['bins']
#         labels = [f"Bin_{i+1}" for i in range(len(bins)-1)]
        
#         binned = pd.cut(df[feature], 
#                        bins=bins, 
#                        labels=labels, 
#                        include_lowest=True,
#                        duplicates='drop')
        
#         # Handle nulls - convert to string and then replace
#         binned = binned.astype(str)
#         binned[df[feature].isna()] = 'Missing'
        
#         return binned
    
#     else:  # categorical
#         top_cats = binning_info['top_categories']
        
#         # Convert categorical to object if needed to avoid category errors
#         if pd.api.types.is_categorical_dtype(df[feature]):
#             feature_data = df[feature].astype(str)
#         else:
#             feature_data = df[feature].astype(str)
        
#         # Replace NaN string representation with 'Missing'
#         feature_data = feature_data.replace('nan', 'Missing')
        
#         # Apply binning logic
#         binned = feature_data.apply(lambda x: x if x in top_cats else ('Missing' if x == 'Missing' else 'Others'))
        
#         return binned


# def calculate_psi(expected_pct: pd.Series, 
#                   actual_pct: pd.Series, 
#                   epsilon: float = 0.0001) -> float:
#     """
#     Calculate Population Stability Index.
    
#     Parameters:
#     -----------
#     expected_pct : pd.Series
#         Expected (baseline) percentages
#     actual_pct : pd.Series
#         Actual percentages
#     epsilon : float
#         Small value to avoid log(0)
    
#     Returns:
#     --------
#     PSI value
#     """
#     # Align indices
#     all_bins = expected_pct.index.union(actual_pct.index)
#     expected_pct = expected_pct.reindex(all_bins, fill_value=0)
#     actual_pct = actual_pct.reindex(all_bins, fill_value=0)
    
#     # Add epsilon to avoid log(0)
#     expected_pct = expected_pct + epsilon
#     actual_pct = actual_pct + epsilon
    
#     # Calculate PSI
#     psi_value = np.sum((actual_pct - expected_pct) * np.log(actual_pct / expected_pct))
    
#     return psi_value


# def calculate_month_on_month_psi(df: pd.DataFrame,
#                                  feature_list: List[str],
#                                  segment_columns: List[str],
#                                  month_col: str = 'Application_month') -> pd.DataFrame:
#     """
#     Calculate month-on-month PSI for each feature, overall and by segments.
#     Each row shows PSI comparing first month vs each subsequent month.
    
#     Parameters:
#     -----------
#     df : pd.DataFrame
#         Input dataframe
#     feature_list : List[str]
#         List of features to calculate PSI for
#     segment_columns : List[str]
#         List of segment columns
#     month_col : str
#         Name of month column
    
#     Returns:
#     --------
#     pd.DataFrame with PSI values with one row per feature-month-segment combination
#     """
#     # Create a copy to avoid modifying original
#     df = df.copy()
    
#     # Identify feature types
#     feature_types = identify_feature_types(df, feature_list)
    
#     # Get sorted months
#     months = sorted(df[month_col].unique())
#     base_month = months[0]
    
#     # Create binning strategy based on base month
#     binning_info = create_bins_for_features(
#         df, 
#         feature_types['numerical'],
#         feature_types['categorical'],
#         base_month,
#         month_col
#     )
    
#     results = []
    
#     # Calculate overall PSI
#     for feature in feature_list:
#         if feature not in df.columns:
#             continue
        
#         # Apply binning
#         df[f'{feature}_binned'] = apply_binning(df, feature, binning_info[feature])
        
#         # Get base month distribution
#         base_dist = df[df[month_col] == base_month][f'{feature}_binned'].value_counts(normalize=True)
        
#         # For each month (including base month for reference)
#         for month in months:
#             actual_dist = df[df[month_col] == month][f'{feature}_binned'].value_counts(normalize=True)
#             psi_value = calculate_psi(base_dist, actual_dist) if month != base_month else 0.0
            
#             results.append({
#                 'Feature': feature,
#                 'Feature_Type': binning_info[feature]['type'],
#                 'Segment_Column': 'Overall',
#                 'Segment_Value': 'All',
#                 'Month': f"{month}",
#                 'Base_Month': base_month,
#                 'Current_Month': month,
#                 'PSI': psi_value
#             })
    
#     # Calculate PSI by segments
#     for segment_col in segment_columns:
#         if segment_col not in df.columns:
#             continue
        
#         segments = df[segment_col].dropna().unique()
        
#         for segment_val in segments:
#             segment_df = df[df[segment_col] == segment_val]
            
#             for feature in feature_list:
#                 if feature not in df.columns:
#                     continue
                
#                 # Get base month distribution for segment
#                 base_segment = segment_df[segment_df[month_col] == base_month]
#                 if len(base_segment) == 0:
#                     continue
                
#                 base_dist = base_segment[f'{feature}_binned'].value_counts(normalize=True)
                
#                 # For each month (including base month for reference)
#                 for month in months:
#                     actual_segment = segment_df[segment_df[month_col] == month]
#                     if len(actual_segment) == 0:
#                         continue
                    
#                     actual_dist = actual_segment[f'{feature}_binned'].value_counts(normalize=True)
#                     psi_value = calculate_psi(base_dist, actual_dist) if month != base_month else 0.0
                    
#                     results.append({
#                         'Feature': feature,
#                         'Feature_Type': binning_info[feature]['type'],
#                         'Segment_Column': segment_col,
#                         'Segment_Value': segment_val,
#                         'Month': f"{month}",
#                         'Base_Month': base_month,
#                         'Current_Month': month,
#                         'PSI': psi_value
#                     })
    
#     return pd.DataFrame(results)


# def calculate_bin_level_psi(df: pd.DataFrame,
#                             feature_list: List[str],
#                             segment_columns: List[str],
#                             month_col: str = 'Application_month') -> pd.DataFrame:
#     """
#     Calculate bin-level PSI for each feature, overall and by segments.
#     Each row shows bin-level PSI comparing first month vs each month.
    
#     Parameters:
#     -----------
#     df : pd.DataFrame
#         Input dataframe
#     feature_list : List[str]
#         List of features to calculate PSI for
#     segment_columns : List[str]
#         List of segment columns
#     month_col : str
#         Name of month column
    
#     Returns:
#     --------
#     pd.DataFrame with bin-level PSI details
#     """
#     # Create a copy to avoid modifying original
#     df = df.copy()
    
#     # Identify feature types
#     feature_types = identify_feature_types(df, feature_list)
    
#     # Get sorted months
#     months = sorted(df[month_col].unique())
#     base_month = months[0]
    
#     # Create binning strategy based on base month
#     binning_info = create_bins_for_features(
#         df, 
#         feature_types['numerical'],
#         feature_types['categorical'],
#         base_month,
#         month_col
#     )
    
#     results = []
#     epsilon = 0.0001
    
#     # Calculate overall bin-level PSI
#     for feature in feature_list:
#         if feature not in df.columns:
#             continue
        
#         # Apply binning
#         df[f'{feature}_binned'] = apply_binning(df, feature, binning_info[feature])
        
#         # Get base month distribution
#         base_dist = df[df[month_col] == base_month][f'{feature}_binned'].value_counts(normalize=True)
        
#         # Calculate bin-level PSI for each month (including base month)
#         for month in months:
#             actual_dist = df[df[month_col] == month][f'{feature}_binned'].value_counts(normalize=True)
            
#             # Get all bins
#             all_bins = base_dist.index.union(actual_dist.index)
            
#             for bin_name in all_bins:
#                 expected_pct = base_dist.get(bin_name, 0) + epsilon
#                 actual_pct = actual_dist.get(bin_name, 0) + epsilon
                
#                 bin_psi = (actual_pct - expected_pct) * np.log(actual_pct / expected_pct) if month != base_month else 0.0
                
#                 results.append({
#                     'Feature': feature,
#                     'Feature_Type': binning_info[feature]['type'],
#                     'Segment_Column': 'Overall',
#                     'Segment_Value': 'All',
#                     'Month': f"{month}",
#                     'Base_Month': base_month,
#                     'Current_Month': month,
#                     'Bin': bin_name,
#                     'Base_Percentage': (base_dist.get(bin_name, 0) * 100),
#                     'Actual_Percentage': (actual_dist.get(bin_name, 0) * 100),
#                     'Bin_PSI': bin_psi
#                 })
    
#     # Calculate bin-level PSI by segments
#     for segment_col in segment_columns:
#         if segment_col not in df.columns:
#             continue
        
#         segments = df[segment_col].dropna().unique()
        
#         for segment_val in segments:
#             segment_df = df[df[segment_col] == segment_val]
            
#             for feature in feature_list:
#                 if feature not in df.columns:
#                     continue
                
#                 # Get base month distribution for segment
#                 base_segment = segment_df[segment_df[month_col] == base_month]
#                 if len(base_segment) == 0:
#                     continue
                
#                 base_dist = base_segment[f'{feature}_binned'].value_counts(normalize=True)
                
#                 # Calculate bin-level PSI for each month (including base month)
#                 for month in months:
#                     actual_segment = segment_df[segment_df[month_col] == month]
#                     if len(actual_segment) == 0:
#                         continue
                    
#                     actual_dist = actual_segment[f'{feature}_binned'].value_counts(normalize=True)
                    
#                     # Get all bins
#                     all_bins = base_dist.index.union(actual_dist.index)
                    
#                     for bin_name in all_bins:
#                         expected_pct = base_dist.get(bin_name, 0) + epsilon
#                         actual_pct = actual_dist.get(bin_name, 0) + epsilon
                        
#                         bin_psi = (actual_pct - expected_pct) * np.log(actual_pct / expected_pct) if month != base_month else 0.0
                        
#                         results.append({
#                             'Feature': feature,
#                             'Feature_Type': binning_info[feature]['type'],
#                             'Segment_Column': segment_col,
#                             'Segment_Value': segment_val,
#                             'Month': f"{month}",
#                             'Base_Month': base_month,
#                             'Current_Month': month,
#                             'Bin': bin_name,
#                             'Base_Percentage': (base_dist.get(bin_name, 0) * 100),
#                             'Actual_Percentage': (actual_dist.get(bin_name, 0) * 100),
#                             'Bin_PSI': bin_psi
#                         })
    
#     return pd.DataFrame(results)

In [5]:
# import pandas as pd
# import numpy as np
# from typing import List, Dict, Tuple
# import warnings
# warnings.filterwarnings('ignore')

# def identify_feature_types(df: pd.DataFrame, feature_list: List[str]) -> Dict[str, List[str]]:
#     """
#     Identify categorical and numerical features from the feature list.
    
#     Parameters:
#     -----------
#     df : pd.DataFrame
#         Input dataframe
#     feature_list : List[str]
#         List of features to classify
    
#     Returns:
#     --------
#     Dict with 'categorical' and 'numerical' keys containing respective feature lists
#     """
#     categorical_features = []
#     numerical_features = []
    
#     for feature in feature_list:
#         if feature not in df.columns:
#             print(f"Warning: Feature '{feature}' not found in dataframe")
#             continue
            
#         # Check if feature is numeric
#         if pd.api.types.is_numeric_dtype(df[feature]):
#             # If unique values are less than 15 and all integers, treat as categorical
#             unique_vals = df[feature].nunique()
#             if unique_vals < 15 and df[feature].dropna().apply(lambda x: x == int(x) if isinstance(x, (int, float)) else False).all():
#                 categorical_features.append(feature)
#             else:
#                 numerical_features.append(feature)
#         else:
#             categorical_features.append(feature)
    
#     return {
#         'categorical': categorical_features,
#         'numerical': numerical_features
#     }


# def create_bins_for_features(df: pd.DataFrame, 
#                              numerical_features: List[str],
#                              categorical_features: List[str],
#                              base_month: str,
#                              month_col: str = 'Application_month') -> Dict:
#     """
#     Create bins for numerical features (deciles) and categorical features (top 6 + others).
    
#     Parameters:
#     -----------
#     df : pd.DataFrame
#         Input dataframe
#     numerical_features : List[str]
#         List of numerical features
#     categorical_features : List[str]
#         List of categorical features
#     base_month : str
#         Base month to determine binning strategy
#     month_col : str
#         Name of month column
    
#     Returns:
#     --------
#     Dictionary containing binning information for each feature
#     """
#     base_df = df[df[month_col] == base_month].copy()
#     binning_info = {}
    
#     # Create bins for numerical features (deciles)
#     for feature in numerical_features:
#         valid_data = base_df[feature].dropna()
        
#         if len(valid_data) == 0:
#             binning_info[feature] = {'type': 'numerical', 'bins': None, 'bin_ranges': {}}
#             continue
        
#         # Create decile bins
#         try:
#             bins = np.percentile(valid_data, np.arange(0, 101, 10))
#             # Remove duplicates and sort
#             bins = np.unique(bins)
            
#             # Create bin ranges dictionary
#             bin_ranges = {}
#             for i in range(len(bins)-1):
#                 bin_name = f"Bin_{i+1}"
#                 bin_ranges[bin_name] = {
#                     'min': bins[i],
#                     'max': bins[i+1],
#                     'range_str': f"[{bins[i]:.2f}, {bins[i+1]:.2f}]"
#                 }
            
#             binning_info[feature] = {
#                 'type': 'numerical',
#                 'bins': bins,
#                 'bin_ranges': bin_ranges
#             }
#         except:
#             binning_info[feature] = {'type': 'numerical', 'bins': None, 'bin_ranges': {}}
    
#     # Create bins for categorical features (top 6 + others)
#     for feature in categorical_features:
#         value_counts = base_df[feature].value_counts()
#         top_6 = value_counts.nlargest(6).index.tolist()
        
#         binning_info[feature] = {
#             'type': 'categorical',
#             'top_categories': top_6,
#             'bin_ranges': {}  # No ranges for categorical
#         }
    
#     return binning_info


# def apply_binning(df: pd.DataFrame, 
#                   feature: str, 
#                   binning_info: Dict) -> pd.Series:
#     """
#     Apply binning to a feature based on binning information.
    
#     Parameters:
#     -----------
#     df : pd.DataFrame
#         Input dataframe
#     feature : str
#         Feature name
#     binning_info : Dict
#         Binning information for the feature
    
#     Returns:
#     --------
#     pd.Series with binned values
#     """
#     if binning_info['type'] == 'numerical':
#         if binning_info['bins'] is None:
#             return pd.Series(['Missing'] * len(df), index=df.index)
        
#         bins = binning_info['bins']
#         labels = [f"Bin_{i+1}" for i in range(len(bins)-1)]
        
#         binned = pd.cut(df[feature], 
#                        bins=bins, 
#                        labels=labels, 
#                        include_lowest=True,
#                        duplicates='drop')
        
#         # Handle nulls - convert to string and then replace
#         binned = binned.astype(str)
#         binned[df[feature].isna()] = 'Missing'
        
#         return binned
    
#     else:  # categorical
#         top_cats = binning_info['top_categories']
        
#         # Convert categorical to object if needed to avoid category errors
#         if pd.api.types.is_categorical_dtype(df[feature]):
#             feature_data = df[feature].astype(str)
#         else:
#             feature_data = df[feature].astype(str)
        
#         # Replace NaN string representation with 'Missing'
#         feature_data = feature_data.replace('nan', 'Missing')
        
#         # Apply binning logic
#         binned = feature_data.apply(lambda x: x if x in top_cats else ('Missing' if x == 'Missing' else 'Others'))
        
#         return binned


# def calculate_psi(expected_pct: pd.Series, 
#                   actual_pct: pd.Series, 
#                   epsilon: float = 0.0001) -> float:
#     """
#     Calculate Population Stability Index.
    
#     Parameters:
#     -----------
#     expected_pct : pd.Series
#         Expected (baseline) percentages
#     actual_pct : pd.Series
#         Actual percentages
#     epsilon : float
#         Small value to avoid log(0)
    
#     Returns:
#     --------
#     PSI value
#     """
#     # Align indices
#     all_bins = expected_pct.index.union(actual_pct.index)
#     expected_pct = expected_pct.reindex(all_bins, fill_value=0)
#     actual_pct = actual_pct.reindex(all_bins, fill_value=0)
    
#     # Add epsilon to avoid log(0)
#     expected_pct = expected_pct + epsilon
#     actual_pct = actual_pct + epsilon
    
#     # Calculate PSI
#     psi_value = np.sum((actual_pct - expected_pct) * np.log(actual_pct / expected_pct))
    
#     return psi_value


# def calculate_month_on_month_psi(df: pd.DataFrame,
#                                  feature_list: List[str],
#                                  segment_columns: List[str],
#                                  month_col: str = 'Application_month') -> pd.DataFrame:
#     """
#     Calculate month-on-month PSI for each feature, overall and by segments.
#     Each row shows PSI comparing first month vs each subsequent month.
    
#     Parameters:
#     -----------
#     df : pd.DataFrame
#         Input dataframe
#     feature_list : List[str]
#         List of features to calculate PSI for
#     segment_columns : List[str]
#         List of segment columns
#     month_col : str
#         Name of month column
    
#     Returns:
#     --------
#     pd.DataFrame with PSI values with one row per feature-month-segment combination
#     """
#     # Create a copy to avoid modifying original
#     df = df.copy()
    
#     # Identify feature types
#     feature_types = identify_feature_types(df, feature_list)
    
#     # Get sorted months
#     months = sorted(df[month_col].unique())
#     base_month = months[0]
    
#     # Create binning strategy based on base month
#     binning_info = create_bins_for_features(
#         df, 
#         feature_types['numerical'],
#         feature_types['categorical'],
#         base_month,
#         month_col
#     )
    
#     results = []
    
#     # Calculate overall PSI
#     for feature in feature_list:
#         if feature not in df.columns:
#             continue
        
#         # Apply binning
#         df[f'{feature}_binned'] = apply_binning(df, feature, binning_info[feature])
        
#         # Get base month distribution
#         base_dist = df[df[month_col] == base_month][f'{feature}_binned'].value_counts(normalize=True)
        
#         # For each month (including base month for reference)
#         for month in months:
#             actual_dist = df[df[month_col] == month][f'{feature}_binned'].value_counts(normalize=True)
#             psi_value = calculate_psi(base_dist, actual_dist) if month != base_month else 0.0
            
#             results.append({
#                 'Feature': feature,
#                 'Feature_Type': binning_info[feature]['type'],
#                 'Segment_Column': 'Overall',
#                 'Segment_Value': 'All',
#                 'Month': f"{month}",
#                 'Base_Month': base_month,
#                 'Current_Month': month,
#                 'PSI': psi_value
#             })
    
#     # Calculate PSI by segments
#     for segment_col in segment_columns:
#         if segment_col not in df.columns:
#             continue
        
#         segments = df[segment_col].dropna().unique()
        
#         for segment_val in segments:
#             segment_df = df[df[segment_col] == segment_val]
            
#             for feature in feature_list:
#                 if feature not in df.columns:
#                     continue
                
#                 # Get base month distribution for segment
#                 base_segment = segment_df[segment_df[month_col] == base_month]
#                 if len(base_segment) == 0:
#                     continue
                
#                 base_dist = base_segment[f'{feature}_binned'].value_counts(normalize=True)
                
#                 # For each month (including base month for reference)
#                 for month in months:
#                     actual_segment = segment_df[segment_df[month_col] == month]
#                     if len(actual_segment) == 0:
#                         continue
                    
#                     actual_dist = actual_segment[f'{feature}_binned'].value_counts(normalize=True)
#                     psi_value = calculate_psi(base_dist, actual_dist) if month != base_month else 0.0
                    
#                     results.append({
#                         'Feature': feature,
#                         'Feature_Type': binning_info[feature]['type'],
#                         'Segment_Column': segment_col,
#                         'Segment_Value': segment_val,
#                         'Month': f"{month}",
#                         'Base_Month': base_month,
#                         'Current_Month': month,
#                         'PSI': psi_value
#                     })
    
#     return pd.DataFrame(results)


# def calculate_bin_level_psi(df: pd.DataFrame,
#                             feature_list: List[str],
#                             segment_columns: List[str],
#                             month_col: str = 'Application_month') -> pd.DataFrame:
#     """
#     Calculate bin-level PSI for each feature, overall and by segments.
#     Each row shows bin-level PSI comparing first month vs each month.
    
#     Parameters:
#     -----------
#     df : pd.DataFrame
#         Input dataframe
#     feature_list : List[str]
#         List of features to calculate PSI for
#     segment_columns : List[str]
#         List of segment columns
#     month_col : str
#         Name of month column
    
#     Returns:
#     --------
#     pd.DataFrame with bin-level PSI details including bin ranges
#     """
#     # Create a copy to avoid modifying original
#     df = df.copy()
    
#     # Identify feature types
#     feature_types = identify_feature_types(df, feature_list)
    
#     # Get sorted months
#     months = sorted(df[month_col].unique())
#     base_month = months[0]
    
#     # Create binning strategy based on base month
#     binning_info = create_bins_for_features(
#         df, 
#         feature_types['numerical'],
#         feature_types['categorical'],
#         base_month,
#         month_col
#     )
    
#     results = []
#     epsilon = 0.0001
    
#     # Calculate overall bin-level PSI
#     for feature in feature_list:
#         if feature not in df.columns:
#             continue
        
#         # Apply binning
#         df[f'{feature}_binned'] = apply_binning(df, feature, binning_info[feature])
        
#         # Get base month distribution
#         base_dist = df[df[month_col] == base_month][f'{feature}_binned'].value_counts(normalize=True)
        
#         # Calculate bin-level PSI for each month (including base month)
#         for month in months:
#             actual_dist = df[df[month_col] == month][f'{feature}_binned'].value_counts(normalize=True)
            
#             # Get all bins
#             all_bins = base_dist.index.union(actual_dist.index)
            
#             for bin_name in all_bins:
#                 expected_pct = base_dist.get(bin_name, 0) + epsilon
#                 actual_pct = actual_dist.get(bin_name, 0) + epsilon
                
#                 bin_psi = (actual_pct - expected_pct) * np.log(actual_pct / expected_pct) if month != base_month else 0.0
                
#                 # Get bin range information
#                 bin_ranges = binning_info[feature]['bin_ranges']
#                 if bin_name in bin_ranges:
#                     bin_min = bin_ranges[bin_name]['min']
#                     bin_max = bin_ranges[bin_name]['max']
#                     bin_range = bin_ranges[bin_name]['range_str']
#                 else:
#                     # For categorical or special bins (Missing, Others)
#                     bin_min = None
#                     bin_max = None
#                     bin_range = bin_name
                
#                 results.append({
#                     'Feature': feature,
#                     'Feature_Type': binning_info[feature]['type'],
#                     'Segment_Column': 'Overall',
#                     'Segment_Value': 'All',
#                     'Month': f"{month}",
#                     'Base_Month': base_month,
#                     'Current_Month': month,
#                     'Bin': bin_name,
#                     'Bin_Range': bin_range,
#                     'Bin_Min': bin_min,
#                     'Bin_Max': bin_max,
#                     'Base_Percentage': (base_dist.get(bin_name, 0) * 100),
#                     'Actual_Percentage': (actual_dist.get(bin_name, 0) * 100),
#                     'Bin_PSI': bin_psi
#                 })
    
#     # Calculate bin-level PSI by segments
#     for segment_col in segment_columns:
#         if segment_col not in df.columns:
#             continue
        
#         segments = df[segment_col].dropna().unique()
        
#         for segment_val in segments:
#             segment_df = df[df[segment_col] == segment_val]
            
#             for feature in feature_list:
#                 if feature not in df.columns:
#                     continue
                
#                 # Get base month distribution for segment
#                 base_segment = segment_df[segment_df[month_col] == base_month]
#                 if len(base_segment) == 0:
#                     continue
                
#                 base_dist = base_segment[f'{feature}_binned'].value_counts(normalize=True)
                
#                 # Calculate bin-level PSI for each month (including base month)
#                 for month in months:
#                     actual_segment = segment_df[segment_df[month_col] == month]
#                     if len(actual_segment) == 0:
#                         continue
                    
#                     actual_dist = actual_segment[f'{feature}_binned'].value_counts(normalize=True)
                    
#                     # Get all bins
#                     all_bins = base_dist.index.union(actual_dist.index)
                    
#                     for bin_name in all_bins:
#                         expected_pct = base_dist.get(bin_name, 0) + epsilon
#                         actual_pct = actual_dist.get(bin_name, 0) + epsilon
                        
#                         bin_psi = (actual_pct - expected_pct) * np.log(actual_pct / expected_pct) if month != base_month else 0.0
                        
#                         # Get bin range information
#                         bin_ranges = binning_info[feature]['bin_ranges']
#                         if bin_name in bin_ranges:
#                             bin_min = bin_ranges[bin_name]['min']
#                             bin_max = bin_ranges[bin_name]['max']
#                             bin_range = bin_ranges[bin_name]['range_str']
#                         else:
#                             # For categorical or special bins (Missing, Others)
#                             bin_min = None
#                             bin_max = None
#                             bin_range = bin_name
                        
#                         results.append({
#                             'Feature': feature,
#                             'Feature_Type': binning_info[feature]['type'],
#                             'Segment_Column': segment_col,
#                             'Segment_Value': segment_val,
#                             'Month': f"{month}",
#                             'Base_Month': base_month,
#                             'Current_Month': month,
#                             'Bin': bin_name,
#                             'Bin_Range': bin_range,
#                             'Bin_Min': bin_min,
#                             'Bin_Max': bin_max,
#                             'Base_Percentage': (base_dist.get(bin_name, 0) * 100),
#                             'Actual_Percentage': (actual_dist.get(bin_name, 0) * 100),
#                             'Bin_PSI': bin_psi
#                         })
    
#     return pd.DataFrame(results)

In [6]:
# import pandas as pd
# import numpy as np
# from typing import List, Dict, Tuple
# import warnings
# warnings.filterwarnings('ignore')

# def identify_feature_types(df: pd.DataFrame, feature_list: List[str]) -> Dict[str, List[str]]:
#     """
#     Identify categorical and numerical features from the feature list.
    
#     Parameters:
#     -----------
#     df : pd.DataFrame
#         Input dataframe
#     feature_list : List[str]
#         List of features to classify
    
#     Returns:
#     --------
#     Dict with 'categorical' and 'numerical' keys containing respective feature lists
#     """
#     categorical_features = []
#     numerical_features = []
    
#     for feature in feature_list:
#         if feature not in df.columns:
#             print(f"Warning: Feature '{feature}' not found in dataframe")
#             continue
            
#         # Check if feature is numeric
#         if pd.api.types.is_numeric_dtype(df[feature]):
#             # If unique values are less than 15 and all integers, treat as categorical
#             unique_vals = df[feature].nunique()
#             if unique_vals < 15 and df[feature].dropna().apply(lambda x: x == int(x) if isinstance(x, (int, float)) else False).all():
#                 categorical_features.append(feature)
#             else:
#                 numerical_features.append(feature)
#         else:
#             categorical_features.append(feature)
    
#     return {
#         'categorical': categorical_features,
#         'numerical': numerical_features
#     }


# def create_bins_for_features(df: pd.DataFrame, 
#                              numerical_features: List[str],
#                              categorical_features: List[str],
#                              base_month: str,
#                              month_col: str = 'Application_month') -> Dict:
#     """
#     Create bins for numerical features (deciles) and categorical features (top 6 + others).
    
#     Parameters:
#     -----------
#     df : pd.DataFrame
#         Input dataframe
#     numerical_features : List[str]
#         List of numerical features
#     categorical_features : List[str]
#         List of categorical features
#     base_month : str
#         Base month to determine binning strategy
#     month_col : str
#         Name of month column
    
#     Returns:
#     --------
#     Dictionary containing binning information for each feature
#     """
#     base_df = df[df[month_col] == base_month].copy()
#     binning_info = {}
    
#     # Create bins for numerical features (deciles)
#     for feature in numerical_features:
#         valid_data = base_df[feature].dropna()
        
#         if len(valid_data) == 0:
#             binning_info[feature] = {'type': 'numerical', 'bins': None, 'bin_ranges': {}}
#             continue
        
#         # Create decile bins
#         try:
#             bins = np.percentile(valid_data, np.arange(0, 101, 10))
#             # Remove duplicates and sort
#             bins = np.unique(bins)
            
#             # Create bin ranges dictionary
#             bin_ranges = {}
#             for i in range(len(bins)-1):
#                 bin_name = f"Bin_{i+1}"
#                 bin_ranges[bin_name] = {
#                     'min': bins[i],
#                     'max': bins[i+1],
#                     'range_str': f"[{bins[i]:.2f}, {bins[i+1]:.2f}]"
#                 }
            
#             binning_info[feature] = {
#                 'type': 'numerical',
#                 'bins': bins,
#                 'bin_ranges': bin_ranges
#             }
#         except:
#             binning_info[feature] = {'type': 'numerical', 'bins': None, 'bin_ranges': {}}
    
#     # Create bins for categorical features (top 6 + others)
#     for feature in categorical_features:
#         value_counts = base_df[feature].value_counts()
#         top_6 = value_counts.nlargest(6).index.tolist()
        
#         binning_info[feature] = {
#             'type': 'categorical',
#             'top_categories': top_6,
#             'bin_ranges': {}  # No ranges for categorical
#         }
    
#     return binning_info


# def apply_binning(df: pd.DataFrame, 
#                   feature: str, 
#                   binning_info: Dict) -> pd.Series:
#     """
#     Apply binning to a feature based on binning information.
    
#     Parameters:
#     -----------
#     df : pd.DataFrame
#         Input dataframe
#     feature : str
#         Feature name
#     binning_info : Dict
#         Binning information for the feature
    
#     Returns:
#     --------
#     pd.Series with binned values
#     """
#     if binning_info['type'] == 'numerical':
#         if binning_info['bins'] is None:
#             return pd.Series(['Missing'] * len(df), index=df.index)
        
#         bins = binning_info['bins']
#         labels = [f"Bin_{i+1}" for i in range(len(bins)-1)]
        
#         binned = pd.cut(df[feature], 
#                        bins=bins, 
#                        labels=labels, 
#                        include_lowest=True,
#                        duplicates='drop')
        
#         # Handle nulls - convert to string and then replace
#         binned = binned.astype(str)
#         binned[df[feature].isna()] = 'Missing'
        
#         return binned
    
#     else:  # categorical
#         top_cats = binning_info['top_categories']
        
#         # Convert categorical to object if needed to avoid category errors
#         if pd.api.types.is_categorical_dtype(df[feature]):
#             feature_data = df[feature].astype(str)
#         else:
#             feature_data = df[feature].astype(str)
        
#         # Replace NaN string representation with 'Missing'
#         feature_data = feature_data.replace('nan', 'Missing')
        
#         # Apply binning logic
#         binned = feature_data.apply(lambda x: x if x in top_cats else ('Missing' if x == 'Missing' else 'Others'))
        
#         return binned


# def calculate_psi(expected_pct: pd.Series, 
#                   actual_pct: pd.Series, 
#                   epsilon: float = 0.0001) -> float:
#     """
#     Calculate Population Stability Index.
    
#     Parameters:
#     -----------
#     expected_pct : pd.Series
#         Expected (baseline) percentages
#     actual_pct : pd.Series
#         Actual percentages
#     epsilon : float
#         Small value to avoid log(0)
    
#     Returns:
#     --------
#     PSI value
#     """
#     # Align indices
#     all_bins = expected_pct.index.union(actual_pct.index)
#     expected_pct = expected_pct.reindex(all_bins, fill_value=0)
#     actual_pct = actual_pct.reindex(all_bins, fill_value=0)
    
#     # Add epsilon to avoid log(0) - but handle it carefully
#     # Only add epsilon where values are zero to avoid changing the distribution
#     expected_pct = expected_pct.apply(lambda x: epsilon if x == 0 else x)
#     actual_pct = actual_pct.apply(lambda x: epsilon if x == 0 else x)
    
#     # Renormalize to ensure they sum to 1 after adding epsilon
#     expected_pct = expected_pct / expected_pct.sum()
#     actual_pct = actual_pct / actual_pct.sum()
    
#     # Calculate PSI
#     psi_value = np.sum((actual_pct - expected_pct) * np.log(actual_pct / expected_pct))
    
#     return psi_value

# def calculate_month_on_month_psi(df: pd.DataFrame,
#                                  feature_list: List[str],
#                                  segment_columns: List[str],
#                                  month_col: str = 'Application_month') -> pd.DataFrame:
#     """
#     Calculate month-on-month PSI for each feature, overall and by segments.
#     Each row shows PSI comparing first month vs each subsequent month.
    
#     Parameters:
#     -----------
#     df : pd.DataFrame
#         Input dataframe
#     feature_list : List[str]
#         List of features to calculate PSI for
#     segment_columns : List[str]
#         List of segment columns
#     month_col : str
#         Name of month column
    
#     Returns:
#     --------
#     pd.DataFrame with PSI values with one row per feature-month-segment combination
#     """
#     # Create a copy to avoid modifying original
#     df = df.copy()
    
#     # Identify feature types
#     feature_types = identify_feature_types(df, feature_list)
    
#     # Get sorted months
#     months = sorted(df[month_col].unique())
#     base_month = months[0]
    
#     # Create binning strategy based on base month
#     binning_info = create_bins_for_features(
#         df, 
#         feature_types['numerical'],
#         feature_types['categorical'],
#         base_month,
#         month_col
#     )
    
#     results = []
    
#     # Calculate overall PSI
#     for feature in feature_list:
#         if feature not in df.columns:
#             continue
        
#         # Apply binning
#         df[f'{feature}_binned'] = apply_binning(df, feature, binning_info[feature])
        
#         # Get base month distribution
#         base_dist = df[df[month_col] == base_month][f'{feature}_binned'].value_counts(normalize=True)
        
#         # For each month (including base month for reference)
#         for month in months:
#             actual_dist = df[df[month_col] == month][f'{feature}_binned'].value_counts(normalize=True)
#             psi_value = calculate_psi(base_dist, actual_dist) if month != base_month else 0.0
            
#             results.append({
#                 'Feature': feature,
#                 'Feature_Type': binning_info[feature]['type'],
#                 'Segment_Column': 'Overall',
#                 'Segment_Value': 'All',
#                 'Month': f"{month}",
#                 'Base_Month': base_month,
#                 'Current_Month': month,
#                 'PSI': psi_value
#             })
    
#     # Calculate PSI by segments
#     for segment_col in segment_columns:
#         if segment_col not in df.columns:
#             continue
        
#         segments = df[segment_col].dropna().unique()
        
#         for segment_val in segments:
#             segment_df = df[df[segment_col] == segment_val]
            
#             for feature in feature_list:
#                 if feature not in df.columns:
#                     continue
                
#                 # Get base month distribution for segment
#                 base_segment = segment_df[segment_df[month_col] == base_month]
#                 if len(base_segment) == 0:
#                     continue
                
#                 base_dist = base_segment[f'{feature}_binned'].value_counts(normalize=True)
                
#                 # For each month (including base month for reference)
#                 for month in months:
#                     actual_segment = segment_df[segment_df[month_col] == month]
#                     if len(actual_segment) == 0:
#                         continue
                    
#                     actual_dist = actual_segment[f'{feature}_binned'].value_counts(normalize=True)
#                     psi_value = calculate_psi(base_dist, actual_dist) if month != base_month else 0.0
                    
#                     results.append({
#                         'Feature': feature,
#                         'Feature_Type': binning_info[feature]['type'],
#                         'Segment_Column': segment_col,
#                         'Segment_Value': segment_val,
#                         'Month': f"{month}",
#                         'Base_Month': base_month,
#                         'Current_Month': month,
#                         'PSI': psi_value
#                     })
    
#     return pd.DataFrame(results)


# def calculate_bin_level_psi(df: pd.DataFrame,
#                             feature_list: List[str],
#                             segment_columns: List[str],
#                             month_col: str = 'Application_month') -> pd.DataFrame:
#     """
#     Calculate bin-level PSI for each feature, overall and by segments.
#     Each row shows bin-level PSI comparing first month vs each month.
    
#     Parameters:
#     -----------
#     df : pd.DataFrame
#         Input dataframe
#     feature_list : List[str]
#         List of features to calculate PSI for
#     segment_columns : List[str]
#         List of segment columns
#     month_col : str
#         Name of month column
    
#     Returns:
#     --------
#     pd.DataFrame with bin-level PSI details including bin ranges
#     """
#     # Create a copy to avoid modifying original
#     df = df.copy()
    
#     # Identify feature types
#     feature_types = identify_feature_types(df, feature_list)
    
#     # Get sorted months
#     months = sorted(df[month_col].unique())
#     base_month = months[0]
    
#     # Create binning strategy based on base month
#     binning_info = create_bins_for_features(
#         df, 
#         feature_types['numerical'],
#         feature_types['categorical'],
#         base_month,
#         month_col
#     )
    
#     results = []
#     epsilon = 0.0001
    
#     # Calculate overall bin-level PSI
#     for feature in feature_list:
#         if feature not in df.columns:
#             continue
        
#         # Apply binning
#         df[f'{feature}_binned'] = apply_binning(df, feature, binning_info[feature])
        
#         # Get base month distribution
#         base_dist = df[df[month_col] == base_month][f'{feature}_binned'].value_counts(normalize=True)
        
#         # Calculate bin-level PSI for each month (including base month)
#         for month in months:
#             actual_dist = df[df[month_col] == month][f'{feature}_binned'].value_counts(normalize=True)
            
#             # Get all bins
#             all_bins = base_dist.index.union(actual_dist.index)
            
#             for bin_name in all_bins:
#                 # expected_pct = base_dist.get(bin_name, 0) + epsilon
#                 # actual_pct = actual_dist.get(bin_name, 0) + epsilon

#                 expected_pct = base_dist.get(bin_name, 0) + epsilon
#                 actual_pct = actual_dist.get(bin_name, 0) + epsilon
#                 expected_pct = epsilon if expected_pct == 0 else expected_pct
#                 actual_pct = epsilon if actual_pct == 0 else actual_pct

                
#                 bin_psi = (actual_pct - expected_pct) * np.log(actual_pct / expected_pct) if month != base_month else 0.0
                
#                 # Get bin range information
#                 bin_ranges = binning_info[feature]['bin_ranges']
#                 if bin_name in bin_ranges:
#                     bin_min = bin_ranges[bin_name]['min']
#                     bin_max = bin_ranges[bin_name]['max']
#                     bin_range = bin_ranges[bin_name]['range_str']
#                 else:
#                     # For categorical or special bins (Missing, Others)
#                     bin_min = None
#                     bin_max = None
#                     bin_range = bin_name
                
#                 results.append({
#                     'Feature': feature,
#                     'Feature_Type': binning_info[feature]['type'],
#                     'Segment_Column': 'Overall',
#                     'Segment_Value': 'All',
#                     'Month': f"{month}",
#                     'Base_Month': base_month,
#                     'Current_Month': month,
#                     'Bin': bin_name,
#                     'Bin_Range': bin_range,
#                     'Bin_Min': bin_min,
#                     'Bin_Max': bin_max,
#                     'Base_Percentage': (base_dist.get(bin_name, 0) * 100),
#                     'Actual_Percentage': (actual_dist.get(bin_name, 0) * 100),
#                     'Bin_PSI': bin_psi
#                 })
    
#     # Calculate bin-level PSI by segments
#     for segment_col in segment_columns:
#         if segment_col not in df.columns:
#             continue
        
#         segments = df[segment_col].dropna().unique()
        
#         for segment_val in segments:
#             segment_df = df[df[segment_col] == segment_val]
            
#             for feature in feature_list:
#                 if feature not in df.columns:
#                     continue
                
#                 # Get base month distribution for segment
#                 base_segment = segment_df[segment_df[month_col] == base_month]
#                 if len(base_segment) == 0:
#                     continue
                
#                 base_dist = base_segment[f'{feature}_binned'].value_counts(normalize=True)
                
#                 # Calculate bin-level PSI for each month (including base month)
#                 for month in months:
#                     actual_segment = segment_df[segment_df[month_col] == month]
#                     if len(actual_segment) == 0:
#                         continue
                    
#                     actual_dist = actual_segment[f'{feature}_binned'].value_counts(normalize=True)
                    
#                     # Get all bins
#                     all_bins = base_dist.index.union(actual_dist.index)
                    
#                     for bin_name in all_bins:
#                         # expected_pct = base_dist.get(bin_name, 0) + epsilon
#                         # actual_pct = actual_dist.get(bin_name, 0) + epsilon

#                         expected_pct = base_dist.get(bin_name, 0) + epsilon
#                         actual_pct = actual_dist.get(bin_name, 0) + epsilon
#                         expected_pct = epsilon if expected_pct == 0 else expected_pct
#                         actual_pct = epsilon if actual_pct == 0 else actual_pct
                        
#                         bin_psi = (actual_pct - expected_pct) * np.log(actual_pct / expected_pct) if month != base_month else 0.0
                        
#                         # Get bin range information
#                         bin_ranges = binning_info[feature]['bin_ranges']
#                         if bin_name in bin_ranges:
#                             bin_min = bin_ranges[bin_name]['min']
#                             bin_max = bin_ranges[bin_name]['max']
#                             bin_range = bin_ranges[bin_name]['range_str']
#                         else:
#                             # For categorical or special bins (Missing, Others)
#                             bin_min = None
#                             bin_max = None
#                             bin_range = bin_name
                        
#                         results.append({
#                             'Feature': feature,
#                             'Feature_Type': binning_info[feature]['type'],
#                             'Segment_Column': segment_col,
#                             'Segment_Value': segment_val,
#                             'Month': f"{month}",
#                             'Base_Month': base_month,
#                             'Current_Month': month,
#                             'Bin': bin_name,
#                             'Bin_Range': bin_range,
#                             'Bin_Min': bin_min,
#                             'Bin_Max': bin_max,
#                             'Base_Percentage': (base_dist.get(bin_name, 0) * 100),
#                             'Actual_Percentage': (actual_dist.get(bin_name, 0) * 100),
#                             'Bin_PSI': bin_psi
#                         })
    
#     return pd.DataFrame(results)

In [7]:
## updated on 21-10-2025
import pandas as pd
import numpy as np
from typing import List, Dict, Tuple
import warnings
warnings.filterwarnings('ignore')

def identify_feature_types(df: pd.DataFrame, feature_list: List[str]) -> Dict[str, List[str]]:
    """
    Identify categorical and numerical features from the feature list.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature_list : List[str]
        List of features to classify
    
    Returns:
    --------
    Dict with 'categorical' and 'numerical' keys containing respective feature lists
    """
    categorical_features = []
    numerical_features = []
    
    for feature in feature_list:
        if feature not in df.columns:
            print(f"Warning: Feature '{feature}' not found in dataframe")
            continue
            
        # Check if feature is numeric
        if pd.api.types.is_numeric_dtype(df[feature]):
            # If unique values are less than 15 and all integers, treat as categorical
            unique_vals = df[feature].nunique()
            if unique_vals < 15 and df[feature].dropna().apply(lambda x: x == int(x) if isinstance(x, (int, float)) else False).all():
                categorical_features.append(feature)
            else:
                numerical_features.append(feature)
        else:
            categorical_features.append(feature)
    
    return {
        'categorical': categorical_features,
        'numerical': numerical_features
    }


def create_bins_for_features(df: pd.DataFrame, 
                             numerical_features: List[str],
                             categorical_features: List[str],
                             base_month: str,
                             month_col: str = 'Application_month') -> Dict:
    """
    Create bins for numerical features (deciles) and categorical features (top 6 + others).
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    numerical_features : List[str]
        List of numerical features
    categorical_features : List[str]
        List of categorical features
    base_month : str
        Base month to determine binning strategy
    month_col : str
        Name of month column
    
    Returns:
    --------
    Dictionary containing binning information for each feature
    """
    base_df = df[df[month_col] == base_month].copy()
    binning_info = {}
    
    # Create bins for numerical features (deciles)
    for feature in numerical_features:
        valid_data = base_df[feature].dropna()
        
        if len(valid_data) == 0:
            binning_info[feature] = {'type': 'numerical', 'bins': None, 'bin_ranges': {}}
            continue
        
        # Create decile bins
        try:
            bins = np.percentile(valid_data, np.arange(0, 101, 10))
            # Remove duplicates and sort
            bins = np.unique(bins)
            
            # IMPROVEMENT: Add infinity edges to capture all values
            bins = bins.copy()  # Ensure we can modify
            bins[0] = -np.inf
            bins[-1] = np.inf
            
            # Warn if fewer bins than expected due to low variance
            if len(bins) < 11:  # 11 edges = 10 bins
                print(f"Warning: Feature '{feature}' has only {len(bins)-1} bins due to low variance in base month")
            
            # Create bin ranges dictionary
            bin_ranges = {}
            for i in range(len(bins)-1):
                bin_name = f"Bin_{i+1}"
                bin_ranges[bin_name] = {
                    'min': bins[i],
                    'max': bins[i+1],
                    'range_str': f"[{bins[i]:.2f}, {bins[i+1]:.2f}]" if not np.isinf(bins[i]) and not np.isinf(bins[i+1]) else f"({bins[i]}, {bins[i+1]})"
                }
            
            binning_info[feature] = {
                'type': 'numerical',
                'bins': bins,
                'bin_ranges': bin_ranges
            }
        except Exception as e:
            print(f"Error creating bins for feature '{feature}': {str(e)}")
            binning_info[feature] = {'type': 'numerical', 'bins': None, 'bin_ranges': {}}
    
    # Create bins for categorical features (top 6 + others)
    for feature in categorical_features:
        value_counts = base_df[feature].value_counts()
        top_6 = value_counts.nlargest(6).index.tolist()
        
        binning_info[feature] = {
            'type': 'categorical',
            'top_categories': top_6,
            'bin_ranges': {}  # No ranges for categorical
        }
    
    return binning_info


def apply_binning(df: pd.DataFrame, 
                  feature: str, 
                  binning_info: Dict) -> pd.Series:
    """
    Apply binning to a feature based on binning information.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature : str
        Feature name
    binning_info : Dict
        Binning information for the feature
    
    Returns:
    --------
    pd.Series with binned values
    """
    if binning_info['type'] == 'numerical':
        if binning_info['bins'] is None:
            return pd.Series(['Missing'] * len(df), index=df.index)
        
        bins = binning_info['bins']
        labels = [f"Bin_{i+1}" for i in range(len(bins)-1)]
        
        binned = pd.cut(df[feature], 
                       bins=bins, 
                       labels=labels, 
                       include_lowest=True,
                       duplicates='drop')
        
        # Handle nulls - convert to string and then replace
        binned = binned.astype(str)
        binned[df[feature].isna()] = 'Missing'
        
        return binned
    
    else:  # categorical
        top_cats = binning_info['top_categories']
        
        # Convert categorical to object if needed to avoid category errors
        if pd.api.types.is_categorical_dtype(df[feature]):
            feature_data = df[feature].astype(str)
        else:
            feature_data = df[feature].astype(str)
        
        # Replace NaN string representation with 'Missing'
        feature_data = feature_data.replace('nan', 'Missing')
        
        # Apply binning logic
        binned = feature_data.apply(lambda x: x if x in top_cats else ('Missing' if x == 'Missing' else 'Others'))
        
        return binned


def calculate_psi(expected_pct: pd.Series, 
                  actual_pct: pd.Series, 
                  epsilon: float = 0.0001) -> float:
    """
    Calculate Population Stability Index with proper epsilon handling and renormalization.
    
    Parameters:
    -----------
    expected_pct : pd.Series
        Expected (baseline) percentages
    actual_pct : pd.Series
        Actual percentages
    epsilon : float
        Small value to avoid log(0)
    
    Returns:
    --------
    PSI value
    """
    # Align indices
    all_bins = expected_pct.index.union(actual_pct.index)
    expected_pct = expected_pct.reindex(all_bins, fill_value=0)
    actual_pct = actual_pct.reindex(all_bins, fill_value=0)
    
    # IMPROVEMENT: Only add epsilon where values are zero
    expected_pct = expected_pct.apply(lambda x: epsilon if x == 0 else x)
    actual_pct = actual_pct.apply(lambda x: epsilon if x == 0 else x)
    
    # IMPROVEMENT: Renormalize to ensure they sum to 1 after adding epsilon
    expected_pct = expected_pct / expected_pct.sum()
    actual_pct = actual_pct / actual_pct.sum()
    
    # Calculate PSI
    psi_value = np.sum((actual_pct - expected_pct) * np.log(actual_pct / expected_pct))
    
    return psi_value


def calculate_month_on_month_psi(df: pd.DataFrame,
                                 feature_list: List[str],
                                 segment_columns: List[str],
                                 month_col: str = 'Application_month') -> pd.DataFrame:
    """
    Calculate month-on-month PSI for each feature, overall and by segments.
    Each row shows PSI comparing first month vs each subsequent month.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature_list : List[str]
        List of features to calculate PSI for
    segment_columns : List[str]
        List of segment columns
    month_col : str
        Name of month column
    
    Returns:
    --------
    pd.DataFrame with PSI values with one row per feature-month-segment combination
    """
    # Create a copy to avoid modifying original
    df = df.copy()
    
    # Identify feature types
    feature_types = identify_feature_types(df, feature_list)
    
    # Get sorted months
    months = sorted(df[month_col].unique())
    base_month = months[0]
    
    # Create binning strategy based on base month
    binning_info = create_bins_for_features(
        df, 
        feature_types['numerical'],
        feature_types['categorical'],
        base_month,
        month_col
    )
    
    results = []
    
    # Calculate overall PSI
    for feature in feature_list:
        if feature not in df.columns:
            continue
        
        # Apply binning
        df[f'{feature}_binned'] = apply_binning(df, feature, binning_info[feature])
        
        # Get base month distribution
        base_dist = df[df[month_col] == base_month][f'{feature}_binned'].value_counts(normalize=True)
        
        # For each month (including base month for reference)
        for month in months:
            actual_dist = df[df[month_col] == month][f'{feature}_binned'].value_counts(normalize=True)
            psi_value = calculate_psi(base_dist, actual_dist) if month != base_month else 0.0
            
            results.append({
                'Feature': feature,
                'Feature_Type': binning_info[feature]['type'],
                'Segment_Column': 'Overall',
                'Segment_Value': 'All',
                'Month': f"{month}",
                'Base_Month': base_month,
                'Current_Month': month,
                'PSI': psi_value
            })
    
    # Calculate PSI by segments
    for segment_col in segment_columns:
        if segment_col not in df.columns:
            continue
        
        segments = df[segment_col].dropna().unique()
        
        for segment_val in segments:
            segment_df = df[df[segment_col] == segment_val]
            
            for feature in feature_list:
                if feature not in df.columns:
                    continue
                
                # Get base month distribution for segment
                base_segment = segment_df[segment_df[month_col] == base_month]
                if len(base_segment) == 0:
                    continue
                
                base_dist = base_segment[f'{feature}_binned'].value_counts(normalize=True)
                
                # For each month (including base month for reference)
                for month in months:
                    actual_segment = segment_df[segment_df[month_col] == month]
                    if len(actual_segment) == 0:
                        continue
                    
                    actual_dist = actual_segment[f'{feature}_binned'].value_counts(normalize=True)
                    psi_value = calculate_psi(base_dist, actual_dist) if month != base_month else 0.0
                    
                    results.append({
                        'Feature': feature,
                        'Feature_Type': binning_info[feature]['type'],
                        'Segment_Column': segment_col,
                        'Segment_Value': segment_val,
                        'Month': f"{month}",
                        'Base_Month': base_month,
                        'Current_Month': month,
                        'PSI': psi_value
                    })
    
    return pd.DataFrame(results)


def calculate_bin_level_psi(df: pd.DataFrame,
                            feature_list: List[str],
                            segment_columns: List[str],
                            month_col: str = 'Application_month') -> pd.DataFrame:
    """
    Calculate bin-level PSI for each feature, overall and by segments.
    Each row shows bin-level PSI comparing first month vs each month.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature_list : List[str]
        List of features to calculate PSI for
    segment_columns : List[str]
        List of segment columns
    month_col : str
        Name of month column
    
    Returns:
    --------
    pd.DataFrame with bin-level PSI details including bin ranges
    """
    # Create a copy to avoid modifying original
    df = df.copy()
    
    # Identify feature types
    feature_types = identify_feature_types(df, feature_list)
    
    # Get sorted months
    months = sorted(df[month_col].unique())
    base_month = months[0]
    
    # Create binning strategy based on base month
    binning_info = create_bins_for_features(
        df, 
        feature_types['numerical'],
        feature_types['categorical'],
        base_month,
        month_col
    )
    
    results = []
    epsilon = 0.0001
    
    # Calculate overall bin-level PSI
    for feature in feature_list:
        if feature not in df.columns:
            continue
        
        # Apply binning
        df[f'{feature}_binned'] = apply_binning(df, feature, binning_info[feature])
        
        # Get base month distribution
        base_dist = df[df[month_col] == base_month][f'{feature}_binned'].value_counts(normalize=True)
        
        # Calculate bin-level PSI for each month (including base month)
        for month in months:
            actual_dist = df[df[month_col] == month][f'{feature}_binned'].value_counts(normalize=True)
            
            # Get all bins
            all_bins = base_dist.index.union(actual_dist.index)
            
            for bin_name in all_bins:
                # IMPROVEMENT: Simplified epsilon logic - no redundancy
                expected_pct = base_dist.get(bin_name, 0)
                actual_pct = actual_dist.get(bin_name, 0)
                
                # Add epsilon only if zero
                expected_pct = epsilon if expected_pct == 0 else expected_pct
                actual_pct = epsilon if actual_pct == 0 else actual_pct
                
                # Calculate bin-level PSI
                bin_psi = (actual_pct - expected_pct) * np.log(actual_pct / expected_pct) if month != base_month else 0.0
                
                # Get bin range information
                bin_ranges = binning_info[feature]['bin_ranges']
                if bin_name in bin_ranges:
                    bin_min = bin_ranges[bin_name]['min']
                    bin_max = bin_ranges[bin_name]['max']
                    bin_range = bin_ranges[bin_name]['range_str']
                else:
                    # For categorical or special bins (Missing, Others)
                    bin_min = None
                    bin_max = None
                    bin_range = bin_name
                
                results.append({
                    'Feature': feature,
                    'Feature_Type': binning_info[feature]['type'],
                    'Segment_Column': 'Overall',
                    'Segment_Value': 'All',
                    'Month': f"{month}",
                    'Base_Month': base_month,
                    'Current_Month': month,
                    'Bin': bin_name,
                    'Bin_Range': bin_range,
                    'Bin_Min': bin_min,
                    'Bin_Max': bin_max,
                    'Base_Percentage': (base_dist.get(bin_name, 0) * 100),
                    'Actual_Percentage': (actual_dist.get(bin_name, 0) * 100),
                    'Bin_PSI': bin_psi
                })
    
    # Calculate bin-level PSI by segments
    for segment_col in segment_columns:
        if segment_col not in df.columns:
            continue
        
        segments = df[segment_col].dropna().unique()
        
        for segment_val in segments:
            segment_df = df[df[segment_col] == segment_val]
            
            for feature in feature_list:
                if feature not in df.columns:
                    continue
                
                # Get base month distribution for segment
                base_segment = segment_df[segment_df[month_col] == base_month]
                if len(base_segment) == 0:
                    continue
                
                base_dist = base_segment[f'{feature}_binned'].value_counts(normalize=True)
                
                # Calculate bin-level PSI for each month (including base month)
                for month in months:
                    actual_segment = segment_df[segment_df[month_col] == month]
                    if len(actual_segment) == 0:
                        continue
                    
                    actual_dist = actual_segment[f'{feature}_binned'].value_counts(normalize=True)
                    
                    # Get all bins
                    all_bins = base_dist.index.union(actual_dist.index)
                    
                    for bin_name in all_bins:
                        # IMPROVEMENT: Simplified epsilon logic - no redundancy
                        expected_pct = base_dist.get(bin_name, 0)
                        actual_pct = actual_dist.get(bin_name, 0)
                        
                        # Add epsilon only if zero
                        expected_pct = epsilon if expected_pct == 0 else expected_pct
                        actual_pct = epsilon if actual_pct == 0 else actual_pct
                        
                        # Calculate bin-level PSI
                        bin_psi = (actual_pct - expected_pct) * np.log(actual_pct / expected_pct) if month != base_month else 0.0
                        
                        # Get bin range information
                        bin_ranges = binning_info[feature]['bin_ranges']
                        if bin_name in bin_ranges:
                            bin_min = bin_ranges[bin_name]['min']
                            bin_max = bin_ranges[bin_name]['max']
                            bin_range = bin_ranges[bin_name]['range_str']
                        else:
                            # For categorical or special bins (Missing, Others)
                            bin_min = None
                            bin_max = None
                            bin_range = bin_name
                        
                        results.append({
                            'Feature': feature,
                            'Feature_Type': binning_info[feature]['type'],
                            'Segment_Column': segment_col,
                            'Segment_Value': segment_val,
                            'Month': f"{month}",
                            'Base_Month': base_month,
                            'Current_Month': month,
                            'Bin': bin_name,
                            'Bin_Range': bin_range,
                            'Bin_Min': bin_min,
                            'Bin_Max': bin_max,
                            'Base_Percentage': (base_dist.get(bin_name, 0) * 100),
                            'Actual_Percentage': (actual_dist.get(bin_name, 0) * 100),
                            'Bin_PSI': bin_psi
                        })
    
    return pd.DataFrame(results)

## PSI function old

In [8]:
# import pandas as pd
# import numpy as np
# from typing import List, Dict, Tuple
# import warnings
# warnings.filterwarnings('ignore')

# def identify_feature_types(df: pd.DataFrame, feature_list: List[str]) -> Dict[str, List[str]]:
#     """
#     Identify categorical and numerical features from the feature list.
    
#     Parameters:
#     -----------
#     df : pd.DataFrame
#         Input dataframe
#     feature_list : List[str]
#         List of features to classify
    
#     Returns:
#     --------
#     Dict with 'categorical' and 'numerical' keys containing respective feature lists
#     """
#     categorical_features = []
#     numerical_features = []
    
#     for feature in feature_list:
#         if feature not in df.columns:
#             print(f"Warning: Feature '{feature}' not found in dataframe")
#             continue
            
#         # Check if feature is numeric
#         if pd.api.types.is_numeric_dtype(df[feature]):
#             # If unique values are less than 15 and all integers, treat as categorical
#             unique_vals = df[feature].nunique()
#             if unique_vals < 15 and df[feature].dropna().apply(lambda x: x == int(x) if isinstance(x, (int, float)) else False).all():
#                 categorical_features.append(feature)
#             else:
#                 numerical_features.append(feature)
#         else:
#             categorical_features.append(feature)
    
#     return {
#         'categorical': categorical_features,
#         'numerical': numerical_features
#     }


# def create_bins_for_features(df: pd.DataFrame, 
#                              numerical_features: List[str],
#                              categorical_features: List[str],
#                              base_month: str,
#                              month_col: str = 'Application_month') -> Dict:
#     """
#     Create bins for numerical features (deciles) and categorical features (top 6 + others).
    
#     Parameters:
#     -----------
#     df : pd.DataFrame
#         Input dataframe
#     numerical_features : List[str]
#         List of numerical features
#     categorical_features : List[str]
#         List of categorical features
#     base_month : str
#         Base month to determine binning strategy
#     month_col : str
#         Name of month column
    
#     Returns:
#     --------
#     Dictionary containing binning information for each feature
#     """
#     base_df = df[df[month_col] == base_month].copy()
#     binning_info = {}
    
#     # Create bins for numerical features (deciles)
#     for feature in numerical_features:
#         valid_data = base_df[feature].dropna()
        
#         if len(valid_data) == 0:
#             binning_info[feature] = {'type': 'numerical', 'bins': None}
#             continue
        
#         # Create decile bins
#         try:
#             bins = np.percentile(valid_data, np.arange(0, 101, 10))
#             # Remove duplicates and sort
#             bins = np.unique(bins)
#             binning_info[feature] = {
#                 'type': 'numerical',
#                 'bins': bins
#             }
#         except:
#             binning_info[feature] = {'type': 'numerical', 'bins': None}
    
#     # Create bins for categorical features (top 6 + others)
#     for feature in categorical_features:
#         value_counts = base_df[feature].value_counts()
#         top_6 = value_counts.nlargest(6).index.tolist()
        
#         binning_info[feature] = {
#             'type': 'categorical',
#             'top_categories': top_6
#         }
    
#     return binning_info


# def apply_binning(df: pd.DataFrame, 
#                   feature: str, 
#                   binning_info: Dict) -> pd.Series:
#     """
#     Apply binning to a feature based on binning information.
    
#     Parameters:
#     -----------
#     df : pd.DataFrame
#         Input dataframe
#     feature : str
#         Feature name
#     binning_info : Dict
#         Binning information for the feature
    
#     Returns:
#     --------
#     pd.Series with binned values
#     """
#     if binning_info['type'] == 'numerical':
#         if binning_info['bins'] is None:
#             return pd.Series(['Missing'] * len(df), index=df.index)
        
#         bins = binning_info['bins']
#         labels = [f"Bin_{i+1}" for i in range(len(bins)-1)]
        
#         binned = pd.cut(df[feature], 
#                        bins=bins, 
#                        labels=labels, 
#                        include_lowest=True,
#                        duplicates='drop')
        
#         # Handle nulls
#         binned = binned.astype(str)
#         binned[df[feature].isna()] = 'Missing'
        
#         return binned
    
#     else:  # categorical
#         top_cats = binning_info['top_categories']
#         binned = df[feature].apply(lambda x: x if x in top_cats else 'Others')
#         binned = binned.fillna('Missing')
        
#         return binned


# def calculate_psi(expected_pct: pd.Series, 
#                   actual_pct: pd.Series, 
#                   epsilon: float = 0.0001) -> float:
#     """
#     Calculate Population Stability Index.
    
#     Parameters:
#     -----------
#     expected_pct : pd.Series
#         Expected (baseline) percentages
#     actual_pct : pd.Series
#         Actual percentages
#     epsilon : float
#         Small value to avoid log(0)
    
#     Returns:
#     --------
#     PSI value
#     """
#     # Align indices
#     all_bins = expected_pct.index.union(actual_pct.index)
#     expected_pct = expected_pct.reindex(all_bins, fill_value=0)
#     actual_pct = actual_pct.reindex(all_bins, fill_value=0)
    
#     # Add epsilon to avoid log(0)
#     expected_pct = expected_pct + epsilon
#     actual_pct = actual_pct + epsilon
    
#     # Calculate PSI
#     psi_value = np.sum((actual_pct - expected_pct) * np.log(actual_pct / expected_pct))
    
#     return psi_value


# def calculate_month_on_month_psi(df: pd.DataFrame,
#                                  feature_list: List[str],
#                                  segment_columns: List[str],
#                                  month_col: str = 'Application_month') -> pd.DataFrame:
#     """
#     Calculate month-on-month PSI for each feature, overall and by segments.
#     Each row shows PSI comparing first month vs each subsequent month.
    
#     Parameters:
#     -----------
#     df : pd.DataFrame
#         Input dataframe
#     feature_list : List[str]
#         List of features to calculate PSI for
#     segment_columns : List[str]
#         List of segment columns
#     month_col : str
#         Name of month column
    
#     Returns:
#     --------
#     pd.DataFrame with PSI values with one row per feature-month-segment combination
#     """
#     # Identify feature types
#     feature_types = identify_feature_types(df, feature_list)
    
#     # Get sorted months
#     months = sorted(df[month_col].unique())
#     base_month = months[0]
    
#     # Create binning strategy based on base month
#     binning_info = create_bins_for_features(
#         df, 
#         feature_types['numerical'],
#         feature_types['categorical'],
#         base_month,
#         month_col
#     )
    
#     results = []
    
#     # Calculate overall PSI
#     for feature in feature_list:
#         if feature not in df.columns:
#             continue
        
#         # Apply binning
#         df[f'{feature}_binned'] = apply_binning(df, feature, binning_info[feature])
        
#         # Get base month distribution
#         base_dist = df[df[month_col] == base_month][f'{feature}_binned'].value_counts(normalize=True)
        
#         # For each month (including base month for reference)
#         for month in months:
#             actual_dist = df[df[month_col] == month][f'{feature}_binned'].value_counts(normalize=True)
#             psi_value = calculate_psi(base_dist, actual_dist) if month != base_month else 0.0
            
#             results.append({
#                 'Feature': feature,
#                 'Feature_Type': binning_info[feature]['type'],
#                 'Segment_Column': 'Overall',
#                 'Segment_Value': 'All',
#                 'Month': f"{month}",
#                 'Base_Month': base_month,
#                 'Current_Month': month,
#                 'PSI': psi_value
#             })
    
#     # Calculate PSI by segments
#     for segment_col in segment_columns:
#         if segment_col not in df.columns:
#             continue
        
#         segments = df[segment_col].dropna().unique()
        
#         for segment_val in segments:
#             segment_df = df[df[segment_col] == segment_val]
            
#             for feature in feature_list:
#                 if feature not in df.columns:
#                     continue
                
#                 # Get base month distribution for segment
#                 base_segment = segment_df[segment_df[month_col] == base_month]
#                 if len(base_segment) == 0:
#                     continue
                
#                 base_dist = base_segment[f'{feature}_binned'].value_counts(normalize=True)
                
#                 # For each month (including base month for reference)
#                 for month in months:
#                     actual_segment = segment_df[segment_df[month_col] == month]
#                     if len(actual_segment) == 0:
#                         continue
                    
#                     actual_dist = actual_segment[f'{feature}_binned'].value_counts(normalize=True)
#                     psi_value = calculate_psi(base_dist, actual_dist) if month != base_month else 0.0
                    
#                     results.append({
#                         'Feature': feature,
#                         'Feature_Type': binning_info[feature]['type'],
#                         'Segment_Column': segment_col,
#                         'Segment_Value': segment_val,
#                         'Month': f"{month}",
#                         'Base_Month': base_month,
#                         'Current_Month': month,
#                         'PSI': psi_value
#                     })
    
#     # Clean up temporary columns
#     for feature in feature_list:
#         if f'{feature}_binned' in df.columns:
#             df.drop(f'{feature}_binned', axis=1, inplace=True)
    
#     return pd.DataFrame(results)


# def calculate_bin_level_psi(df: pd.DataFrame,
#                             feature_list: List[str],
#                             segment_columns: List[str],
#                             month_col: str = 'Application_month') -> pd.DataFrame:
#     """
#     Calculate bin-level PSI for each feature, overall and by segments.
#     Each row shows bin-level PSI comparing first month vs each month.
    
#     Parameters:
#     -----------
#     df : pd.DataFrame
#         Input dataframe
#     feature_list : List[str]
#         List of features to calculate PSI for
#     segment_columns : List[str]
#         List of segment columns
#     month_col : str
#         Name of month column
    
#     Returns:
#     --------
#     pd.DataFrame with bin-level PSI details
#     """
#     # Identify feature types
#     feature_types = identify_feature_types(df, feature_list)
    
#     # Get sorted months
#     months = sorted(df[month_col].unique())
#     base_month = months[0]
    
#     # Create binning strategy based on base month
#     binning_info = create_bins_for_features(
#         df, 
#         feature_types['numerical'],
#         feature_types['categorical'],
#         base_month,
#         month_col
#     )
    
#     results = []
#     epsilon = 0.0001
    
#     # Calculate overall bin-level PSI
#     for feature in feature_list:
#         if feature not in df.columns:
#             continue
        
#         # Apply binning
#         df[f'{feature}_binned'] = apply_binning(df, feature, binning_info[feature])
        
#         # Get base month distribution
#         base_dist = df[df[month_col] == base_month][f'{feature}_binned'].value_counts(normalize=True)
        
#         # Calculate bin-level PSI for each month (including base month)
#         for month in months:
#             actual_dist = df[df[month_col] == month][f'{feature}_binned'].value_counts(normalize=True)
            
#             # Get all bins
#             all_bins = base_dist.index.union(actual_dist.index)
            
#             for bin_name in all_bins:
#                 expected_pct = base_dist.get(bin_name, 0) + epsilon
#                 actual_pct = actual_dist.get(bin_name, 0) + epsilon
                
#                 bin_psi = (actual_pct - expected_pct) * np.log(actual_pct / expected_pct) if month != base_month else 0.0
                
#                 results.append({
#                     'Feature': feature,
#                     'Feature_Type': binning_info[feature]['type'],
#                     'Segment_Column': 'Overall',
#                     'Segment_Value': 'All',
#                     'Month': f"{month}",
#                     'Base_Month': base_month,
#                     'Current_Month': month,
#                     'Bin': bin_name,
#                     'Base_Percentage': (base_dist.get(bin_name, 0) * 100),
#                     'Actual_Percentage': (actual_dist.get(bin_name, 0) * 100),
#                     'Bin_PSI': bin_psi
#                 })
    
#     # Calculate bin-level PSI by segments
#     for segment_col in segment_columns:
#         if segment_col not in df.columns:
#             continue
        
#         segments = df[segment_col].dropna().unique()
        
#         for segment_val in segments:
#             segment_df = df[df[segment_col] == segment_val]
            
#             for feature in feature_list:
#                 if feature not in df.columns:
#                     continue
                
#                 # Get base month distribution for segment
#                 base_segment = segment_df[segment_df[month_col] == base_month]
#                 if len(base_segment) == 0:
#                     continue
                
#                 base_dist = base_segment[f'{feature}_binned'].value_counts(normalize=True)
                
#                 # Calculate bin-level PSI for each month (including base month)
#                 for month in months:
#                     actual_segment = segment_df[segment_df[month_col] == month]
#                     if len(actual_segment) == 0:
#                         continue
                    
#                     actual_dist = actual_segment[f'{feature}_binned'].value_counts(normalize=True)
                    
#                     # Get all bins
#                     all_bins = base_dist.index.union(actual_dist.index)
                    
#                     for bin_name in all_bins:
#                         expected_pct = base_dist.get(bin_name, 0) + epsilon
#                         actual_pct = actual_dist.get(bin_name, 0) + epsilon
                        
#                         bin_psi = (actual_pct - expected_pct) * np.log(actual_pct / expected_pct) if month != base_month else 0.0
                        
#                         results.append({
#                             'Feature': feature,
#                             'Feature_Type': binning_info[feature]['type'],
#                             'Segment_Column': segment_col,
#                             'Segment_Value': segment_val,
#                             'Month': f"{base_month} vs {month}",
#                             'Base_Month': base_month,
#                             'Current_Month': month,
#                             'Bin': bin_name,
#                             'Base_Percentage': (base_dist.get(bin_name, 0) * 100),
#                             'Actual_Percentage': (actual_dist.get(bin_name, 0) * 100),
#                             'Bin_PSI': bin_psi
#                         })
    
#     # Clean up temporary columns
#     for feature in feature_list:
#         if f'{feature}_binned' in df.columns:
#             df.drop(f'{feature}_binned', axis=1, inplace=True)
    
#     return pd.DataFrame(results)




In [9]:
# sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data;"""

# job = client.query(sq)
# job.result()  # Wait for job to complete
# print(f"Table  prj-prod-dataplatform.dap_ds_poweruser_playground.temp_csi_new_monitoring_data dropped successfully.")

# Queries

## Alpha-Cash-CIC-Model (All Trench)

### Drop table temp_csi_new_monitoring_data

In [10]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_cic_model_psi;"""

job = client.query(sq)
job.result()  # Wait for job to complete
print(f"Table  prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_cic_model_psi dropped successfully.")

sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_cic_model_csi;"""

job = client.query(sq)
job.result()  # Wait for job to complete
print(f"Table prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_cic_model_csi dropped successfully.")

Table  prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_cic_model_psi dropped successfully.
Table prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_cic_model_csi dropped successfully.


## Trench 1

In [11]:
sq = r"""  
WITH parsed as (
  select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
--REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName = 'Alpha-Cash-CIC-Model'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
--REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1),
base as (
select * from (
  select 
 r.customerId,
 r.digitalLoanAccountId,
 r.prediction,
 r.start_time,
 r.end_time,
 r.modelDisplayName,
 r.modelVersionId,
 loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
 'Cash' as product,
 'Alpha-Cash-CIC-Model_All_Trench' Model_Name,
  REGEXP_EXTRACT(m.requestPayload_clean, r"osType[:=]['\"]?([^'\"]+)['\"]?") AS osType,
  REGEXP_EXTRACT(m.requestPayload_clean, r"loanType[:=]['\"]?([^'\"]+)['\"]?") AS loanType,
  REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?") AS trenchCategory,
   SAFE_CAST(REGEXP_EXTRACT(m.requestPayload_clean, r"aStackScore[:= ]([0-9\.]+)") AS FLOAT64) AS aStackScore,
  SAFE_CAST(REGEXP_EXTRACT(m.requestPayload_clean, r"aCicScore[:= ]([0-9\.]+)") AS FLOAT64) AS aCicScore,
  calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
FROM latest_request r
left join model_run m
on r.digitalLoanAccountId = m.digitalLoanAccountId 
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by r.start_time desc) = 1
 )
)
select * from base where trenchCategory = 'Trench 1';
"""
dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,product,Model_Name,osType,loanType,trenchCategory,aStackScore,aCicScore,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,3691621,83c3ce3d-f173-4224-bd5e-ac0b16a5835f,0.6194309452490948,2025-10-12 07:48:11.008693,2025-10-12 07:48:11.108286,Alpha-Cash-CIC-Model,v1,Quick,F,not applicable,Cash,Alpha-Cash-CIC-Model_All_Trench,android,Quick,Trench 1,0.731594,0.619431,"{""cic_max_age_all_contracts_snapshot"": null, ""...",2025-10-12 15:48:01,NaT,2025-10
1,3711445,a14e8b74-a7d9-474e-9d4a-5305ad72583e,0.547154590354465,2025-10-08 05:02:57.805287,2025-10-08 05:02:58.004019,Alpha-Cash-CIC-Model,v1,Quick,F,not applicable,Cash,Alpha-Cash-CIC-Model_All_Trench,ios,Quick,Trench 1,0.532951,0.547155,"{""cic_max_age_all_contracts_snapshot"": 2647.0,...",2025-10-08 13:02:45,NaT,2025-10
2,3711909,abc37158-9c29-4acf-bad6-42cbb83c79c1,0.6323842876871181,2025-09-28 17:29:40.182250,2025-09-28 17:29:40.298214,Alpha-Cash-CIC-Model,v1,Quick,F,not applicable,Cash,Alpha-Cash-CIC-Model_All_Trench,android,Quick,Trench 1,0.719134,0.632384,"{""cic_max_age_all_contracts_snapshot"": 303, ""c...",2025-09-29 01:29:33,NaT,2025-09
3,3720121,3bb5e0fd-3c3a-42ab-afbc-7684ab84fb46,0.6194309452490948,2025-10-28 08:48:36.690176,2025-10-28 08:48:36.786084,Alpha-Cash-CIC-Model,v1,Quick,F,not applicable,Cash,Alpha-Cash-CIC-Model_All_Trench,ios,Quick,Trench 1,0.733279,0.619431,"{""cic_max_age_all_contracts_snapshot"": null, ""...",2025-10-28 16:48:30,NaT,2025-10
4,3728636,a8fc91cc-704f-4d67-85c9-dbcb23511650,0.7400605747758611,2025-10-09 15:41:40.671405,2025-10-09 15:41:40.769544,Alpha-Cash-CIC-Model,v1,Quick,F,not applicable,Cash,Alpha-Cash-CIC-Model_All_Trench,ios,Quick,Trench 1,0.677692,0.740061,"{""cic_max_age_all_contracts_snapshot"": 176.0, ...",2025-10-09 23:41:32,NaT,2025-10


In [12]:
dfd[['trenchCategory', 'Application_month']].value_counts(dropna=False)

trenchCategory  Application_month
Trench 1        2025-10              6039
                2025-11              3934
                2025-09                86
Name: count, dtype: int64

### expand df

In [13]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 21
Expanded columns: 34


,customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,product,Model_Name,osType,loanType,trenchCategory,aStackScore,aCicScore,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month,calc_cic_max_age_all_contracts_snapshot,calc_cic_ratio_overdue_contracts_to_granted_contracts,calc_cic_ScoreRange,calc_cic_ln_loan_level_user_type,calc_cic_has_ever_been_overdue,calc_cic_latest_granted_contract_overdue_flag,calc_cic_ratio_closed_over_new_granted_cnt_24M,calc_cic_ratio_risky_contracts_to_granted_contracts,calc_cic_Short_and_Term_Loans_granted_contracts_cnt_24M,calc_cic_flg_zero_non_granted_ever,calc_cic_Personal_Loans_granted_contracts_amt_24M,calc_cic_CreditAvgCreditLimit,calc_cic_flg_zero_granted_ever
0,3691621,83c3ce3d-f173-4224-bd5e-ac0b16a5835f,0.6194309452490948,2025-10-12 07:48:11.008693,2025-10-12 07:48:11.108286,Alpha-Cash-CIC-Model,v1,Quick,F,not applicable,Cash,Alpha-Cash-CIC-Model_All_Trench,android,Quick,Trench 1,0.731594,0.619431,"{""cic_max_age_all_contracts_snapshot"": null, ""...",2025-10-12 15:48:01,NaT,2025-10,NaN,NaN,NH_Gi,2_New Applicant,Unknown,Unknown,NaN,NaN,NaN,0,NaN,0.0,1
1,3711445,a14e8b74-a7d9-474e-9d4a-5305ad72583e,0.547154590354465,2025-10-08 05:02:57.805287,2025-10-08 05:02:58.004019,Alpha-Cash-CIC-Model,v1,Quick,F,not applicable,Cash,Alpha-Cash-CIC-Model_All_Trench,ios,Quick,Trench 1,0.532951,0.547155,"{""cic_max_age_all_contracts_snapshot"": 2647.0,...",2025-10-08 13:02:45,NaT,2025-10,2647.0,0.315789,Unknown,2_New Applicant,1.0,0.0,1.666667,0.263158,NaN,0,NaN,9000.0,0
2,3711909,abc37158-9c29-4acf-bad6-42cbb83c79c1,0.6323842876871181,2025-09-28 17:29:40.182250,2025-09-28 17:29:40.298214,Alpha-Cash-CIC-Model,v1,Quick,F,not applicable,Cash,Alpha-Cash-CIC-Model_All_Trench,android,Quick,Trench 1,0.719134,0.632384,"{""cic_max_age_all_contracts_snapshot"": 303, ""c...",2025-09-29 01:29:33,NaT,2025-09,303.0,0.100000,Unknown,2_New Applicant,1.0,0.0,NaN,0.000000,NaN,0,NaN,0.0,0
3,3720121,3bb5e0fd-3c3a-42ab-afbc-7684ab84fb46,0.6194309452490948,2025-10-28 08:48:36.690176,2025-10-28 08:48:36.786084,Alpha-Cash-CIC-Model,v1,Quick,F,not applicable,Cash,Alpha-Cash-CIC-Model_All_Trench,ios,Quick,Trench 1,0.733279,0.619431,"{""cic_max_age_all_contracts_snapshot"": null, ""...",2025-10-28 16:48:30,NaT,2025-10,NaN,NaN,NH_Gi,2_New Applicant,Unknown,Unknown,NaN,NaN,NaN,0,NaN,0.0,1
4,3728636,a8fc91cc-704f-4d67-85c9-dbcb23511650,0.7400605747758611,2025-10-09 15:41:40.671405,2025-10-09 15:41:40.769544,Alpha-Cash-CIC-Model,v1,Quick,F,not applicable,Cash,Alpha-Cash-CIC-Model_All_Trench,ios,Quick,Trench 1,0.677692,0.740061,"{""cic_max_age_all_contracts_snapshot"": 176.0, ...",2025-10-09 23:41:32,NaT,2025-10,176.0,0.125000,Unknown,2_New Applicant,1.0,1.0,0.750000,0.125000,NaN,1,1610.0,0.0,0


In [14]:
expanded_df.to_csv('alpha_cash_cic_model_data_for_psi.csv', index=False)

### copy to df

In [15]:
df = expanded_df.drop(columns=['calcFeatures']).copy()

### Run the PSI calculation

In [16]:
df = df.copy()

# Define feature list
feature_list = ['aCicScore',
     'calc_cic_max_age_all_contracts_snapshot',
       'calc_cic_ratio_overdue_contracts_to_granted_contracts',
       'calc_cic_ScoreRange', 'calc_cic_ln_loan_level_user_type',
       'calc_cic_has_ever_been_overdue',
       'calc_cic_latest_granted_contract_overdue_flag',
       'calc_cic_ratio_closed_over_new_granted_cnt_24M',
       'calc_cic_ratio_risky_contracts_to_granted_contracts',
       'calc_cic_Short_and_Term_Loans_granted_contracts_cnt_24M',
       'calc_cic_flg_zero_non_granted_ever',
       'calc_cic_Personal_Loans_granted_contracts_amt_24M',
       'calc_cic_CreditAvgCreditLimit', 'calc_cic_flg_zero_granted_ever'
]

# Define segment columns
segment_columns = ['new_loan_type', 'osType']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_cic_model_psi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_cic_model_csi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=7443116b-cdd9-4242-bc1c-c12195bfc190>

In [17]:
psi_results.head()

,modelDisplayName,Model_Name,modelVersionId,trenchCategory,Feature,Feature_Type,Segment_Column,Segment_Value,Month,Base_Month,Current_Month,PSI
0,Alpha-Cash-CIC-Model,Alpha-Cash-CIC-Model_All_Trench,v1,Trench 1,aCicScore,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
1,Alpha-Cash-CIC-Model,Alpha-Cash-CIC-Model_All_Trench,v1,Trench 1,aCicScore,numerical,Overall,All,2025-10,2025-09,2025-10,0.142999
2,Alpha-Cash-CIC-Model,Alpha-Cash-CIC-Model_All_Trench,v1,Trench 1,aCicScore,numerical,Overall,All,2025-11,2025-09,2025-11,0.135840
3,Alpha-Cash-CIC-Model,Alpha-Cash-CIC-Model_All_Trench,v1,Trench 1,calc_cic_max_age_all_contracts_snapshot,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
4,Alpha-Cash-CIC-Model,Alpha-Cash-CIC-Model_All_Trench,v1,Trench 1,calc_cic_max_age_all_contracts_snapshot,numerical,Overall,All,2025-10,2025-09,2025-10,0.096918


## Trench 2

In [18]:
sq = r"""  
WITH parsed as (
  select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
--REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName = 'Alpha-Cash-CIC-Model'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
--REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1),
base as (
select * from (
  select 
 r.customerId,
 r.digitalLoanAccountId,
 r.prediction,
 r.start_time,
 r.end_time,
 r.modelDisplayName,
 r.modelVersionId,
 loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
 'Cash' as product,
 'Alpha-Cash-CIC-Model_All_Trench' Model_Name,
  REGEXP_EXTRACT(m.requestPayload_clean, r"osType[:=]['\"]?([^'\"]+)['\"]?") AS osType,
  REGEXP_EXTRACT(m.requestPayload_clean, r"loanType[:=]['\"]?([^'\"]+)['\"]?") AS loanType,
  REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?") AS trenchCategory,
   SAFE_CAST(REGEXP_EXTRACT(m.requestPayload_clean, r"aStackScore[:= ]([0-9\.]+)") AS FLOAT64) AS aStackScore,
  SAFE_CAST(REGEXP_EXTRACT(m.requestPayload_clean, r"aCicScore[:= ]([0-9\.]+)") AS FLOAT64) AS aCicScore,
  calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
FROM latest_request r
left join model_run m
on r.digitalLoanAccountId = m.digitalLoanAccountId 
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by r.start_time desc) = 1
 )
)
select * from base where trenchCategory = 'Trench 2';
"""
dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,product,Model_Name,osType,loanType,trenchCategory,aStackScore,aCicScore,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,1026925,a6ac7b7e-9d1b-4d9e-806e-da97ddfa3b5d,0.38356382212081114,2025-11-14 15:56:13.317832,2025-11-14 15:56:13.457233,Alpha-Cash-CIC-Model,v1,Quick,F,not applicable,Cash,Alpha-Cash-CIC-Model_All_Trench,android,Quick,Trench 2,0.166102,0.383564,"{""cic_max_age_all_contracts_snapshot"": 5366.0,...",2025-11-14 23:55:46,2025-11-15 17:18:59,2025-11
1,1115541,1961bab0-8943-423f-951e-f07927a833a1,0.48381006422098377,2025-11-12 08:51:02.411490,2025-11-12 08:51:02.560943,Alpha-Cash-CIC-Model,v1,Quick,F,not applicable,Cash,Alpha-Cash-CIC-Model_All_Trench,ios,Quick,Trench 2,0.393062,0.483810,"{""cic_max_age_all_contracts_snapshot"": 5487.0,...",2025-11-12 16:50:24,2025-11-12 17:10:10,2025-11
2,1173675,83bf0b81-137b-4d31-a20e-32303d5aa2fc,0.4297724962408501,2025-11-08 13:50:24.577744,2025-11-08 13:50:24.682209,Alpha-Cash-CIC-Model,v1,Quick,M,not applicable,Cash,Alpha-Cash-CIC-Model_All_Trench,ios,Quick,Trench 2,0.247891,0.429772,"{""cic_max_age_all_contracts_snapshot"": 1081, ""...",2025-11-08 21:50:16,NaT,2025-11
3,1204195,b4b6072c-b354-4bd0-9de7-8e4ffcb20fc5,0.6054938855593464,2025-10-14 10:43:55.282504,2025-10-14 10:43:55.670163,Alpha-Cash-CIC-Model,v1,Quick,M,not applicable,Cash,Alpha-Cash-CIC-Model_All_Trench,android,Quick,Trench 2,0.500006,0.605494,"{""cic_max_age_all_contracts_snapshot"": 349.0, ...",2025-10-14 18:43:41,NaT,2025-10
4,1258143,4936f023-a345-444b-881e-649414f0420c,0.5634416538416653,2025-11-12 09:28:05.330635,2025-11-12 09:28:05.476053,Alpha-Cash-CIC-Model,v1,Quick,F,not applicable,Cash,Alpha-Cash-CIC-Model_All_Trench,ios,Quick,Trench 2,0.598362,0.563442,"{""cic_max_age_all_contracts_snapshot"": 1602.0,...",2025-11-12 17:27:53,NaT,2025-11


### expand df

In [19]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 21
Expanded columns: 34


,customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,product,Model_Name,osType,loanType,trenchCategory,aStackScore,aCicScore,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month,calc_cic_max_age_all_contracts_snapshot,calc_cic_ratio_overdue_contracts_to_granted_contracts,calc_cic_ScoreRange,calc_cic_ln_loan_level_user_type,calc_cic_has_ever_been_overdue,calc_cic_latest_granted_contract_overdue_flag,calc_cic_ratio_closed_over_new_granted_cnt_24M,calc_cic_ratio_risky_contracts_to_granted_contracts,calc_cic_Short_and_Term_Loans_granted_contracts_cnt_24M,calc_cic_flg_zero_non_granted_ever,calc_cic_Personal_Loans_granted_contracts_amt_24M,calc_cic_CreditAvgCreditLimit,calc_cic_flg_zero_granted_ever
0,1026925,a6ac7b7e-9d1b-4d9e-806e-da97ddfa3b5d,0.38356382212081114,2025-11-14 15:56:13.317832,2025-11-14 15:56:13.457233,Alpha-Cash-CIC-Model,v1,Quick,F,not applicable,Cash,Alpha-Cash-CIC-Model_All_Trench,android,Quick,Trench 2,0.166102,0.383564,"{""cic_max_age_all_contracts_snapshot"": 5366.0,...",2025-11-14 23:55:46,2025-11-15 17:18:59,2025-11,5366.0,0.100000,Di,2_New Applicant,1.0,0.0,0.666667,0.100000,NaN,1,NaN,69000.0,0
1,1115541,1961bab0-8943-423f-951e-f07927a833a1,0.48381006422098377,2025-11-12 08:51:02.411490,2025-11-12 08:51:02.560943,Alpha-Cash-CIC-Model,v1,Quick,F,not applicable,Cash,Alpha-Cash-CIC-Model_All_Trench,ios,Quick,Trench 2,0.393062,0.483810,"{""cic_max_age_all_contracts_snapshot"": 5487.0,...",2025-11-12 16:50:24,2025-11-12 17:10:10,2025-11,5487.0,0.272727,Unknown,2_New Applicant,1.0,0.0,1.000000,0.363636,1.0,0,3657.0,0.0,0
2,1173675,83bf0b81-137b-4d31-a20e-32303d5aa2fc,0.4297724962408501,2025-11-08 13:50:24.577744,2025-11-08 13:50:24.682209,Alpha-Cash-CIC-Model,v1,Quick,M,not applicable,Cash,Alpha-Cash-CIC-Model_All_Trench,ios,Quick,Trench 2,0.247891,0.429772,"{""cic_max_age_all_contracts_snapshot"": 1081, ""...",2025-11-08 21:50:16,NaT,2025-11,1081.0,0.000000,NH_Fi,2_New Applicant,0.0,0.0,1.200000,0.000000,5.0,0,NaN,0.0,0
3,1204195,b4b6072c-b354-4bd0-9de7-8e4ffcb20fc5,0.6054938855593464,2025-10-14 10:43:55.282504,2025-10-14 10:43:55.670163,Alpha-Cash-CIC-Model,v1,Quick,M,not applicable,Cash,Alpha-Cash-CIC-Model_All_Trench,android,Quick,Trench 2,0.500006,0.605494,"{""cic_max_age_all_contracts_snapshot"": 349.0, ...",2025-10-14 18:43:41,NaT,2025-10,349.0,0.301205,NH_Hi,2_New Applicant,1.0,0.0,3.000000,0.036145,3.0,0,13093.0,50000.0,0
4,1258143,4936f023-a345-444b-881e-649414f0420c,0.5634416538416653,2025-11-12 09:28:05.330635,2025-11-12 09:28:05.476053,Alpha-Cash-CIC-Model,v1,Quick,F,not applicable,Cash,Alpha-Cash-CIC-Model_All_Trench,ios,Quick,Trench 2,0.598362,0.563442,"{""cic_max_age_all_contracts_snapshot"": 1602.0,...",2025-11-12 17:27:53,NaT,2025-11,1602.0,0.357143,NH_Fi,2_New Applicant,1.0,0.0,1.400000,0.071429,1.0,0,8500.0,17500.0,0


### copy to df

In [20]:
df = expanded_df.drop(columns=['calcFeatures']).copy()

### Run the PSI calculation

In [21]:
df = df.copy()

# Define feature list
feature_list = ['aCicScore',
     'calc_cic_max_age_all_contracts_snapshot',
       'calc_cic_ratio_overdue_contracts_to_granted_contracts',
       'calc_cic_ScoreRange', 'calc_cic_ln_loan_level_user_type',
       'calc_cic_has_ever_been_overdue',
       'calc_cic_latest_granted_contract_overdue_flag',
       'calc_cic_ratio_closed_over_new_granted_cnt_24M',
       'calc_cic_ratio_risky_contracts_to_granted_contracts',
       'calc_cic_Short_and_Term_Loans_granted_contracts_cnt_24M',
       'calc_cic_flg_zero_non_granted_ever',
       'calc_cic_Personal_Loans_granted_contracts_amt_24M',
       'calc_cic_CreditAvgCreditLimit', 'calc_cic_flg_zero_granted_ever'
]

# Define segment columns
segment_columns = ['new_loan_type', 'osType']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_cic_model_psi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_cic_model_csi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=df875b7e-32d6-4f2c-895b-d94719b41912>

In [22]:
psi_results.head()

,modelDisplayName,Model_Name,modelVersionId,trenchCategory,Feature,Feature_Type,Segment_Column,Segment_Value,Month,Base_Month,Current_Month,PSI
0,Alpha-Cash-CIC-Model,Alpha-Cash-CIC-Model_All_Trench,v1,Trench 2,aCicScore,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
1,Alpha-Cash-CIC-Model,Alpha-Cash-CIC-Model_All_Trench,v1,Trench 2,aCicScore,numerical,Overall,All,2025-10,2025-09,2025-10,0.462222
2,Alpha-Cash-CIC-Model,Alpha-Cash-CIC-Model_All_Trench,v1,Trench 2,aCicScore,numerical,Overall,All,2025-11,2025-09,2025-11,0.345709
3,Alpha-Cash-CIC-Model,Alpha-Cash-CIC-Model_All_Trench,v1,Trench 2,calc_cic_max_age_all_contracts_snapshot,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
4,Alpha-Cash-CIC-Model,Alpha-Cash-CIC-Model_All_Trench,v1,Trench 2,calc_cic_max_age_all_contracts_snapshot,numerical,Overall,All,2025-10,2025-09,2025-10,0.235447


## Trench 3

In [23]:
sq = r"""  
WITH parsed as (
  select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
--REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName = 'Alpha-Cash-CIC-Model'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
--REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1),
base as (
select * from (
  select 
 r.customerId,
 r.digitalLoanAccountId,
 r.prediction,
 r.start_time,
 r.end_time,
 r.modelDisplayName,
 r.modelVersionId,
 loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
 'Cash' as product,
 'Alpha-Cash-CIC-Model_All_Trench' Model_Name,
  REGEXP_EXTRACT(m.requestPayload_clean, r"osType[:=]['\"]?([^'\"]+)['\"]?") AS osType,
  REGEXP_EXTRACT(m.requestPayload_clean, r"loanType[:=]['\"]?([^'\"]+)['\"]?") AS loanType,
  REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?") AS trenchCategory,
   SAFE_CAST(REGEXP_EXTRACT(m.requestPayload_clean, r"aStackScore[:= ]([0-9\.]+)") AS FLOAT64) AS aStackScore,
  SAFE_CAST(REGEXP_EXTRACT(m.requestPayload_clean, r"aCicScore[:= ]([0-9\.]+)") AS FLOAT64) AS aCicScore,
  calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
FROM latest_request r
left join model_run m
on r.digitalLoanAccountId = m.digitalLoanAccountId 
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by r.start_time desc) = 1
 )
)
select * from base where trenchCategory = 'Trench 3';
"""
dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,product,Model_Name,osType,loanType,trenchCategory,aStackScore,aCicScore,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,1200777,33c5aa15-4b17-4c10-939a-376e5fb00d74,0.37292797464415767,2025-11-16 14:54:59.562808,2025-11-16 14:54:59.788254,Alpha-Cash-CIC-Model,v1,Quick,F,not applicable,Cash,Alpha-Cash-CIC-Model_All_Trench,android,Quick,Trench 3,0.352459,0.372928,"{""cic_max_age_all_contracts_snapshot"": 1476.0,...",2025-11-16 22:54:36,2025-11-17 11:20:03,2025-11
1,1732633,00a2ae21-457b-42c4-af71-42d181984285,0.4261063097695881,2025-11-14 20:24:56.464587,2025-11-14 20:24:56.864264,Alpha-Cash-CIC-Model,v1,Quick,F,not applicable,Cash,Alpha-Cash-CIC-Model_All_Trench,android,Quick,Trench 3,0.549900,0.426106,"{""cic_max_age_all_contracts_snapshot"": 1780.0,...",2025-11-15 04:24:35,2025-11-15 10:20:07,2025-11
2,1798822,a0e9dfae-80ca-43b6-a63c-007eea8dd53e,0.43944393736533327,2025-10-24 02:54:45.205405,2025-10-24 02:54:45.303889,Alpha-Cash-CIC-Model,v1,Quick,F,not applicable,Cash,Alpha-Cash-CIC-Model_All_Trench,ios,Quick,Trench 3,0.526616,0.439444,"{""cic_max_age_all_contracts_snapshot"": 415.0, ...",2025-10-24 10:54:25,2025-10-26 17:24:06,2025-10
3,1851438,11cc25e2-be3d-4600-bf8f-19efd4c7d432,0.41211848685859614,2025-10-29 02:43:35.896479,2025-10-29 02:43:36.125900,Alpha-Cash-CIC-Model,v1,Quick,F,not applicable,Cash,Alpha-Cash-CIC-Model_All_Trench,ios,Quick,Trench 3,0.516727,0.412118,"{""cic_max_age_all_contracts_snapshot"": 1241.0,...",2025-10-29 10:43:11,2025-10-29 11:53:37,2025-10
4,2266830,476520a9-32ea-4d39-80fe-cb7fa70c0b88,0.42837890919948235,2025-10-22 11:13:06.249249,2025-10-22 11:13:06.481791,Alpha-Cash-CIC-Model,v1,Quick,F,not applicable,Cash,Alpha-Cash-CIC-Model_All_Trench,android,Quick,Trench 3,0.397230,0.428379,"{""cic_max_age_all_contracts_snapshot"": 2204, ""...",2025-10-22 19:12:41,2025-10-22 19:20:03,2025-10


### expand df

In [24]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 21
Expanded columns: 34


,customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,product,Model_Name,osType,loanType,trenchCategory,aStackScore,aCicScore,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month,calc_cic_max_age_all_contracts_snapshot,calc_cic_ratio_overdue_contracts_to_granted_contracts,calc_cic_ScoreRange,calc_cic_ln_loan_level_user_type,calc_cic_has_ever_been_overdue,calc_cic_latest_granted_contract_overdue_flag,calc_cic_ratio_closed_over_new_granted_cnt_24M,calc_cic_ratio_risky_contracts_to_granted_contracts,calc_cic_Short_and_Term_Loans_granted_contracts_cnt_24M,calc_cic_flg_zero_non_granted_ever,calc_cic_Personal_Loans_granted_contracts_amt_24M,calc_cic_CreditAvgCreditLimit,calc_cic_flg_zero_granted_ever
0,1200777,33c5aa15-4b17-4c10-939a-376e5fb00d74,0.37292797464415767,2025-11-16 14:54:59.562808,2025-11-16 14:54:59.788254,Alpha-Cash-CIC-Model,v1,Quick,F,not applicable,Cash,Alpha-Cash-CIC-Model_All_Trench,android,Quick,Trench 3,0.352459,0.372928,"{""cic_max_age_all_contracts_snapshot"": 1476.0,...",2025-11-16 22:54:36,2025-11-17 11:20:03,2025-11,1476.0,0.000000,Bi,1_Repeat Applicant,0.0,0.0,0.813953,0.000000,32.0,0,13935.0,0.0,0
1,1732633,00a2ae21-457b-42c4-af71-42d181984285,0.4261063097695881,2025-11-14 20:24:56.464587,2025-11-14 20:24:56.864264,Alpha-Cash-CIC-Model,v1,Quick,F,not applicable,Cash,Alpha-Cash-CIC-Model_All_Trench,android,Quick,Trench 3,0.549900,0.426106,"{""cic_max_age_all_contracts_snapshot"": 1780.0,...",2025-11-15 04:24:35,2025-11-15 10:20:07,2025-11,1780.0,0.075630,Unknown,1_Repeat Applicant,1.0,0.0,1.009434,0.058824,82.0,0,263279.0,52500.0,0
2,1798822,a0e9dfae-80ca-43b6-a63c-007eea8dd53e,0.43944393736533327,2025-10-24 02:54:45.205405,2025-10-24 02:54:45.303889,Alpha-Cash-CIC-Model,v1,Quick,F,not applicable,Cash,Alpha-Cash-CIC-Model_All_Trench,ios,Quick,Trench 3,0.526616,0.439444,"{""cic_max_age_all_contracts_snapshot"": 415.0, ...",2025-10-24 10:54:25,2025-10-26 17:24:06,2025-10,415.0,0.000000,Ci,1_Repeat Applicant,0.0,0.0,0.500000,0.000000,NaN,0,NaN,25000.0,0
3,1851438,11cc25e2-be3d-4600-bf8f-19efd4c7d432,0.41211848685859614,2025-10-29 02:43:35.896479,2025-10-29 02:43:36.125900,Alpha-Cash-CIC-Model,v1,Quick,F,not applicable,Cash,Alpha-Cash-CIC-Model_All_Trench,ios,Quick,Trench 3,0.516727,0.412118,"{""cic_max_age_all_contracts_snapshot"": 1241.0,...",2025-10-29 10:43:11,2025-10-29 11:53:37,2025-10,1241.0,0.021739,Ei,1_Repeat Applicant,1.0,0.0,2.400000,0.021739,NaN,1,31167.0,36000.0,0
4,2266830,476520a9-32ea-4d39-80fe-cb7fa70c0b88,0.42837890919948235,2025-10-22 11:13:06.249249,2025-10-22 11:13:06.481791,Alpha-Cash-CIC-Model,v1,Quick,F,not applicable,Cash,Alpha-Cash-CIC-Model_All_Trench,android,Quick,Trench 3,0.397230,0.428379,"{""cic_max_age_all_contracts_snapshot"": 2204, ""...",2025-10-22 19:12:41,2025-10-22 19:20:03,2025-10,2204.0,0.018182,Unknown,1_Repeat Applicant,1.0,0.0,0.961538,0.000000,8.0,0,45050.0,30000.0,0


In [25]:
# sq = r"""  
# WITH parsed as (
#   select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,
# REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
# --REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
# FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
# where modelDisplayName = 'Alpha-Cash-CIC-Model'
# ),

# latest_request as (
# select * from parsed
# QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

# model_run as (
# select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
# --REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
# from `prj-prod-dataplatform.audit_balance.ml_request_details` 
# WHERE modelName = 'Alpha-Cash-Model-response'
# QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1),
# base as (
# select * from (
#   select 
#  r.customerId,
#  r.digitalLoanAccountId,
#  r.prediction,
#  r.start_time,
#  r.end_time,
#  r.modelDisplayName,
#  r.modelVersionId,
#  loanmaster.new_loan_type,
#  loanmaster.gender,
#   REGEXP_EXTRACT(m.requestPayload_clean, r"osType[:=]['\"]?([^'\"]+)['\"]?") AS osType,
#   REGEXP_EXTRACT(m.requestPayload_clean, r"loanType[:=]['\"]?([^'\"]+)['\"]?") AS loanType,
#   REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?") AS trenchCategory,
#    SAFE_CAST(REGEXP_EXTRACT(m.requestPayload_clean, r"aStackScore[:= ]([0-9\.]+)") AS FLOAT64) AS aStackScore,
#   SAFE_CAST(REGEXP_EXTRACT(m.requestPayload_clean, r"aCicScore[:= ]([0-9\.]+)") AS FLOAT64) AS aCicScore,
#   --  Alpha CIC Score Model Features for Trench 1
#   SAFE_CAST(JSON_VALUE(r.calcFeatures, "$.cic_max_age_all_contracts_snapshot") AS INT64) AS cic_max_age_all_contracts_snapshot,
#   SAFE_CAST(JSON_VALUE(r.calcFeatures, "$.cic_ratio_overdue_contracts_to_granted_contracts") AS FLOAT64) AS cic_ratio_overdue_contracts_to_granted_contracts,
#   JSON_VALUE(r.calcFeatures, "$.cic_ScoreRange") AS cic_ScoreRange,
#   JSON_VALUE(r.calcFeatures, "$.cic_ln_loan_level_user_type") AS cic_ln_loan_level_user_type,
#   JSON_VALUE(r.calcFeatures, "$.cic_has_ever_been_overdue") AS cic_has_ever_been_overdue,
#   JSON_VALUE(r.calcFeatures, "$.cic_latest_granted_contract_overdue_flag") AS cic_latest_granted_contract_overdue_flag,
#   JSON_VALUE(r.calcFeatures, "$.cic_ratio_closed_over_new_granted_cnt_24M") AS cic_ratio_closed_over_new_granted_cnt_24M,
#   JSON_VALUE(r.calcFeatures, "$.cic_ratio_risky_contracts_to_granted_contracts") AS cic_ratio_risky_contracts_to_granted_contracts,
#   JSON_VALUE(r.calcFeatures, "$.cic_Short_and_Term_Loans_granted_contracts_cnt_24M") AS cic_Short_and_Term_Loans_granted_contracts_cnt_24M,
#   JSON_VALUE(r.calcFeatures, "$.cic_flg_zero_non_granted_ever") AS cic_flg_zero_non_granted_ever,
#   JSON_VALUE(r.calcFeatures, "$.cic_Personal_Loans_granted_contracts_amt_24M") AS cic_Personal_Loans_granted_contracts_amt_24M,
#   JSON_VALUE(r.calcFeatures, "$.cic_CreditAvgCreditLimit") AS cic_CreditAvgCreditLimit,
#   JSON_VALUE(r.calcFeatures, "$.cic_flg_zero_granted_ever") AS cic_flg_zero_granted_ever,
#   coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
#   loanmaster.disbursementDateTime,
#   format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
# FROM latest_request r
# left join model_run m
# on r.digitalLoanAccountId = m.digitalLoanAccountId 
# left join risk_credit_mis.loan_master_table loanmaster 
#   ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
#  )
# where trenchCategory = 'Trench 1'
# )
# select *, case when appln_submit_datetime <= '2025-09-30' then 'Train' else 'Test' end dataselection from base
# ;
# """

# df = client.query(sq).to_dataframe()
# df.head()

### copy to df

In [26]:
df = expanded_df.drop(columns=['calcFeatures']).copy()

### Run the PSI calculation

In [27]:
df = df.copy()

# Define feature list
feature_list = ['aCicScore',
     'calc_cic_max_age_all_contracts_snapshot',
       'calc_cic_ratio_overdue_contracts_to_granted_contracts',
       'calc_cic_ScoreRange', 'calc_cic_ln_loan_level_user_type',
       'calc_cic_has_ever_been_overdue',
       'calc_cic_latest_granted_contract_overdue_flag',
       'calc_cic_ratio_closed_over_new_granted_cnt_24M',
       'calc_cic_ratio_risky_contracts_to_granted_contracts',
       'calc_cic_Short_and_Term_Loans_granted_contracts_cnt_24M',
       'calc_cic_flg_zero_non_granted_ever',
       'calc_cic_Personal_Loans_granted_contracts_amt_24M',
       'calc_cic_CreditAvgCreditLimit', 'calc_cic_flg_zero_granted_ever'
]

# Define segment columns
segment_columns = ['new_loan_type', 'osType']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_cic_model_psi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_cic_model_csi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=fee11e7e-ba0a-4bc9-a048-d23417282db4>

In [28]:
psi_results

,modelDisplayName,Model_Name,modelVersionId,trenchCategory,Feature,Feature_Type,Segment_Column,Segment_Value,Month,Base_Month,Current_Month,PSI
0,Alpha-Cash-CIC-Model,Alpha-Cash-CIC-Model_All_Trench,v1,Trench 3,aCicScore,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
1,Alpha-Cash-CIC-Model,Alpha-Cash-CIC-Model_All_Trench,v1,Trench 3,aCicScore,numerical,Overall,All,2025-10,2025-09,2025-10,0.360532
2,Alpha-Cash-CIC-Model,Alpha-Cash-CIC-Model_All_Trench,v1,Trench 3,aCicScore,numerical,Overall,All,2025-11,2025-09,2025-11,0.309356
3,Alpha-Cash-CIC-Model,Alpha-Cash-CIC-Model_All_Trench,v1,Trench 3,calc_cic_max_age_all_contracts_snapshot,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
4,Alpha-Cash-CIC-Model,Alpha-Cash-CIC-Model_All_Trench,v1,Trench 3,calc_cic_max_age_all_contracts_snapshot,numerical,Overall,All,2025-10,2025-09,2025-10,0.615051
...,...,...,...,...,...,...,...,...,...,...,...,...
163,Alpha-Cash-CIC-Model,Alpha-Cash-CIC-Model_All_Trench,v1,Trench 3,calc_cic_CreditAvgCreditLimit,numerical,osType,ios,2025-10,2025-09,2025-10,0.796462
164,Alpha-Cash-CIC-Model,Alpha-Cash-CIC-Model_All_Trench,v1,Trench 3,calc_cic_CreditAvgCreditLimit,numerical,osType,ios,2025-11,2025-09,2025-11,0.746040
165,Alpha-Cash-CIC-Model,Alpha-Cash-CIC-Model_All_Trench,v1,Trench 3,calc_cic_flg_zero_granted_ever,categorical,osType,ios,2025-09,2025-09,2025-09,0.000000
166,Alpha-Cash-CIC-Model,Alpha-Cash-CIC-Model_All_Trench,v1,Trench 3,calc_cic_flg_zero_granted_ever,categorical,osType,ios,2025-10,2025-09,2025-10,0.024315


## Alpha-Cash-Stack-Model

### Drop table temp_csi_new_monitoring_data

In [29]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_stack_model_psi;"""

job = client.query(sq)
job.result()  # Wait for job to complete
print(f"Table  prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_stack_model_psi dropped successfully.")

sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_stack_model_csi;"""

job = client.query(sq)
job.result()  # Wait for job to complete
print(f"Table prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_stack_model_csi dropped successfully.")

Table  prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_stack_model_psi dropped successfully.
Table prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_stack_model_csi dropped successfully.


## Trench1

In [30]:
sq = r""" 
WITH parsed as (
  select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
--REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName = 'Alpha-Cash-Stack-Model'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
--REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1),
base as (
select * from (
  select 
 r.customerId,
 r.digitalLoanAccountId,
 r.prediction ,
 r.start_time,
 r.end_time,
 r.modelDisplayName,
 r.modelVersionId,
 loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
 'Alpha-Cash-Stack-Model_Trench1' Model_Name,
 'Cash' as product,
  REGEXP_EXTRACT(m.requestPayload_clean, r"osType[:=]['\"]?([^'\"]+)['\"]?") AS osType,
  REGEXP_EXTRACT(m.requestPayload_clean, r"loanType[:=]['\"]?([^'\"]+)['\"]?") AS loanType,
  REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?") AS trenchCategory,
   SAFE_CAST(REGEXP_EXTRACT(m.requestPayload_clean, r"aStackScore[:= ]([0-9\.]+)") AS FLOAT64) AS aStackScore,
  SAFE_CAST(REGEXP_EXTRACT(m.requestPayload_clean, r"aCicScore[:= ]([0-9\.]+)") AS FLOAT64) AS aCicScore,
  calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
FROM latest_request r
left join model_run m
on r.digitalLoanAccountId = m.digitalLoanAccountId 
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by r.start_time desc) = 1
 )
)
select * from base where trenchCategory = 'Trench 1';
"""

dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,Model_Name,product,osType,loanType,trenchCategory,aStackScore,aCicScore,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,3688517,a15a2d70-06ab-48ae-b5f1-b4c7e7fc9cbb,0.08713585170462493,2025-10-13 05:56:07.762755,2025-10-13 05:56:07.925693,Alpha-Cash-Stack-Model,v1,Quick,F,not applicable,Alpha-Cash-Stack-Model_Trench1,Cash,android,Quick,Trench 1,0.087136,0.352887,"{""apps_score"": 0.3182877770451089, ""c_demo_sco...",2025-10-13 13:55:56,2025-10-15 11:09:28,2025-10
1,3726802,1792d116-2962-4209-98ee-b719dcd646e3,0.7689729070726964,2025-10-18 05:10:28.758299,2025-10-18 05:10:28.868348,Alpha-Cash-Stack-Model,v1,Quick,M,not applicable,Alpha-Cash-Stack-Model_Trench1,Cash,ios,Quick,Trench 1,0.768973,0.747946,"{""apps_score"": null, ""c_demo_score"": 0.5418075...",2025-10-18 13:10:17,NaT,2025-10
2,3732376,dadde442-c21b-49bc-899d-03692cd73059,0.449130362057625,2025-10-08 14:55:14.783709,2025-10-08 14:55:14.906723,Alpha-Cash-Stack-Model,v1,Quick,F,not applicable,Alpha-Cash-Stack-Model_Trench1,Cash,android,Quick,Trench 1,0.449130,0.510535,"{""apps_score"": 0.448076099580623, ""c_demo_scor...",2025-10-08 22:55:06,NaT,2025-10
3,3734572,be1a37de-9582-4b7f-9bfd-d65ca03dcfe9,0.25557099187686794,2025-10-10 01:20:50.956710,2025-10-10 01:20:51.113699,Alpha-Cash-Stack-Model,v1,Quick,F,not applicable,Alpha-Cash-Stack-Model_Trench1,Cash,android,Quick,Trench 1,0.255571,0.458981,"{""apps_score"": 0.40908745493661797, ""c_demo_sc...",2025-10-10 09:20:39,2025-10-10 11:26:37,2025-10
4,3737148,0f082edc-c2cc-424c-a81e-afb404199f67,0.8173447333081139,2025-10-11 12:51:03.669347,2025-10-11 12:51:03.761917,Alpha-Cash-Stack-Model,v1,Quick,M,not applicable,Alpha-Cash-Stack-Model_Trench1,Cash,android,Quick,Trench 1,0.817345,0.757970,"{""apps_score"": 0.5465938429044461, ""c_demo_sco...",2025-10-11 20:50:56,NaT,2025-10


### Expand df

In [31]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 21
Expanded columns: 26


,customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,Model_Name,product,osType,loanType,trenchCategory,aStackScore,aCicScore,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month,calc_apps_score,calc_c_demo_score,calc_c_credo_score,calc_c_tx_score,calc_ca_cic_score
0,3688517,a15a2d70-06ab-48ae-b5f1-b4c7e7fc9cbb,0.08713585170462493,2025-10-13 05:56:07.762755,2025-10-13 05:56:07.925693,Alpha-Cash-Stack-Model,v1,Quick,F,not applicable,Alpha-Cash-Stack-Model_Trench1,Cash,android,Quick,Trench 1,0.087136,0.352887,"{""apps_score"": 0.3182877770451089, ""c_demo_sco...",2025-10-13 13:55:56,2025-10-15 11:09:28,2025-10,0.318288,0.539578,0.1220,None,0.352887
1,3726802,1792d116-2962-4209-98ee-b719dcd646e3,0.7689729070726964,2025-10-18 05:10:28.758299,2025-10-18 05:10:28.868348,Alpha-Cash-Stack-Model,v1,Quick,M,not applicable,Alpha-Cash-Stack-Model_Trench1,Cash,ios,Quick,Trench 1,0.768973,0.747946,"{""apps_score"": null, ""c_demo_score"": 0.5418075...",2025-10-18 13:10:17,NaT,2025-10,NaN,0.541808,0.2040,None,0.747946
2,3732376,dadde442-c21b-49bc-899d-03692cd73059,0.449130362057625,2025-10-08 14:55:14.783709,2025-10-08 14:55:14.906723,Alpha-Cash-Stack-Model,v1,Quick,F,not applicable,Alpha-Cash-Stack-Model_Trench1,Cash,android,Quick,Trench 1,0.449130,0.510535,"{""apps_score"": 0.448076099580623, ""c_demo_scor...",2025-10-08 22:55:06,NaT,2025-10,0.448076,0.615720,0.2308,None,0.510535
3,3734572,be1a37de-9582-4b7f-9bfd-d65ca03dcfe9,0.25557099187686794,2025-10-10 01:20:50.956710,2025-10-10 01:20:51.113699,Alpha-Cash-Stack-Model,v1,Quick,F,not applicable,Alpha-Cash-Stack-Model_Trench1,Cash,android,Quick,Trench 1,0.255571,0.458981,"{""apps_score"": 0.40908745493661797, ""c_demo_sc...",2025-10-10 09:20:39,2025-10-10 11:26:37,2025-10,0.409087,0.471561,0.0482,None,0.458981
4,3737148,0f082edc-c2cc-424c-a81e-afb404199f67,0.8173447333081139,2025-10-11 12:51:03.669347,2025-10-11 12:51:03.761917,Alpha-Cash-Stack-Model,v1,Quick,M,not applicable,Alpha-Cash-Stack-Model_Trench1,Cash,android,Quick,Trench 1,0.817345,0.757970,"{""apps_score"": 0.5465938429044461, ""c_demo_sco...",2025-10-11 20:50:56,NaT,2025-10,0.546594,0.422912,0.2843,None,0.757970


In [32]:
df = expanded_df.drop(columns=['calcFeatures']).copy()

### run the psi calculation

In [33]:
df = df.copy()

# Define feature list
feature_list = ['aStackScore',  
                    'calc_apps_score', 
                    'calc_c_demo_score', 
                    'calc_c_credo_score', 
                    'calc_ca_cic_score'
]

# Define segment columns
segment_columns = ['new_loan_type','osType', ]
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_stack_model_psi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_stack_model_csi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=bb0334cf-25fb-4e2b-af00-b41351cf971a>

In [34]:
psi_results

,modelDisplayName,Model_Name,modelVersionId,trenchCategory,Feature,Feature_Type,Segment_Column,Segment_Value,Month,Base_Month,Current_Month,PSI
0,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench1,v1,Trench 1,aStackScore,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
1,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench1,v1,Trench 1,aStackScore,numerical,Overall,All,2025-10,2025-09,2025-10,0.162096
2,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench1,v1,Trench 1,aStackScore,numerical,Overall,All,2025-11,2025-09,2025-11,0.206575
3,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench1,v1,Trench 1,calc_apps_score,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
4,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench1,v1,Trench 1,calc_apps_score,numerical,Overall,All,2025-10,2025-09,2025-10,0.274044
5,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench1,v1,Trench 1,calc_apps_score,numerical,Overall,All,2025-11,2025-09,2025-11,0.362256
6,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench1,v1,Trench 1,calc_c_demo_score,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
7,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench1,v1,Trench 1,calc_c_demo_score,numerical,Overall,All,2025-10,2025-09,2025-10,0.072157
8,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench1,v1,Trench 1,calc_c_demo_score,numerical,Overall,All,2025-11,2025-09,2025-11,0.073856
9,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench1,v1,Trench 1,calc_c_credo_score,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000


## Alpha-Cash-Stack-Model_Trench2

## Trench2

In [35]:
sq = r""" 
WITH parsed as (
  select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName = 'Alpha-Cash-Stack-Model'),

latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)

select * from (
  select 
 r.customerId,
 r.digitalLoanAccountId,
 r.prediction aStackScore,
 r.start_time,
 r.end_time,
 r.modelDisplayName,
 r.modelVersionId,
 loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
'Alpha-Cash-Stack-Model_Trench2' Model_Name,
'Cash' as product,
  REGEXP_EXTRACT(m.requestPayload_clean, r"osType[:=]['\"]?([^'\"]+)['\"]?") AS osType,
  REGEXP_EXTRACT(m.requestPayload_clean, r"loanType[:=]['\"]?([^'\"]+)['\"]?") AS loanType,
  REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?") AS trenchCategory,
r.calcFeatures,
 coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,  
FROM latest_request r
left join model_run m
on r.digitalLoanAccountId = m.digitalLoanAccountId 
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by r.start_time desc) = 1
  
) where trenchCategory = 'Trench 2'
;
"""

dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,aStackScore,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,Model_Name,product,osType,loanType,trenchCategory,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,1008636,e592e5c9-43e0-4a8a-876e-c881320e1fd0,0.34459969905138377,2025-11-15 07:08:36.083429,2025-11-15 07:08:36.209109,Alpha-Cash-Stack-Model,v1,Quick,M,not applicable,Alpha-Cash-Stack-Model_Trench2,Cash,android,Quick,Trench 2,"{""apps_score"": 0.46035731506608, ""c_demo_score...",2025-11-15 15:08:15,2025-11-15 23:10:50,2025-11
1,1076155,608e3bd0-bd84-466d-8c93-9d2bed3097dd,0.7790113668238682,2025-11-14 08:35:53.019550,2025-11-14 08:35:53.262772,Alpha-Cash-Stack-Model,v1,Quick,F,not applicable,Alpha-Cash-Stack-Model_Trench2,Cash,android,Quick,Trench 2,"{""apps_score"": 0.561895576198325, ""c_demo_scor...",2025-11-14 16:35:40,NaT,2025-11
2,1167845,8bc731ea-64ef-4618-b7ea-a56b5eb3f3d3,0.16012053432926368,2025-11-14 23:38:51.194930,2025-11-14 23:38:51.750336,Alpha-Cash-Stack-Model,v1,Quick,F,not applicable,Alpha-Cash-Stack-Model_Trench2,Cash,ios,Quick,Trench 2,"{""apps_score"": null, ""c_demo_score"": 0.2817976...",2025-11-15 07:38:27,2025-11-17 05:05:36,2025-11
3,1186160,12ddcb24-33b0-4589-b8a8-64768d281f6f,0.611615902763221,2025-11-15 09:26:05.980000,2025-11-15 09:26:06.087336,Alpha-Cash-Stack-Model,v1,Quick,F,not applicable,Alpha-Cash-Stack-Model_Trench2,Cash,ios,Quick,Trench 2,"{""apps_score"": null, ""c_demo_score"": 0.5482552...",2025-11-15 17:25:59,NaT,2025-11
4,1201993,d325fe38-e4ab-4ad0-81ea-d50b83448ea7,0.541107410125863,2025-10-01 08:31:47.489007,2025-10-01 08:31:47.607561,Alpha-Cash-Stack-Model,v1,Quick,M,not applicable,Alpha-Cash-Stack-Model_Trench2,Cash,android,Quick,Trench 2,"{""apps_score"": 0.4593573472827892, ""c_demo_sco...",2025-10-01 16:31:38,NaT,2025-10


### Expand_df

In [36]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 19
Expanded columns: 24


,customerId,digitalLoanAccountId,aStackScore,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,Model_Name,product,osType,loanType,trenchCategory,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month,calc_apps_score,calc_c_demo_score,calc_c_credo_score,calc_c_tx_score,calc_ca_cic_score
0,1008636,e592e5c9-43e0-4a8a-876e-c881320e1fd0,0.34459969905138377,2025-11-15 07:08:36.083429,2025-11-15 07:08:36.209109,Alpha-Cash-Stack-Model,v1,Quick,M,not applicable,Alpha-Cash-Stack-Model_Trench2,Cash,android,Quick,Trench 2,"{""apps_score"": 0.46035731506608, ""c_demo_score...",2025-11-15 15:08:15,2025-11-15 23:10:50,2025-11,0.460357,0.484375,0.2987,0.458472,0.461234
1,1076155,608e3bd0-bd84-466d-8c93-9d2bed3097dd,0.7790113668238682,2025-11-14 08:35:53.019550,2025-11-14 08:35:53.262772,Alpha-Cash-Stack-Model,v1,Quick,F,not applicable,Alpha-Cash-Stack-Model_Trench2,Cash,android,Quick,Trench 2,"{""apps_score"": 0.561895576198325, ""c_demo_scor...",2025-11-14 16:35:40,NaT,2025-11,0.561896,0.600556,0.2910,0.566004,0.613644
2,1167845,8bc731ea-64ef-4618-b7ea-a56b5eb3f3d3,0.16012053432926368,2025-11-14 23:38:51.194930,2025-11-14 23:38:51.750336,Alpha-Cash-Stack-Model,v1,Quick,F,not applicable,Alpha-Cash-Stack-Model_Trench2,Cash,ios,Quick,Trench 2,"{""apps_score"": null, ""c_demo_score"": 0.2817976...",2025-11-15 07:38:27,2025-11-17 05:05:36,2025-11,NaN,0.281798,0.1765,0.534801,0.418072
3,1186160,12ddcb24-33b0-4589-b8a8-64768d281f6f,0.611615902763221,2025-11-15 09:26:05.980000,2025-11-15 09:26:06.087336,Alpha-Cash-Stack-Model,v1,Quick,F,not applicable,Alpha-Cash-Stack-Model_Trench2,Cash,ios,Quick,Trench 2,"{""apps_score"": null, ""c_demo_score"": 0.5482552...",2025-11-15 17:25:59,NaT,2025-11,NaN,0.548255,0.1517,0.586048,0.614365
4,1201993,d325fe38-e4ab-4ad0-81ea-d50b83448ea7,0.541107410125863,2025-10-01 08:31:47.489007,2025-10-01 08:31:47.607561,Alpha-Cash-Stack-Model,v1,Quick,M,not applicable,Alpha-Cash-Stack-Model_Trench2,Cash,android,Quick,Trench 2,"{""apps_score"": 0.4593573472827892, ""c_demo_sco...",2025-10-01 16:31:38,NaT,2025-10,0.459357,0.574213,0.2936,0.543809,0.619431


In [37]:
df = expanded_df.drop(columns=['calcFeatures']).copy()

### run the psi calculation

In [38]:
df = df.copy()

# Define feature list
feature_list = ['aStackScore',  
                   'calc_apps_score', 'calc_c_demo_score',
       'calc_c_credo_score', 'calc_ca_cic_score'
]

# Define segment columns
segment_columns = ['new_loan_type', 'osType']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',   'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_stack_model_psi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_stack_model_csi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=60b21bd4-69a4-489f-b0cc-9fa2189441d4>

In [39]:
psi_results

,modelDisplayName,Model_Name,modelVersionId,trenchCategory,Feature,Feature_Type,Segment_Column,Segment_Value,Month,Base_Month,Current_Month,PSI
0,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench2,v1,Trench 2,aStackScore,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000
1,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench2,v1,Trench 2,aStackScore,categorical,Overall,All,2025-10,2025-09,2025-10,1.156046
2,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench2,v1,Trench 2,aStackScore,categorical,Overall,All,2025-11,2025-09,2025-11,1.156046
3,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench2,v1,Trench 2,calc_apps_score,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
4,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench2,v1,Trench 2,calc_apps_score,numerical,Overall,All,2025-10,2025-09,2025-10,0.109546
5,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench2,v1,Trench 2,calc_apps_score,numerical,Overall,All,2025-11,2025-09,2025-11,0.157700
6,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench2,v1,Trench 2,calc_c_demo_score,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
7,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench2,v1,Trench 2,calc_c_demo_score,numerical,Overall,All,2025-10,2025-09,2025-10,0.361758
8,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench2,v1,Trench 2,calc_c_demo_score,numerical,Overall,All,2025-11,2025-09,2025-11,0.258820
9,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench2,v1,Trench 2,calc_c_credo_score,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000


## Alpha-Cash-Stack-Model_Trench3

## Trench3

In [40]:
sq = r""" 
WITH parsed as (
  select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName = 'Alpha-Cash-Stack-Model'),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
select * from (
  select 
 r.customerId,
 r.digitalLoanAccountId,
 r.prediction aStackScore,
 r.start_time,
 r.end_time,
 r.modelDisplayName,
 r.modelVersionId,
 loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
 'Alpha-Cash-Stack-Model_Trench3' Model_Name,
 'Cash' as product,
  REGEXP_EXTRACT(m.requestPayload_clean, r"osType[:=]['\"]?([^'\"]+)['\"]?") AS osType,
  REGEXP_EXTRACT(m.requestPayload_clean, r"loanType[:=]['\"]?([^'\"]+)['\"]?") AS loanType,
  REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?") AS trenchCategory,
 r.calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,  
FROM latest_request r
left join model_run m
on r.digitalLoanAccountId = m.digitalLoanAccountId 
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by r.start_time desc) = 1
) where trenchCategory = 'Trench 3'
;
"""

dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,aStackScore,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,Model_Name,product,osType,loanType,trenchCategory,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,2418549,0715cbd1-89ed-4739-ab6d-6793734e995f,0.25839981584724175,2025-11-18 02:22:06.981322,2025-11-18 02:22:07.640004,Alpha-Cash-Stack-Model,v1,None,None,not applicable,Alpha-Cash-Stack-Model_Trench3,Cash,android,Quick,Trench 3,"{""apps_score"": 0.46437442487752445, ""c_demo_sc...",2025-11-18 02:22:06.981322,NaT,2025-11
1,3187613,9e2c779b-90b7-451e-8a63-ce6cff37ab66,0.3339123021594783,2025-11-18 03:06:54.949772,2025-11-18 03:06:55.059778,Alpha-Cash-Stack-Model,v1,None,None,not applicable,Alpha-Cash-Stack-Model_Trench3,Cash,android,Quick,Trench 3,"{""apps_score"": 0.47611126672267895, ""c_demo_sc...",2025-11-18 03:06:54.949772,NaT,2025-11
2,2045382,4e6914c7-04b6-4572-9b2f-e7cdd704270a,0.5188233602675918,2025-11-18 03:02:02.211313,2025-11-18 03:02:02.456420,Alpha-Cash-Stack-Model,v1,None,None,not applicable,Alpha-Cash-Stack-Model_Trench3,Cash,ios,Quick,Trench 3,"{""apps_score"": null, ""c_demo_score"": 0.5587199...",2025-11-18 03:02:02.211313,NaT,2025-11
3,3385696,a112266c-bbe5-4092-8289-b528d6e0e5d9,0.4398303947154668,2025-11-18 02:29:13.542212,2025-11-18 02:29:13.667338,Alpha-Cash-Stack-Model,v1,None,None,not applicable,Alpha-Cash-Stack-Model_Trench3,Cash,ios,Quick,Trench 3,"{""apps_score"": null, ""c_demo_score"": 0.4360624...",2025-11-18 02:29:13.542212,NaT,2025-11
4,3525503,c39ab164-762b-4029-9a5e-2ad552fe9a26,0.677693796637115,2025-11-18 02:17:01.137547,2025-11-18 02:17:01.260623,Alpha-Cash-Stack-Model,v1,None,None,not applicable,Alpha-Cash-Stack-Model_Trench3,Cash,android,Quick,Trench 3,"{""apps_score"": 0.5151803650719444, ""c_demo_sco...",2025-11-18 02:17:01.137547,NaT,2025-11


### Expand_df

In [41]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 19
Expanded columns: 24


,customerId,digitalLoanAccountId,aStackScore,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,Model_Name,product,osType,loanType,trenchCategory,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month,calc_apps_score,calc_c_demo_score,calc_c_credo_score,calc_c_tx_score,calc_ca_cic_score
0,2418549,0715cbd1-89ed-4739-ab6d-6793734e995f,0.25839981584724175,2025-11-18 02:22:06.981322,2025-11-18 02:22:07.640004,Alpha-Cash-Stack-Model,v1,None,None,not applicable,Alpha-Cash-Stack-Model_Trench3,Cash,android,Quick,Trench 3,"{""apps_score"": 0.46437442487752445, ""c_demo_sc...",2025-11-18 02:22:06.981322,NaT,2025-11,0.464374,0.355024,0.0997,0.419543,0.340125
1,3187613,9e2c779b-90b7-451e-8a63-ce6cff37ab66,0.3339123021594783,2025-11-18 03:06:54.949772,2025-11-18 03:06:55.059778,Alpha-Cash-Stack-Model,v1,None,None,not applicable,Alpha-Cash-Stack-Model_Trench3,Cash,android,Quick,Trench 3,"{""apps_score"": 0.47611126672267895, ""c_demo_sc...",2025-11-18 03:06:54.949772,NaT,2025-11,0.476111,0.465198,0.0557,0.439197,0.447614
2,2045382,4e6914c7-04b6-4572-9b2f-e7cdd704270a,0.5188233602675918,2025-11-18 03:02:02.211313,2025-11-18 03:02:02.456420,Alpha-Cash-Stack-Model,v1,None,None,not applicable,Alpha-Cash-Stack-Model_Trench3,Cash,ios,Quick,Trench 3,"{""apps_score"": null, ""c_demo_score"": 0.5587199...",2025-11-18 03:02:02.211313,NaT,2025-11,NaN,0.558720,0.1221,0.550980,0.415356
3,3385696,a112266c-bbe5-4092-8289-b528d6e0e5d9,0.4398303947154668,2025-11-18 02:29:13.542212,2025-11-18 02:29:13.667338,Alpha-Cash-Stack-Model,v1,None,None,not applicable,Alpha-Cash-Stack-Model_Trench3,Cash,ios,Quick,Trench 3,"{""apps_score"": null, ""c_demo_score"": 0.4360624...",2025-11-18 02:29:13.542212,NaT,2025-11,NaN,0.436062,0.1013,0.523564,0.419925
4,3525503,c39ab164-762b-4029-9a5e-2ad552fe9a26,0.677693796637115,2025-11-18 02:17:01.137547,2025-11-18 02:17:01.260623,Alpha-Cash-Stack-Model,v1,None,None,not applicable,Alpha-Cash-Stack-Model_Trench3,Cash,android,Quick,Trench 3,"{""apps_score"": 0.5151803650719444, ""c_demo_sco...",2025-11-18 02:17:01.137547,NaT,2025-11,0.515180,0.428077,0.0894,0.618576,0.606740


In [42]:
df = expanded_df.drop(columns=['calcFeatures']).copy()

### run the psi calculation

In [43]:
df = df.copy()

# Define feature list
feature_list = ['aStackScore', 'calc_apps_score', 'calc_c_demo_score', 'calc_c_credo_score', 
                # 'calc_c_tx_score', 
                'calc_ca_cic_score']

# Define segment columns
segment_columns = ['new_loan_type', 'osType']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',   'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_stack_model_psi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_stack_model_csi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=35e13097-3a61-437e-a0bb-524df3f9d9f3>

In [44]:
psi_results

,modelDisplayName,Model_Name,modelVersionId,trenchCategory,Feature,Feature_Type,Segment_Column,Segment_Value,Month,Base_Month,Current_Month,PSI
0,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench3,v1,Trench 3,aStackScore,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000
1,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench3,v1,Trench 3,aStackScore,categorical,Overall,All,2025-10,2025-09,2025-10,2.400307
2,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench3,v1,Trench 3,aStackScore,categorical,Overall,All,2025-11,2025-09,2025-11,2.400307
3,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench3,v1,Trench 3,calc_apps_score,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
4,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench3,v1,Trench 3,calc_apps_score,numerical,Overall,All,2025-10,2025-09,2025-10,1.968488
5,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench3,v1,Trench 3,calc_apps_score,numerical,Overall,All,2025-11,2025-09,2025-11,1.901704
6,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench3,v1,Trench 3,calc_c_demo_score,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
7,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench3,v1,Trench 3,calc_c_demo_score,numerical,Overall,All,2025-10,2025-09,2025-10,0.116396
8,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench3,v1,Trench 3,calc_c_demo_score,numerical,Overall,All,2025-11,2025-09,2025-11,0.156623
9,Alpha-Cash-Stack-Model,Alpha-Cash-Stack-Model_Trench3,v1,Trench 3,calc_c_credo_score,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000


## Beta-Cash-Demo-Model (Trench1,2,3)

In [45]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_demo_model_psi;"""
job = client.query(sq)
job.result()  # Wait for job to complete
print(f"Table  prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_demo_model_psi dropped successfully.")

sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_demo_model_csi;"""
job = client.query(sq)
job.result()  # Wait for job to complete
print(f"Table prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_demo_model_csi dropped successfully.")

Table  prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_demo_model_psi dropped successfully.
Table prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_demo_model_csi dropped successfully.


## Trench 1

In [46]:
sq = r"""
WITH parsed as (
  select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName = 'Beta-Cash-Demo-Model'),

latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)

select * from (select 
 r.customerId,
 r.digitalLoanAccountId,
 r.prediction Beta_Cash_Demo_Score,
 r.start_time,
 r.end_time,
 r.modelDisplayName,
 r.modelVersionId,
 loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
 case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
        when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
        when lower(loanmaster.deviceType) like '%andro%' then 'android'
        else 'ios' end osType,
  'Beta-Cash-Demo-Model_All_Trench' Model_Name,
  'Cash' as product,
  JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory") AS trenchCategory,
  JSON_VALUE(m.requestPayload_clean, "$.predictions.loanType") AS loanType,
    --  Demo Score Model Features for all Trench 1,2,3
  calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
FROM latest_request r
left join model_run m
on r.digitalLoanAccountId = m.digitalLoanAccountId 
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by r.start_time desc) = 1
)
where trenchCategory = 'Trench 1'
;
"""

dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,Beta_Cash_Demo_Score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,loanType,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,3683489,fb9aa32e-8203-4ad2-bb8e-eff984b833ca,0.528966824358384,2025-10-09 18:11:03.332130,2025-10-09 18:11:03.344913,Beta-Cash-Demo-Model,v1,Quick,F,not applicable,ios,Beta-Cash-Demo-Model_All_Trench,Cash,Trench 1,Quick,"{""ln_vas_opted_flag"": 1, ""ln_self_dec_income"":...",2025-10-10 02:11:01,2025-10-10 20:27:40,2025-10
1,3695875,e781309e-7f5d-48e1-8301-b4dfa3259d30,0.6736363064063619,2025-10-10 04:51:10.546042,2025-10-10 04:51:10.805035,Beta-Cash-Demo-Model,v1,Quick,M,not applicable,android,Beta-Cash-Demo-Model_All_Trench,Cash,Trench 1,Quick,"{""ln_vas_opted_flag"": 1, ""ln_self_dec_income"":...",2025-10-10 12:51:07,NaT,2025-10
2,3703772,0a3cb3a3-05d4-4f25-8655-7285905c32e7,0.6494739579410224,2025-09-25 00:53:26.222629,2025-09-25 00:53:26.462761,Beta-Cash-Demo-Model,v1,Quick,M,not applicable,android,Beta-Cash-Demo-Model_All_Trench,Cash,Trench 1,Quick,"{""ln_vas_opted_flag"": 1, ""ln_self_dec_income"":...",2025-09-25 08:53:24,NaT,2025-09
3,3711907,5ab60286-3566-4886-8d5e-23aee90c0964,0.521391204244882,2025-09-30 07:46:28.261076,2025-09-30 07:46:28.501704,Beta-Cash-Demo-Model,v1,Quick,M,not applicable,android,Beta-Cash-Demo-Model_All_Trench,Cash,Trench 1,Quick,"{""ln_vas_opted_flag"": 0, ""ln_self_dec_income"":...",2025-09-30 15:46:24,NaT,2025-09
4,3713890,4d54687d-afba-4dc1-a58e-c733efa4abe7,0.607090057882049,2025-09-29 19:21:44.838330,2025-09-29 19:21:45.077182,Beta-Cash-Demo-Model,v1,Quick,F,not applicable,android,Beta-Cash-Demo-Model_All_Trench,Cash,Trench 1,Quick,"{""ln_vas_opted_flag"": 1, ""ln_self_dec_income"":...",2025-09-30 03:21:42,NaT,2025-09


### expand df

In [47]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 19
Expanded columns: 35


,customerId,digitalLoanAccountId,Beta_Cash_Demo_Score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,loanType,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month,calc_ln_vas_opted_flag,calc_ln_self_dec_income,calc_ln_age,calc_ln_source_funds_new_bin,calc_ln_loan_level_user_type,calc_ln_industry_new_cat_bin,calc_ln_marital_status,calc_ln_doc_type_rolled,calc_ln_education_level,calc_ln_ref2_type,calc_ln_email_primary_domain,calc_ln_os_type,calc_ln_mature_fspd30_flag,calc_ln_fspd30_flag,calc_ln_appln_submit_datetime,calc_ln_province_bin
0,3683489,fb9aa32e-8203-4ad2-bb8e-eff984b833ca,0.528966824358384,2025-10-09 18:11:03.332130,2025-10-09 18:11:03.344913,Beta-Cash-Demo-Model,v1,Quick,F,not applicable,ios,Beta-Cash-Demo-Model_All_Trench,Cash,Trench 1,Quick,"{""ln_vas_opted_flag"": 1, ""ln_self_dec_income"":...",2025-10-10 02:11:01,2025-10-10 20:27:40,2025-10,1,30000,28,salary,2_New Applicant,3.0,Single,less_frequent_cat,College Graduate,Friend,gmail.com,1,None,None,2025-10-09T18:11:01+00:00,others
1,3695875,e781309e-7f5d-48e1-8301-b4dfa3259d30,0.6736363064063619,2025-10-10 04:51:10.546042,2025-10-10 04:51:10.805035,Beta-Cash-Demo-Model,v1,Quick,M,not applicable,android,Beta-Cash-Demo-Model_All_Trench,Cash,Trench 1,Quick,"{""ln_vas_opted_flag"": 1, ""ln_self_dec_income"":...",2025-10-10 12:51:07,NaT,2025-10,1,10000,22,less_frequent_cat,2_New Applicant,missing,Single,driving license,College Graduate,Parent,gmail.com,0,None,None,2025-10-10T04:51:07+00:00,others
2,3703772,0a3cb3a3-05d4-4f25-8655-7285905c32e7,0.6494739579410224,2025-09-25 00:53:26.222629,2025-09-25 00:53:26.462761,Beta-Cash-Demo-Model,v1,Quick,M,not applicable,android,Beta-Cash-Demo-Model_All_Trench,Cash,Trench 1,Quick,"{""ln_vas_opted_flag"": 1, ""ln_self_dec_income"":...",2025-09-25 08:53:24,NaT,2025-09,1,45000,33,income from business,2_New Applicant,2.0,Single,driving license,College Graduate,Sibling,gmail.com,0,None,None,2025-09-25T00:53:24+00:00,metro manila
3,3711907,5ab60286-3566-4886-8d5e-23aee90c0964,0.521391204244882,2025-09-30 07:46:28.261076,2025-09-30 07:46:28.501704,Beta-Cash-Demo-Model,v1,Quick,M,not applicable,android,Beta-Cash-Demo-Model_All_Trench,Cash,Trench 1,Quick,"{""ln_vas_opted_flag"": 0, ""ln_self_dec_income"":...",2025-09-30 15:46:24,NaT,2025-09,0,22000,27,salary,2_New Applicant,2.0,Single,driving license,High School Graduate,Friend,gmail.com,0,None,None,2025-09-30T07:46:24+00:00,cavite
4,3713890,4d54687d-afba-4dc1-a58e-c733efa4abe7,0.607090057882049,2025-09-29 19:21:44.838330,2025-09-29 19:21:45.077182,Beta-Cash-Demo-Model,v1,Quick,F,not applicable,android,Beta-Cash-Demo-Model_All_Trench,Cash,Trench 1,Quick,"{""ln_vas_opted_flag"": 1, ""ln_self_dec_income"":...",2025-09-30 03:21:42,NaT,2025-09,1,28000,32,salary,2_New Applicant,1.0,Single,passport,College Graduate,Co-worker,gmail.com,0,None,None,2025-09-29T19:21:42+00:00,pampanga


In [48]:
df = expanded_df.drop(columns=['calcFeatures']).copy()

In [49]:
bin_columns = [col for col in df.columns if 'bin' in col.lower()]
df[bin_columns] = df[bin_columns].astype('category')

### run the psi calculation

In [50]:
df = df.copy()

# Define feature list
feature_list = ['Beta_Cash_Demo_Score',  
                    'calc_ln_vas_opted_flag', 'calc_ln_self_dec_income', 'calc_ln_age',
       'calc_ln_source_funds_new_bin', 'calc_ln_loan_level_user_type',
       'calc_ln_industry_new_cat_bin', 'calc_ln_marital_status',
       'calc_ln_doc_type_rolled', 'calc_ln_education_level',
       'calc_ln_ref2_type', 'calc_ln_email_primary_domain', 'calc_ln_os_type', 'calc_ln_province_bin' 
]

# Define segment columns
segment_columns = ['new_loan_type','osType']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_demo_model_psi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_demo_model_csi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=7ac850d6-3ecf-448d-90d5-700763771b49>

In [51]:
psi_results

,modelDisplayName,Model_Name,modelVersionId,trenchCategory,Feature,Feature_Type,Segment_Column,Segment_Value,Month,Base_Month,Current_Month,PSI
0,Beta-Cash-Demo-Model,Beta-Cash-Demo-Model_All_Trench,v1,Trench 1,Beta_Cash_Demo_Score,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000
1,Beta-Cash-Demo-Model,Beta-Cash-Demo-Model_All_Trench,v1,Trench 1,Beta_Cash_Demo_Score,categorical,Overall,All,2025-10,2025-09,2025-10,0.021295
2,Beta-Cash-Demo-Model,Beta-Cash-Demo-Model_All_Trench,v1,Trench 1,Beta_Cash_Demo_Score,categorical,Overall,All,2025-11,2025-09,2025-11,0.021995
3,Beta-Cash-Demo-Model,Beta-Cash-Demo-Model_All_Trench,v1,Trench 1,calc_ln_vas_opted_flag,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000
4,Beta-Cash-Demo-Model,Beta-Cash-Demo-Model_All_Trench,v1,Trench 1,calc_ln_vas_opted_flag,categorical,Overall,All,2025-10,2025-09,2025-10,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
163,Beta-Cash-Demo-Model,Beta-Cash-Demo-Model_All_Trench,v1,Trench 1,calc_ln_os_type,categorical,osType,android,2025-10,2025-09,2025-10,0.000000
164,Beta-Cash-Demo-Model,Beta-Cash-Demo-Model_All_Trench,v1,Trench 1,calc_ln_os_type,categorical,osType,android,2025-11,2025-09,2025-11,0.000000
165,Beta-Cash-Demo-Model,Beta-Cash-Demo-Model_All_Trench,v1,Trench 1,calc_ln_province_bin,categorical,osType,android,2025-09,2025-09,2025-09,0.000000
166,Beta-Cash-Demo-Model,Beta-Cash-Demo-Model_All_Trench,v1,Trench 1,calc_ln_province_bin,categorical,osType,android,2025-10,2025-09,2025-10,0.013750


## Trench 2

In [52]:
sq = r"""
WITH parsed as (
  select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName = 'Beta-Cash-Demo-Model'),

latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)

select * from (select 
 r.customerId,
 r.digitalLoanAccountId,
 r.prediction Beta_Cash_Demo_Score,
 r.start_time,
 r.end_time,
 r.modelDisplayName,
 r.modelVersionId,
 loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
  case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
        when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
        when lower(loanmaster.deviceType) like '%andro%' then 'android'
        else 'ios' end osType,
  'Beta-Cash-Demo-Model_All_Trench' Model_Name,
  'Cash' as product,
  JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory") AS trenchCategory,
  JSON_VALUE(m.requestPayload_clean, "$.predictions.loanType") AS loanType,
  --  Demo Score Model Features for all Trench 1,2,3
  calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
FROM latest_request r
left join model_run m
on r.digitalLoanAccountId = m.digitalLoanAccountId 
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by r.start_time desc) = 1
)
where trenchCategory = 'Trench 2'
;
"""

dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,Beta_Cash_Demo_Score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,loanType,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,1024439,8daba0dd-1da6-4bbe-ab71-11d0ae26df5d,0.48302960271466705,2025-10-11 02:50:11.184008,2025-10-11 02:50:11.731140,Beta-Cash-Demo-Model,v1,Quick,F,not applicable,ios,Beta-Cash-Demo-Model_All_Trench,Cash,Trench 2,Quick,"{""ln_vas_opted_flag"": 0, ""ln_self_dec_income"":...",2025-10-11 10:50:07,NaT,2025-10
1,1085372,ff2b4105-0363-41ce-a9bd-7d793ec24868,0.5990524652593161,2025-11-13 08:00:44.597771,2025-11-13 08:00:45.364714,Beta-Cash-Demo-Model,v1,Quick,M,not applicable,android,Beta-Cash-Demo-Model_All_Trench,Cash,Trench 2,Quick,"{""ln_vas_opted_flag"": 1, ""ln_self_dec_income"":...",2025-11-13 16:00:39,2025-11-13 16:13:27,2025-11
2,1091745,b5431917-44aa-40d6-9813-81659e6cf9f7,0.6663020840182471,2025-11-12 15:22:16.725400,2025-11-12 15:22:17.594796,Beta-Cash-Demo-Model,v1,Quick,M,not applicable,android,Beta-Cash-Demo-Model_All_Trench,Cash,Trench 2,Quick,"{""ln_vas_opted_flag"": 1, ""ln_self_dec_income"":...",2025-11-12 23:22:14,NaT,2025-11
3,1111683,7850f074-82cd-47a1-9708-3936a676db91,0.5771148612736594,2025-11-03 10:54:56.165671,2025-11-03 10:54:57.037919,Beta-Cash-Demo-Model,v1,Quick,F,not applicable,android,Beta-Cash-Demo-Model_All_Trench,Cash,Trench 2,Quick,"{""ln_vas_opted_flag"": 1, ""ln_self_dec_income"":...",2025-11-03 18:54:52,NaT,2025-11
4,1121374,58c474c4-82d8-4390-bf82-5d5113c24900,0.5730341833041995,2025-11-15 10:04:14.210702,2025-11-15 10:04:14.998032,Beta-Cash-Demo-Model,v1,Quick,F,not applicable,android,Beta-Cash-Demo-Model_All_Trench,Cash,Trench 2,Quick,"{""ln_vas_opted_flag"": 1, ""ln_self_dec_income"":...",2025-11-15 18:04:11,NaT,2025-11


### expand df

In [53]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 19
Expanded columns: 35


,customerId,digitalLoanAccountId,Beta_Cash_Demo_Score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,loanType,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month,calc_ln_vas_opted_flag,calc_ln_self_dec_income,calc_ln_age,calc_ln_source_funds_new_bin,calc_ln_loan_level_user_type,calc_ln_industry_new_cat_bin,calc_ln_marital_status,calc_ln_doc_type_rolled,calc_ln_education_level,calc_ln_ref2_type,calc_ln_email_primary_domain,calc_ln_os_type,calc_ln_mature_fspd30_flag,calc_ln_fspd30_flag,calc_ln_appln_submit_datetime,calc_ln_province_bin
0,1024439,8daba0dd-1da6-4bbe-ab71-11d0ae26df5d,0.48302960271466705,2025-10-11 02:50:11.184008,2025-10-11 02:50:11.731140,Beta-Cash-Demo-Model,v1,Quick,F,not applicable,ios,Beta-Cash-Demo-Model_All_Trench,Cash,Trench 2,Quick,"{""ln_vas_opted_flag"": 0, ""ln_self_dec_income"":...",2025-10-11 10:50:07,NaT,2025-10,0,50000,24,salary,2_New Applicant,3.0,Single,passport,College Undergraduate,Spouse,gmail.com,1,None,None,2025-10-11T02:50:07+00:00,metro manila
1,1085372,ff2b4105-0363-41ce-a9bd-7d793ec24868,0.5990524652593161,2025-11-13 08:00:44.597771,2025-11-13 08:00:45.364714,Beta-Cash-Demo-Model,v1,Quick,M,not applicable,android,Beta-Cash-Demo-Model_All_Trench,Cash,Trench 2,Quick,"{""ln_vas_opted_flag"": 1, ""ln_self_dec_income"":...",2025-11-13 16:00:39,2025-11-13 16:13:27,2025-11,1,35200,30,salary,2_New Applicant,1.0,Single,umid,College Graduate,Sibling,gmail.com,0,None,None,2025-11-13T08:00:39+00:00,others
2,1091745,b5431917-44aa-40d6-9813-81659e6cf9f7,0.6663020840182471,2025-11-12 15:22:16.725400,2025-11-12 15:22:17.594796,Beta-Cash-Demo-Model,v1,Quick,M,not applicable,android,Beta-Cash-Demo-Model_All_Trench,Cash,Trench 2,Quick,"{""ln_vas_opted_flag"": 1, ""ln_self_dec_income"":...",2025-11-12 23:22:14,NaT,2025-11,1,60000,29,income from business,2_New Applicant,1.0,Single,umid,College Graduate,Friend,gmail.com,0,None,None,2025-11-12T15:22:14+00:00,bulacan
3,1111683,7850f074-82cd-47a1-9708-3936a676db91,0.5771148612736594,2025-11-03 10:54:56.165671,2025-11-03 10:54:57.037919,Beta-Cash-Demo-Model,v1,Quick,F,not applicable,android,Beta-Cash-Demo-Model_All_Trench,Cash,Trench 2,Quick,"{""ln_vas_opted_flag"": 1, ""ln_self_dec_income"":...",2025-11-03 18:54:52,NaT,2025-11,1,28000,36,salary,2_New Applicant,2.0,Single,umid,College Graduate,Parent,gmail.com,0,None,None,2025-11-03T10:54:52+00:00,others
4,1121374,58c474c4-82d8-4390-bf82-5d5113c24900,0.5730341833041995,2025-11-15 10:04:14.210702,2025-11-15 10:04:14.998032,Beta-Cash-Demo-Model,v1,Quick,F,not applicable,android,Beta-Cash-Demo-Model_All_Trench,Cash,Trench 2,Quick,"{""ln_vas_opted_flag"": 1, ""ln_self_dec_income"":...",2025-11-15 18:04:11,NaT,2025-11,1,19000,23,salary,2_New Applicant,3.0,Single,national id,College Graduate,Parent,gmail.com,0,None,None,2025-11-15T10:04:11+00:00,bulacan


In [54]:
df = expanded_df.drop(columns=['calcFeatures']).copy()

In [55]:
bin_columns = [col for col in df.columns if 'bin' in col.lower()]
df[bin_columns] = df[bin_columns].astype('category')

### run the psi calculation

In [56]:
df = df.copy()

# Define feature list
feature_list = ['Beta_Cash_Demo_Score',  
                    'calc_ln_vas_opted_flag', 'calc_ln_self_dec_income', 'calc_ln_age',
       'calc_ln_source_funds_new_bin', 'calc_ln_loan_level_user_type',
       'calc_ln_industry_new_cat_bin', 'calc_ln_marital_status',
       'calc_ln_doc_type_rolled', 'calc_ln_education_level',
       'calc_ln_ref2_type', 'calc_ln_email_primary_domain', 'calc_ln_os_type', 'calc_ln_province_bin' 
]

# Define segment columns
segment_columns = ['new_loan_type','osType']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_demo_model_psi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_demo_model_csi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=cd19a978-d98b-4756-956a-0e63394f35fe>

In [57]:
psi_results

,modelDisplayName,Model_Name,modelVersionId,trenchCategory,Feature,Feature_Type,Segment_Column,Segment_Value,Month,Base_Month,Current_Month,PSI
0,Beta-Cash-Demo-Model,Beta-Cash-Demo-Model_All_Trench,v1,Trench 2,Beta_Cash_Demo_Score,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000
1,Beta-Cash-Demo-Model,Beta-Cash-Demo-Model_All_Trench,v1,Trench 2,Beta_Cash_Demo_Score,categorical,Overall,All,2025-10,2025-09,2025-10,0.174229
2,Beta-Cash-Demo-Model,Beta-Cash-Demo-Model_All_Trench,v1,Trench 2,Beta_Cash_Demo_Score,categorical,Overall,All,2025-11,2025-09,2025-11,0.182227
3,Beta-Cash-Demo-Model,Beta-Cash-Demo-Model_All_Trench,v1,Trench 2,calc_ln_vas_opted_flag,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000
4,Beta-Cash-Demo-Model,Beta-Cash-Demo-Model_All_Trench,v1,Trench 2,calc_ln_vas_opted_flag,categorical,Overall,All,2025-10,2025-09,2025-10,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
163,Beta-Cash-Demo-Model,Beta-Cash-Demo-Model_All_Trench,v1,Trench 2,calc_ln_os_type,categorical,osType,android,2025-10,2025-09,2025-10,0.020199
164,Beta-Cash-Demo-Model,Beta-Cash-Demo-Model_All_Trench,v1,Trench 2,calc_ln_os_type,categorical,osType,android,2025-11,2025-09,2025-11,0.017965
165,Beta-Cash-Demo-Model,Beta-Cash-Demo-Model_All_Trench,v1,Trench 2,calc_ln_province_bin,categorical,osType,android,2025-09,2025-09,2025-09,0.000000
166,Beta-Cash-Demo-Model,Beta-Cash-Demo-Model_All_Trench,v1,Trench 2,calc_ln_province_bin,categorical,osType,android,2025-10,2025-09,2025-10,0.028847


## Trench 3

In [58]:
sq = r"""
WITH parsed as (
  select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName = 'Beta-Cash-Demo-Model'),

latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)

select * from (select 
 r.customerId,
 r.digitalLoanAccountId,
 r.prediction Beta_Cash_Demo_Score,
 r.start_time,
 r.end_time,
 r.modelDisplayName,
 r.modelVersionId,
 loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
 case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
        when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
        when lower(loanmaster.deviceType) like '%andro%' then 'android'
        else 'ios' end osType,
  'Beta-Cash-Demo-Model_All_Trench' Model_Name,
  'Cash' as product,
  JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory") AS trenchCategory,
  JSON_VALUE(m.requestPayload_clean, "$.predictions.loanType") AS loanType,
  --  Demo Score Model Features for all Trench 1,2,3
  calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
FROM latest_request r
left join model_run m
on r.digitalLoanAccountId = m.digitalLoanAccountId 
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by r.start_time desc) = 1
)
where trenchCategory = 'Trench 3'
;
"""

dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,Beta_Cash_Demo_Score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,loanType,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,1360407,caf7ec35-5d38-4bcf-98f2-52f1abc20f25,0.412737396573637,2025-10-23 22:49:29.257538,2025-10-23 22:49:30.031409,Beta-Cash-Demo-Model,v1,Quick,M,not applicable,android,Beta-Cash-Demo-Model_All_Trench,Cash,Trench 3,Quick,"{""ln_vas_opted_flag"": 0, ""ln_self_dec_income"":...",2025-10-24 06:49:25,2025-10-26 07:26:41,2025-10
1,1372660,7780781a-4233-4512-800f-5311aa00b3a4,0.43129598170332545,2025-10-08 11:14:36.964090,2025-10-08 11:14:37.601300,Beta-Cash-Demo-Model,v1,Quick,F,not applicable,ios,Beta-Cash-Demo-Model_All_Trench,Cash,Trench 3,Quick,"{""ln_vas_opted_flag"": 0, ""ln_self_dec_income"":...",2025-10-08 19:14:34,NaT,2025-10
2,1447948,d6b74e17-a6e8-405e-97a1-4ad5403e635f,0.2901005335774055,2025-10-24 21:21:20.250597,2025-10-24 21:21:20.837942,Beta-Cash-Demo-Model,v1,Quick,M,not applicable,ios,Beta-Cash-Demo-Model_All_Trench,Cash,Trench 3,Quick,"{""ln_vas_opted_flag"": 0, ""ln_self_dec_income"":...",2025-10-25 05:21:17,NaT,2025-10
3,1529560,5cfab2c6-f84c-49e3-9ca5-65ad4183f15b,0.4952225368283653,2025-10-31 00:33:40.009270,2025-10-31 00:33:40.595331,Beta-Cash-Demo-Model,v1,Quick,F,not applicable,ios,Beta-Cash-Demo-Model_All_Trench,Cash,Trench 3,Quick,"{""ln_vas_opted_flag"": 1, ""ln_self_dec_income"":...",2025-10-31 08:33:36,2025-10-31 09:00:24,2025-10
4,1732633,14b9ce88-1f1a-442f-b09e-7b087a1150ef,0.43068510572616364,2025-10-26 19:31:52.495737,2025-10-26 19:31:53.399719,Beta-Cash-Demo-Model,v1,Quick,F,not applicable,android,Beta-Cash-Demo-Model_All_Trench,Cash,Trench 3,Quick,"{""ln_vas_opted_flag"": 1, ""ln_self_dec_income"":...",2025-10-27 03:31:49,2025-10-27 08:20:59,2025-10


### expand df

In [59]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 19
Expanded columns: 35


,customerId,digitalLoanAccountId,Beta_Cash_Demo_Score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,loanType,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month,calc_ln_vas_opted_flag,calc_ln_self_dec_income,calc_ln_age,calc_ln_source_funds_new_bin,calc_ln_loan_level_user_type,calc_ln_industry_new_cat_bin,calc_ln_marital_status,calc_ln_doc_type_rolled,calc_ln_education_level,calc_ln_ref2_type,calc_ln_email_primary_domain,calc_ln_os_type,calc_ln_mature_fspd30_flag,calc_ln_fspd30_flag,calc_ln_appln_submit_datetime,calc_ln_province_bin
0,1360407,caf7ec35-5d38-4bcf-98f2-52f1abc20f25,0.412737396573637,2025-10-23 22:49:29.257538,2025-10-23 22:49:30.031409,Beta-Cash-Demo-Model,v1,Quick,M,not applicable,android,Beta-Cash-Demo-Model_All_Trench,Cash,Trench 3,Quick,"{""ln_vas_opted_flag"": 0, ""ln_self_dec_income"":...",2025-10-24 06:49:25,2025-10-26 07:26:41,2025-10,0,25000,33,salary,1_Repeat Applicant,1.0,Married,umid,College Undergraduate,Parent,gmail.com,0,None,None,2025-10-23T22:49:25+00:00,metro manila
1,1372660,7780781a-4233-4512-800f-5311aa00b3a4,0.43129598170332545,2025-10-08 11:14:36.964090,2025-10-08 11:14:37.601300,Beta-Cash-Demo-Model,v1,Quick,F,not applicable,ios,Beta-Cash-Demo-Model_All_Trench,Cash,Trench 3,Quick,"{""ln_vas_opted_flag"": 0, ""ln_self_dec_income"":...",2025-10-08 19:14:34,NaT,2025-10,0,32000,37,salary,1_Repeat Applicant,1.0,Single,umid,College Graduate,Co-worker,yahoo.com,1,None,None,2025-10-08T11:14:34+00:00,bulacan
2,1447948,d6b74e17-a6e8-405e-97a1-4ad5403e635f,0.2901005335774055,2025-10-24 21:21:20.250597,2025-10-24 21:21:20.837942,Beta-Cash-Demo-Model,v1,Quick,M,not applicable,ios,Beta-Cash-Demo-Model_All_Trench,Cash,Trench 3,Quick,"{""ln_vas_opted_flag"": 0, ""ln_self_dec_income"":...",2025-10-25 05:21:17,NaT,2025-10,0,130000,34,salary,1_Repeat Applicant,3.0,Single,umid,College Graduate,Co-worker,yahoo.com,1,None,None,2025-10-24T21:21:17+00:00,cavite
3,1529560,5cfab2c6-f84c-49e3-9ca5-65ad4183f15b,0.4952225368283653,2025-10-31 00:33:40.009270,2025-10-31 00:33:40.595331,Beta-Cash-Demo-Model,v1,Quick,F,not applicable,ios,Beta-Cash-Demo-Model_All_Trench,Cash,Trench 3,Quick,"{""ln_vas_opted_flag"": 1, ""ln_self_dec_income"":...",2025-10-31 08:33:36,2025-10-31 09:00:24,2025-10,1,65000,38,salary,1_Repeat Applicant,2.0,Single,umid,College Graduate,Parent,yahoo.com,1,None,None,2025-10-31T00:33:36+00:00,metro manila
4,1732633,14b9ce88-1f1a-442f-b09e-7b087a1150ef,0.43068510572616364,2025-10-26 19:31:52.495737,2025-10-26 19:31:53.399719,Beta-Cash-Demo-Model,v1,Quick,F,not applicable,android,Beta-Cash-Demo-Model_All_Trench,Cash,Trench 3,Quick,"{""ln_vas_opted_flag"": 1, ""ln_self_dec_income"":...",2025-10-27 03:31:49,2025-10-27 08:20:59,2025-10,1,150000,28,salary,1_Repeat Applicant,3.0,Single,umid,Post-Graduate/Doctorate Degree Holder,Co-worker,gmail.com,0,None,None,2025-10-26T19:31:49+00:00,bulacan


In [60]:
df = expanded_df.drop(columns=['calcFeatures']).copy()

In [61]:
bin_columns = [col for col in df.columns if 'bin' in col.lower()]
df[bin_columns] = df[bin_columns].astype('category')

### run the psi calculation

In [62]:
df = df.copy()

# Define feature list
feature_list = ['Beta_Cash_Demo_Score',  
                    'calc_ln_vas_opted_flag', 'calc_ln_self_dec_income', 'calc_ln_age',
       'calc_ln_source_funds_new_bin', 'calc_ln_loan_level_user_type',
       'calc_ln_industry_new_cat_bin', 'calc_ln_marital_status',
       'calc_ln_doc_type_rolled', 'calc_ln_education_level',
       'calc_ln_ref2_type', 'calc_ln_email_primary_domain', 'calc_ln_os_type', 'calc_ln_province_bin' 
]

# Define segment columns
segment_columns = ['new_loan_type','osType']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_demo_model_psi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_demo_model_csi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=d4b02639-6252-45fc-98ee-ebc1a1174aec>

In [63]:
psi_results

,modelDisplayName,Model_Name,modelVersionId,trenchCategory,Feature,Feature_Type,Segment_Column,Segment_Value,Month,Base_Month,Current_Month,PSI
0,Beta-Cash-Demo-Model,Beta-Cash-Demo-Model_All_Trench,v1,Trench 3,Beta_Cash_Demo_Score,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000
1,Beta-Cash-Demo-Model,Beta-Cash-Demo-Model_All_Trench,v1,Trench 3,Beta_Cash_Demo_Score,categorical,Overall,All,2025-10,2025-09,2025-10,0.508651
2,Beta-Cash-Demo-Model,Beta-Cash-Demo-Model_All_Trench,v1,Trench 3,Beta_Cash_Demo_Score,categorical,Overall,All,2025-11,2025-09,2025-11,0.508651
3,Beta-Cash-Demo-Model,Beta-Cash-Demo-Model_All_Trench,v1,Trench 3,calc_ln_vas_opted_flag,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000
4,Beta-Cash-Demo-Model,Beta-Cash-Demo-Model_All_Trench,v1,Trench 3,calc_ln_vas_opted_flag,categorical,Overall,All,2025-10,2025-09,2025-10,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
163,Beta-Cash-Demo-Model,Beta-Cash-Demo-Model_All_Trench,v1,Trench 3,calc_ln_os_type,categorical,osType,ios,2025-10,2025-09,2025-10,0.000000
164,Beta-Cash-Demo-Model,Beta-Cash-Demo-Model_All_Trench,v1,Trench 3,calc_ln_os_type,categorical,osType,ios,2025-11,2025-09,2025-11,0.282874
165,Beta-Cash-Demo-Model,Beta-Cash-Demo-Model_All_Trench,v1,Trench 3,calc_ln_province_bin,categorical,osType,ios,2025-09,2025-09,2025-09,0.000000
166,Beta-Cash-Demo-Model,Beta-Cash-Demo-Model_All_Trench,v1,Trench 3,calc_ln_province_bin,categorical,osType,ios,2025-10,2025-09,2025-10,0.684686


## Beta-Cash-AppScore-Model

In [64]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_appscore_model_psi;"""
job = client.query(sq)
job.result()  # Wait for job to complete

sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_appscore_model_csi;"""
job = client.query(sq)
job.result()  # Wait for job to complete

### Trench 1

In [65]:
sq = r"""  
WITH parsed as (
  select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName = 'Beta-Cash-AppScore-Model'),

latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)

select * from (
  select 
 r.customerId,
 r.digitalLoanAccountId,
 --r.prediction,
 r.start_time,
 r.end_time,
 r.modelDisplayName,
 r.modelVersionId,
 loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
 'Beta-Cash-AppScore-Model_Trench1' Model_Name,
 'Cash' as product,
  JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory") AS trenchCategory,
  JSON_VALUE(m.requestPayload_clean, "$.predictions.loanType") AS loanType,
  --  Beta App Score Model Features for all Trench 1
  SAFE_CAST(JSON_VALUE(r.prediction_clean, "$.combined_score") AS Float64) AS beta_cash_app_score,
  calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,  
FROM latest_request r
left join model_run m
on r.digitalLoanAccountId = m.digitalLoanAccountId 
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by r.start_time desc) = 1
)
where trenchCategory = 'Trench 1'
;
"""

dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,Model_Name,product,trenchCategory,loanType,beta_cash_app_score,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,3697747,5111c645-037b-4e84-b293-49b00f561832,2025-10-13 05:01:12.594295,2025-10-13 05:01:12.862058,Beta-Cash-AppScore-Model,v1,Quick,M,not applicable,Beta-Cash-AppScore-Model_Trench1,Cash,Trench 1,Quick,0.628731,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-10-13 13:01:10,NaT,2025-10
1,3701465,6dcdd1ed-4607-47ef-b3cc-759a1bb95368,2025-10-19 12:22:13.303735,2025-10-19 12:22:13.581364,Beta-Cash-AppScore-Model,v1,Quick,M,not applicable,Beta-Cash-AppScore-Model_Trench1,Cash,Trench 1,Quick,0.557303,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-10-19 20:22:09,NaT,2025-10
2,3702403,2356f358-db4a-4e3c-8303-642365846b33,2025-10-09 05:31:10.451223,2025-10-09 05:31:10.721157,Beta-Cash-AppScore-Model,v1,Quick,F,not applicable,Beta-Cash-AppScore-Model_Trench1,Cash,Trench 1,Quick,0.440701,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-10-09 13:31:08,NaT,2025-10
3,3702842,0cae12ad-edf1-42ac-8b33-e2d30a2795ff,2025-09-24 11:38:31.905201,2025-09-24 11:38:32.146335,Beta-Cash-AppScore-Model,v1,Quick,F,not applicable,Beta-Cash-AppScore-Model_Trench1,Cash,Trench 1,Quick,0.595027,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-09-24 19:38:29,NaT,2025-09
4,3706124,cdb24027-f188-4fe7-965e-5f2b134ce720,2025-09-26 07:28:36.242588,2025-09-26 07:28:36.475830,Beta-Cash-AppScore-Model,v1,Quick,M,not applicable,Beta-Cash-AppScore-Model_Trench1,Cash,Trench 1,Quick,0.701129,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-09-26 15:28:34,NaT,2025-09


### expand df

In [66]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 18
Expanded columns: 28


,customerId,digitalLoanAccountId,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,Model_Name,product,trenchCategory,loanType,beta_cash_app_score,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month,calc_app_cnt_health_and_fitness_ever,calc_app_cnt_shopping_ever,calc_app_cnt_crypto_ever,calc_app_cnt_driver_ever,calc_app_cnt_payday_180d,calc_app_cnt_gambling_180d,calc_app_avg_time_bw_installed_mins_3d,calc_app_median_time_bw_installed_mins_3d,calc_app_avg_time_bw_installed_mins_ever,calc_app_median_time_bw_installed_mins_ever
0,3697747,5111c645-037b-4e84-b293-49b00f561832,2025-10-13 05:01:12.594295,2025-10-13 05:01:12.862058,Beta-Cash-AppScore-Model,v1,Quick,M,not applicable,Beta-Cash-AppScore-Model_Trench1,Cash,Trench 1,Quick,0.628731,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-10-13 13:01:10,NaT,2025-10,0,2,0,0,1,0,670.605556,382.250000,160459.404848,1305.183333
1,3701465,6dcdd1ed-4607-47ef-b3cc-759a1bb95368,2025-10-19 12:22:13.303735,2025-10-19 12:22:13.581364,Beta-Cash-AppScore-Model,v1,Quick,M,not applicable,Beta-Cash-AppScore-Model_Trench1,Cash,Trench 1,Quick,0.557303,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-10-19 20:22:09,NaT,2025-10,0,2,0,0,3,0,60.233333,60.233333,8675.342727,3353.750000
2,3702403,2356f358-db4a-4e3c-8303-642365846b33,2025-10-09 05:31:10.451223,2025-10-09 05:31:10.721157,Beta-Cash-AppScore-Model,v1,Quick,F,not applicable,Beta-Cash-AppScore-Model_Trench1,Cash,Trench 1,Quick,0.440701,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-10-09 13:31:08,NaT,2025-10,0,1,0,0,0,0,NaN,NaN,3974.817347,0.250000
3,3702842,0cae12ad-edf1-42ac-8b33-e2d30a2795ff,2025-09-24 11:38:31.905201,2025-09-24 11:38:32.146335,Beta-Cash-AppScore-Model,v1,Quick,F,not applicable,Beta-Cash-AppScore-Model_Trench1,Cash,Trench 1,Quick,0.595027,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-09-24 19:38:29,NaT,2025-09,0,2,0,1,3,0,651.466667,204.366667,507833.514035,9695.966667
4,3706124,cdb24027-f188-4fe7-965e-5f2b134ce720,2025-09-26 07:28:36.242588,2025-09-26 07:28:36.475830,Beta-Cash-AppScore-Model,v1,Quick,M,not applicable,Beta-Cash-AppScore-Model_Trench1,Cash,Trench 1,Quick,0.701129,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-09-26 15:28:34,NaT,2025-09,0,3,0,1,1,0,2002.833333,2002.833333,17125.206373,6593.183333


In [67]:
df = expanded_df.drop(columns=['calcFeatures']).copy()

### run the psi calculation

In [68]:
df = df.copy()

# Define feature list
feature_list = ['beta_cash_app_score',  
                    'calc_app_cnt_health_and_fitness_ever', 'calc_app_cnt_shopping_ever',
       'calc_app_cnt_crypto_ever', 'calc_app_cnt_driver_ever',
       'calc_app_cnt_payday_180d', 'calc_app_cnt_gambling_180d',
       'calc_app_avg_time_bw_installed_mins_3d',
       'calc_app_median_time_bw_installed_mins_3d',
       'calc_app_avg_time_bw_installed_mins_ever',
       'calc_app_median_time_bw_installed_mins_ever'
]

# Define segment columns
segment_columns = ['new_loan_type', 'osType']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',   'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_appscore_model_psi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_appscore_model_csi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=ec3ac28d-230e-4f0d-a052-446e96c5e4cf>

In [69]:
psi_results

,modelDisplayName,Model_Name,modelVersionId,trenchCategory,Feature,Feature_Type,Segment_Column,Segment_Value,Month,Base_Month,Current_Month,PSI
0,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench1,v1,Trench 1,beta_cash_app_score,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
1,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench1,v1,Trench 1,beta_cash_app_score,numerical,Overall,All,2025-10,2025-09,2025-10,0.028333
2,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench1,v1,Trench 1,beta_cash_app_score,numerical,Overall,All,2025-11,2025-09,2025-11,0.031303
3,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench1,v1,Trench 1,calc_app_cnt_health_and_fitness_ever,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
4,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench1,v1,Trench 1,calc_app_cnt_health_and_fitness_ever,numerical,Overall,All,2025-10,2025-09,2025-10,0.001157
5,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench1,v1,Trench 1,calc_app_cnt_health_and_fitness_ever,numerical,Overall,All,2025-11,2025-09,2025-11,0.004830
6,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench1,v1,Trench 1,calc_app_cnt_shopping_ever,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
7,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench1,v1,Trench 1,calc_app_cnt_shopping_ever,numerical,Overall,All,2025-10,2025-09,2025-10,0.008977
8,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench1,v1,Trench 1,calc_app_cnt_shopping_ever,numerical,Overall,All,2025-11,2025-09,2025-11,0.013271
9,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench1,v1,Trench 1,calc_app_cnt_crypto_ever,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000


### Trench 2

## Beta-Cash-AppScore-Model_Trench2

In [70]:
sq = r""" 
WITH parsed as (
  select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName = 'Beta-Cash-AppScore-Model'),

latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)

select * from (
  select 
 r.customerId,
 r.digitalLoanAccountId,
 --r.prediction,
 r.start_time,
 r.end_time,
 r.modelDisplayName,
 r.modelVersionId,
 loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
 'Beta-Cash-AppScore-Model_Trench2' Model_Name,
 'Cash' as product,
  JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory") AS trenchCategory,
  JSON_VALUE(m.requestPayload_clean, "$.predictions.loanType") AS loanType,
  --  Beta App Score Model Features for all Trench 1
  SAFE_CAST(JSON_VALUE(r.prediction_clean, "$.combined_score") AS Float64) AS beta_cash_app_score,
  r.calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,  
FROM latest_request r
left join model_run m
on r.digitalLoanAccountId = m.digitalLoanAccountId 
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by r.start_time desc) = 1
)
where trenchCategory = 'Trench 2'
;
"""

dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,Model_Name,product,trenchCategory,loanType,beta_cash_app_score,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,1081534,88189859-d2d0-4878-84be-b62d2b0d7c18,2025-10-29 05:42:34.216528,2025-10-29 05:42:35.056184,Beta-Cash-AppScore-Model,v1,Quick,F,not applicable,Beta-Cash-AppScore-Model_Trench2,Cash,Trench 2,Quick,0.438616,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-10-29 13:42:11,2025-10-30 11:27:33,2025-10
1,1104619,39eaf8e1-f619-44c8-a278-172d01f7afd7,2025-11-15 08:31:48.177234,2025-11-15 08:31:48.893524,Beta-Cash-AppScore-Model,v1,Quick,M,not applicable,Beta-Cash-AppScore-Model_Trench2,Cash,Trench 2,Quick,0.485746,"{""app_cnt_health_and_fitness_ever"": 2, ""app_cn...",2025-11-15 16:31:45,2025-11-15 17:07:40,2025-11
2,1132454,fa74bdf0-55ac-42cc-8681-351710599f0d,2025-11-14 08:54:39.275547,2025-11-14 08:54:40.032369,Beta-Cash-AppScore-Model,v1,Quick,M,not applicable,Beta-Cash-AppScore-Model_Trench2,Cash,Trench 2,Quick,0.517376,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-11-14 16:54:36,NaT,2025-11
3,1220935,80a725b0-612d-4618-aaaf-359d1791aabd,2025-10-11 05:28:12.022955,2025-10-11 05:28:13.048960,Beta-Cash-AppScore-Model,v1,Quick,F,not applicable,Beta-Cash-AppScore-Model_Trench2,Cash,Trench 2,Quick,0.319691,"{""app_cnt_health_and_fitness_ever"": 3, ""app_cn...",2025-10-11 13:28:09,NaT,2025-10
4,1256794,3f2a9212-7ae2-43f5-9f80-04b8f434aaea,2025-11-16 12:15:15.080013,2025-11-16 12:15:15.894926,Beta-Cash-AppScore-Model,v1,Quick,M,not applicable,Beta-Cash-AppScore-Model_Trench2,Cash,Trench 2,Quick,0.337420,"{""app_cnt_health_and_fitness_ever"": 1, ""app_cn...",2025-11-16 20:15:12,NaT,2025-11


### Expand_df

In [71]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 18
Expanded columns: 28


,customerId,digitalLoanAccountId,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,Model_Name,product,trenchCategory,loanType,beta_cash_app_score,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month,calc_app_cnt_health_and_fitness_ever,calc_app_cnt_shopping_ever,calc_app_cnt_crypto_ever,calc_app_cnt_driver_ever,calc_app_cnt_payday_180d,calc_app_cnt_gambling_180d,calc_app_avg_time_bw_installed_mins_3d,calc_app_median_time_bw_installed_mins_3d,calc_app_avg_time_bw_installed_mins_ever,calc_app_median_time_bw_installed_mins_ever
0,1081534,88189859-d2d0-4878-84be-b62d2b0d7c18,2025-10-29 05:42:34.216528,2025-10-29 05:42:35.056184,Beta-Cash-AppScore-Model,v1,Quick,F,not applicable,Beta-Cash-AppScore-Model_Trench2,Cash,Trench 2,Quick,0.438616,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-10-29 13:42:11,2025-10-30 11:27:33,2025-10,0,3,1,0,3,0,NaN,NaN,122028.288194,502.150000
1,1104619,39eaf8e1-f619-44c8-a278-172d01f7afd7,2025-11-15 08:31:48.177234,2025-11-15 08:31:48.893524,Beta-Cash-AppScore-Model,v1,Quick,M,not applicable,Beta-Cash-AppScore-Model_Trench2,Cash,Trench 2,Quick,0.485746,"{""app_cnt_health_and_fitness_ever"": 2, ""app_cn...",2025-11-15 16:31:45,2025-11-15 17:07:40,2025-11,2,3,2,0,14,0,NaN,NaN,100660.951515,155.366667
2,1132454,fa74bdf0-55ac-42cc-8681-351710599f0d,2025-11-14 08:54:39.275547,2025-11-14 08:54:40.032369,Beta-Cash-AppScore-Model,v1,Quick,M,not applicable,Beta-Cash-AppScore-Model_Trench2,Cash,Trench 2,Quick,0.517376,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-11-14 16:54:36,NaT,2025-11,0,2,0,1,5,0,16.716667,16.716667,92731.247407,2616.608333
3,1220935,80a725b0-612d-4618-aaaf-359d1791aabd,2025-10-11 05:28:12.022955,2025-10-11 05:28:13.048960,Beta-Cash-AppScore-Model,v1,Quick,F,not applicable,Beta-Cash-AppScore-Model_Trench2,Cash,Trench 2,Quick,0.319691,"{""app_cnt_health_and_fitness_ever"": 3, ""app_cn...",2025-10-11 13:28:09,NaT,2025-10,3,7,0,0,0,0,NaN,NaN,97928.448519,10.633333
4,1256794,3f2a9212-7ae2-43f5-9f80-04b8f434aaea,2025-11-16 12:15:15.080013,2025-11-16 12:15:15.894926,Beta-Cash-AppScore-Model,v1,Quick,M,not applicable,Beta-Cash-AppScore-Model_Trench2,Cash,Trench 2,Quick,0.337420,"{""app_cnt_health_and_fitness_ever"": 1, ""app_cn...",2025-11-16 20:15:12,NaT,2025-11,1,3,6,0,1,1,647.470833,567.625000,4914.430435,1394.091667


In [72]:
df = expanded_df.drop(columns=['calcFeatures']).copy()

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15582 entries, 0 to 15581
Data columns (total 27 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   customerId                                   15582 non-null  object        
 1   digitalLoanAccountId                         15582 non-null  object        
 2   start_time                                   15582 non-null  datetime64[us]
 3   end_time                                     15582 non-null  datetime64[us]
 4   modelDisplayName                             15582 non-null  object        
 5   modelVersionId                               15582 non-null  object        
 6   new_loan_type                                15356 non-null  object        
 7   gender                                       15356 non-null  object        
 8   loan_product_type                            15582 non-null  object        


### run the psi calculation

In [74]:
df = df.copy()

# Define feature list
feature_list = ['beta_cash_app_score', 'calc_app_cnt_health_and_fitness_ever',  'calc_app_cnt_shopping_ever', 'calc_app_cnt_crypto_ever',
       'calc_app_cnt_driver_ever', 'calc_app_cnt_payday_180d',  'calc_app_cnt_gambling_180d', 'calc_app_avg_time_bw_installed_mins_3d',
       'calc_app_median_time_bw_installed_mins_3d', 'calc_app_avg_time_bw_installed_mins_ever', 'calc_app_median_time_bw_installed_mins_ever'
]

# Define segment columns
segment_columns = ['new_loan_type', 'osType']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',   'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_appscore_model_psi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_appscore_model_csi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=5ded2ebd-f602-460d-9391-94fa08ff106b>

In [75]:
psi_results

,modelDisplayName,Model_Name,modelVersionId,trenchCategory,Feature,Feature_Type,Segment_Column,Segment_Value,Month,Base_Month,Current_Month,PSI
0,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench2,v1,Trench 2,beta_cash_app_score,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
1,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench2,v1,Trench 2,beta_cash_app_score,numerical,Overall,All,2025-10,2025-09,2025-10,0.059084
2,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench2,v1,Trench 2,beta_cash_app_score,numerical,Overall,All,2025-11,2025-09,2025-11,0.037129
3,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench2,v1,Trench 2,calc_app_cnt_health_and_fitness_ever,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
4,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench2,v1,Trench 2,calc_app_cnt_health_and_fitness_ever,numerical,Overall,All,2025-10,2025-09,2025-10,0.023197
5,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench2,v1,Trench 2,calc_app_cnt_health_and_fitness_ever,numerical,Overall,All,2025-11,2025-09,2025-11,0.014970
6,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench2,v1,Trench 2,calc_app_cnt_shopping_ever,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
7,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench2,v1,Trench 2,calc_app_cnt_shopping_ever,numerical,Overall,All,2025-10,2025-09,2025-10,0.170900
8,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench2,v1,Trench 2,calc_app_cnt_shopping_ever,numerical,Overall,All,2025-11,2025-09,2025-11,0.174238
9,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench2,v1,Trench 2,calc_app_cnt_crypto_ever,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000


### Trench 3

## Beta-Cash-AppScore-Model_Trench3

In [76]:
sq = r""" WITH parsed as (
  select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName = 'Beta-Cash-AppScore-Model'),

latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)

select * from (
  select 
 r.customerId,
 r.digitalLoanAccountId,
 --r.prediction,
 r.start_time,
 r.end_time,
 r.modelDisplayName,
 r.modelVersionId,
 loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
 'Beta-Cash-AppScore-Model_Trench3' Model_Name,
  JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory") AS trenchCategory,
  JSON_VALUE(m.requestPayload_clean, "$.predictions.loanType") AS loanType,
  --  Beta App Score Model Features for Trench 3
  SAFE_CAST(JSON_VALUE(r.prediction_clean, "$.combined_score") AS Float64) AS beta_cash_app_score,
 r.calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,  
FROM latest_request r
left join model_run m
on r.digitalLoanAccountId = m.digitalLoanAccountId 
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by r.start_time desc) = 1
)
where trenchCategory = 'Trench 3'

;"""
dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,Model_Name,trenchCategory,loanType,beta_cash_app_score,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,1099812,7ba62b26-aaca-4cbb-97c5-8e8f341b3a35,2025-10-26 07:17:00.099879,2025-10-26 07:17:00.923152,Beta-Cash-AppScore-Model,v1,Quick,F,not applicable,Beta-Cash-AppScore-Model_Trench3,Trench 3,Quick,0.444101,"{""app_cnt_health_and_fitness_ever"": 1, ""app_cn...",2025-10-26 15:16:56,2025-10-26 15:25:24,2025-10
1,1337475,8e7bd4ad-f1bf-4dd8-82e7-e12f7c656e35,2025-10-12 07:47:22.530874,2025-10-12 07:47:23.443766,Beta-Cash-AppScore-Model,v1,Quick,M,not applicable,Beta-Cash-AppScore-Model_Trench3,Trench 3,Quick,0.534571,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-10-12 15:47:14,NaT,2025-10
2,1490639,5911d955-a13f-469a-9c56-58e24d11fff9,2025-11-02 11:30:25.457551,2025-11-02 11:30:26.238939,Beta-Cash-AppScore-Model,v1,Quick,M,not applicable,Beta-Cash-AppScore-Model_Trench3,Trench 3,Quick,0.490077,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-11-02 19:30:21,NaT,2025-11
3,1718836,cf0a3ade-79cb-425e-bf02-600a1abd1e4e,2025-11-01 04:53:01.553763,2025-11-01 04:53:02.460724,Beta-Cash-AppScore-Model,v1,Quick,M,not applicable,Beta-Cash-AppScore-Model_Trench3,Trench 3,Quick,0.559094,"{""app_cnt_health_and_fitness_ever"": 2, ""app_cn...",2025-11-01 12:52:57,NaT,2025-11
4,1761593,dadc9c9c-6a08-4b87-bd90-ed0472baea1a,2025-11-09 08:39:19.521068,2025-11-09 08:39:20.255999,Beta-Cash-AppScore-Model,v1,Quick,M,not applicable,Beta-Cash-AppScore-Model_Trench3,Trench 3,Quick,0.558047,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-11-09 16:39:16,NaT,2025-11


### Expand_df

In [77]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 17
Expanded columns: 32


,customerId,digitalLoanAccountId,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,Model_Name,trenchCategory,loanType,beta_cash_app_score,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month,calc_app_cnt_health_and_fitness_ever,calc_app_cnt_productivity_ever,calc_app_cnt_rated_for_18plus_ever,calc_app_cnt_books_and_reference_ever,calc_app_cnt_gaming_180d,calc_app_cnt_absence_tag_365d,calc_app_last_payday_install_to_apply_days,calc_app_cnt_absence_tag_365d_binned,calc_app_cnt_gaming_180d_binned,calc_app_cnt_productivity_ever_binned,calc_app_cnt_rated_for_18plus_ever_binned,calc_app_cnt_health_and_fitness_ever_binned,calc_app_cnt_books_and_reference_ever_binned,calc_app_last_payday_install_to_apply_days_binned,calc_ln_user_type
0,1099812,7ba62b26-aaca-4cbb-97c5-8e8f341b3a35,2025-10-26 07:17:00.099879,2025-10-26 07:17:00.923152,Beta-Cash-AppScore-Model,v1,Quick,F,not applicable,Beta-Cash-AppScore-Model_Trench3,Trench 3,Quick,0.444101,"{""app_cnt_health_and_fitness_ever"": 1, ""app_cn...",2025-10-26 15:16:56,2025-10-26 15:25:24,2025-10,1,4,0,0,1,5,57.5,4.182 – 6.970,<= 5.717,> 2.333,<= 1.010,<= 2.121,<= 1.273,> 18.344,1_Repeat Applicant
1,1337475,8e7bd4ad-f1bf-4dd8-82e7-e12f7c656e35,2025-10-12 07:47:22.530874,2025-10-12 07:47:23.443766,Beta-Cash-AppScore-Model,v1,Quick,M,not applicable,Beta-Cash-AppScore-Model_Trench3,Trench 3,Quick,0.534571,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-10-12 15:47:14,NaT,2025-10,0,3,2,0,0,9,8.1,> 6.970,<= 5.717,> 2.333,> 1.010,<= 2.121,<= 1.273,<= 18.344,1_Repeat Applicant
2,1490639,5911d955-a13f-469a-9c56-58e24d11fff9,2025-11-02 11:30:25.457551,2025-11-02 11:30:26.238939,Beta-Cash-AppScore-Model,v1,Quick,M,not applicable,Beta-Cash-AppScore-Model_Trench3,Trench 3,Quick,0.490077,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-11-02 19:30:21,NaT,2025-11,0,3,1,0,2,1,163.0,<= 4.182,<= 5.717,> 2.333,<= 1.010,<= 2.121,<= 1.273,> 18.344,1_Repeat Applicant
3,1718836,cf0a3ade-79cb-425e-bf02-600a1abd1e4e,2025-11-01 04:53:01.553763,2025-11-01 04:53:02.460724,Beta-Cash-AppScore-Model,v1,Quick,M,not applicable,Beta-Cash-AppScore-Model_Trench3,Trench 3,Quick,0.559094,"{""app_cnt_health_and_fitness_ever"": 2, ""app_cn...",2025-11-01 12:52:57,NaT,2025-11,2,8,4,0,1,31,16.6,> 6.970,<= 5.717,> 2.333,> 1.010,<= 2.121,<= 1.273,<= 18.344,1_Repeat Applicant
4,1761593,dadc9c9c-6a08-4b87-bd90-ed0472baea1a,2025-11-09 08:39:19.521068,2025-11-09 08:39:20.255999,Beta-Cash-AppScore-Model,v1,Quick,M,not applicable,Beta-Cash-AppScore-Model_Trench3,Trench 3,Quick,0.558047,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",2025-11-09 16:39:16,NaT,2025-11,0,1,0,0,3,2,NaN,<= 4.182,<= 5.717,<= 2.333,<= 1.010,<= 2.121,<= 1.273,nan,1_Repeat Applicant


In [78]:
df = expanded_df.drop(columns=['calcFeatures']).copy()

### run the psi calculation

In [79]:
df = df.copy()

# Define feature list
feature_list = ['beta_cash_app_score', 'calc_app_cnt_health_and_fitness_ever',
       'calc_app_cnt_productivity_ever', 'calc_app_cnt_rated_for_18plus_ever',
       'calc_app_cnt_books_and_reference_ever', 'calc_app_cnt_gaming_180d',
       'calc_app_cnt_absence_tag_365d',
       'calc_app_last_payday_install_to_apply_days',
       'calc_app_cnt_absence_tag_365d_binned',
       'calc_app_cnt_gaming_180d_binned',
       'calc_app_cnt_productivity_ever_binned',
       'calc_app_cnt_rated_for_18plus_ever_binned',
       'calc_app_cnt_health_and_fitness_ever_binned',
       'calc_app_cnt_books_and_reference_ever_binned',
       'calc_app_last_payday_install_to_apply_days_binned',
       'calc_ln_user_type']

# Define segment columns
segment_columns = ['new_loan_type','osType']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',   'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_appscore_model_psi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_appscore_model_csi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=03199e38-63c4-405d-9dfc-4523f5d52dda>

In [80]:
psi_results

,modelDisplayName,Model_Name,modelVersionId,trenchCategory,Feature,Feature_Type,Segment_Column,Segment_Value,Month,Base_Month,Current_Month,PSI
0,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench3,v1,Trench 3,beta_cash_app_score,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
1,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench3,v1,Trench 3,beta_cash_app_score,numerical,Overall,All,2025-10,2025-09,2025-10,0.176788
2,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench3,v1,Trench 3,beta_cash_app_score,numerical,Overall,All,2025-11,2025-09,2025-11,0.159490
3,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench3,v1,Trench 3,calc_app_cnt_health_and_fitness_ever,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000
4,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench3,v1,Trench 3,calc_app_cnt_health_and_fitness_ever,categorical,Overall,All,2025-10,2025-09,2025-10,0.000000
5,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench3,v1,Trench 3,calc_app_cnt_health_and_fitness_ever,categorical,Overall,All,2025-11,2025-09,2025-11,0.000000
6,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench3,v1,Trench 3,calc_app_cnt_productivity_ever,numerical,Overall,All,2025-09,2025-09,2025-09,0.000000
7,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench3,v1,Trench 3,calc_app_cnt_productivity_ever,numerical,Overall,All,2025-10,2025-09,2025-10,0.051058
8,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench3,v1,Trench 3,calc_app_cnt_productivity_ever,numerical,Overall,All,2025-11,2025-09,2025-11,0.059972
9,Beta-Cash-AppScore-Model,Beta-Cash-AppScore-Model_Trench3,v1,Trench 3,calc_app_cnt_rated_for_18plus_ever,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000


## Beta-Cash-Stack-Model Trench1

In [81]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_stack_model_psi;"""
job = client.query(sq)
job.result()  # Wait for job to complete

sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_stack_model_csi;"""
job = client.query(sq)
job.result()  # Wait for job to complete

### Trench 1

In [82]:
sq = r""" 
WITH parsed as (
  select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
--REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName = 'Beta-Cash-Stack-Model'),

latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)

select * from (
  select 
 r.customerId,
 r.digitalLoanAccountId,
 r.prediction Beta_cash_stack_score,
 r.start_time,
 r.end_time,
 r.modelDisplayName,
 r.modelVersionId,
 loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
  case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
        when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
        when lower(loanmaster.deviceType) like '%andro%' then 'android'
        else 'ios' end osType,
 'Beta-Cash-Stack-Model_Trench1' Model_Name,
 'Cash' as product,
 JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory") AS trenchCategory,
  r.calcFeatures,
 coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,  
FROM latest_request r
left join model_run m
on r.digitalLoanAccountId = m.digitalLoanAccountId 
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by r.start_time desc) = 1
  )
 where trenchCategory = 'Trench 1'
;
"""

dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,Beta_cash_stack_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,3677055,07649fad-a0ed-4926-84ea-72448b6a810a,0.708230050419592,2025-10-03 05:49:02.538336,2025-10-03 05:49:02.801597,Beta-Cash-Stack-Model,v1,Quick,M,not applicable,android,Beta-Cash-Stack-Model_Trench1,Cash,Trench 1,"{""demo_score"": ""0.5547454635396003"", ""credo_sc...",2025-10-03 13:49:00,NaT,2025-10
1,3685272,59fff0fd-ccae-4441-881e-f2eed9bf4ae8,0.5987124903728266,2025-10-08 05:32:44.385329,2025-10-08 05:32:44.397039,Beta-Cash-Stack-Model,v1,Quick,F,not applicable,ios,Beta-Cash-Stack-Model_Trench1,Cash,Trench 1,"{""demo_score"": ""0.572803479142278"", ""credo_sco...",2025-10-08 13:32:42,NaT,2025-10
2,3703195,7b83580d-de33-47c5-966d-1af8b62103e7,0.8672323616028619,2025-09-24 10:40:00.320644,2025-09-24 10:40:00.710021,Beta-Cash-Stack-Model,v1,Quick,F,not applicable,android,Beta-Cash-Stack-Model_Trench1,Cash,Trench 1,"{""demo_score"": ""0.6170742250834379"", ""credo_sc...",2025-09-24 18:39:57,NaT,2025-09
3,3706555,055e81bc-36cc-482c-aa88-68a01f6ed291,0.49159064045488854,2025-09-26 10:06:11.475823,2025-09-26 10:06:11.489401,Beta-Cash-Stack-Model,v1,Quick,M,not applicable,ios,Beta-Cash-Stack-Model_Trench1,Cash,Trench 1,"{""demo_score"": ""0.5092290884539968"", ""credo_sc...",2025-09-26 18:06:09,NaT,2025-09
4,3706656,71e381ac-c91f-4494-8fe1-4aacdc69eaf7,0.5489917552867839,2025-09-26 10:46:49.086449,2025-09-26 10:46:49.098226,Beta-Cash-Stack-Model,v1,Quick,F,not applicable,ios,Beta-Cash-Stack-Model_Trench1,Cash,Trench 1,"{""demo_score"": ""0.5966140168327027"", ""credo_sc...",2025-09-26 18:46:46,NaT,2025-09


### Expand_df

In [83]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 18
Expanded columns: 22


,customerId,digitalLoanAccountId,Beta_cash_stack_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month,calc_demo_score,calc_credo_score,calc_trx_score,calc_app_score
0,3677055,07649fad-a0ed-4926-84ea-72448b6a810a,0.708230050419592,2025-10-03 05:49:02.538336,2025-10-03 05:49:02.801597,Beta-Cash-Stack-Model,v1,Quick,M,not applicable,android,Beta-Cash-Stack-Model_Trench1,Cash,Trench 1,"{""demo_score"": ""0.5547454635396003"", ""credo_sc...",2025-10-03 13:49:00,NaT,2025-10,0.5547454635396003,0.2535,null,0.5545814060759389
1,3685272,59fff0fd-ccae-4441-881e-f2eed9bf4ae8,0.5987124903728266,2025-10-08 05:32:44.385329,2025-10-08 05:32:44.397039,Beta-Cash-Stack-Model,v1,Quick,F,not applicable,ios,Beta-Cash-Stack-Model_Trench1,Cash,Trench 1,"{""demo_score"": ""0.572803479142278"", ""credo_sco...",2025-10-08 13:32:42,NaT,2025-10,0.572803479142278,0.2252,null,NaN
2,3703195,7b83580d-de33-47c5-966d-1af8b62103e7,0.8672323616028619,2025-09-24 10:40:00.320644,2025-09-24 10:40:00.710021,Beta-Cash-Stack-Model,v1,Quick,F,not applicable,android,Beta-Cash-Stack-Model_Trench1,Cash,Trench 1,"{""demo_score"": ""0.6170742250834379"", ""credo_sc...",2025-09-24 18:39:57,NaT,2025-09,0.6170742250834379,0.3974,null,0.6226764349394909
3,3706555,055e81bc-36cc-482c-aa88-68a01f6ed291,0.49159064045488854,2025-09-26 10:06:11.475823,2025-09-26 10:06:11.489401,Beta-Cash-Stack-Model,v1,Quick,M,not applicable,ios,Beta-Cash-Stack-Model_Trench1,Cash,Trench 1,"{""demo_score"": ""0.5092290884539968"", ""credo_sc...",2025-09-26 18:06:09,NaT,2025-09,0.5092290884539968,0.1901,null,NaN
4,3706656,71e381ac-c91f-4494-8fe1-4aacdc69eaf7,0.5489917552867839,2025-09-26 10:46:49.086449,2025-09-26 10:46:49.098226,Beta-Cash-Stack-Model,v1,Quick,F,not applicable,ios,Beta-Cash-Stack-Model_Trench1,Cash,Trench 1,"{""demo_score"": ""0.5966140168327027"", ""credo_sc...",2025-09-26 18:46:46,NaT,2025-09,0.5966140168327027,0.1504,null,NaN


In [84]:
df = expanded_df.drop(columns=['calcFeatures']).copy()

### run the psi calculation

In [85]:
df = df.copy()

# Define feature list
feature_list = ['Beta_cash_stack_score',  
                    'calc_demo_score', 'calc_credo_score',
       'calc_trx_score', 'calc_app_score'
]

# Define segment columns
segment_columns = ['new_loan_type', 'osType']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',   'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_stack_model_psi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_stack_model_csi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=56d17700-a875-4fef-befe-1a38f915051f>

In [86]:
psi_results

,modelDisplayName,Model_Name,modelVersionId,trenchCategory,Feature,Feature_Type,Segment_Column,Segment_Value,Month,Base_Month,Current_Month,PSI
0,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench1,v1,Trench 1,Beta_cash_stack_score,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000
1,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench1,v1,Trench 1,Beta_cash_stack_score,categorical,Overall,All,2025-10,2025-09,2025-10,0.027985
2,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench1,v1,Trench 1,Beta_cash_stack_score,categorical,Overall,All,2025-11,2025-09,2025-11,0.027985
3,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench1,v1,Trench 1,calc_demo_score,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000
4,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench1,v1,Trench 1,calc_demo_score,categorical,Overall,All,2025-10,2025-09,2025-10,0.021295
5,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench1,v1,Trench 1,calc_demo_score,categorical,Overall,All,2025-11,2025-09,2025-11,0.021998
6,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench1,v1,Trench 1,calc_credo_score,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000
7,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench1,v1,Trench 1,calc_credo_score,categorical,Overall,All,2025-10,2025-09,2025-10,0.032965
8,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench1,v1,Trench 1,calc_credo_score,categorical,Overall,All,2025-11,2025-09,2025-11,0.031793
9,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench1,v1,Trench 1,calc_trx_score,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000


### Trench 2

In [87]:
sq = r""" 
WITH parsed as (
  select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
--REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName = 'Beta-Cash-Stack-Model'),

latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)

select * from (
  select 
 r.customerId,
 r.digitalLoanAccountId,
 r.prediction Beta_cash_stack_score,
 r.start_time,
 r.end_time,
 r.modelDisplayName,
 r.modelVersionId,
 loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
   case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
        when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
        when lower(loanmaster.deviceType) like '%andro%' then 'android'
        else 'ios' end osType,
 'Beta-Cash-Stack-Model_Trench2' Model_Name,
'Cash' as product,
 JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory") AS trenchCategory,
 r.calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
FROM latest_request r
left join model_run m
on r.digitalLoanAccountId = m.digitalLoanAccountId 
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by r.start_time desc) = 1
  )
 where trenchCategory = 'Trench 2'
;
"""

dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,Beta_cash_stack_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,1015055,4ead8628-cf9f-46db-bb95-797359b30b6b,0.046161475498969665,2025-10-28 04:50:45.773474,2025-10-28 04:50:46.803882,Beta-Cash-Stack-Model,v1,Quick,M,not applicable,android,Beta-Cash-Stack-Model_Trench2,Cash,Trench 2,"{""demo_score"": ""0.4435704467567995"", ""credo_sc...",2025-10-28 12:50:42,2025-10-31 00:23:39,2025-10
1,1049877,7ea50d03-6ec6-4bf7-817c-3d356192d595,0.8713590698122456,2025-11-13 15:27:28.704958,2025-11-13 15:27:29.532219,Beta-Cash-Stack-Model,v1,Quick,F,not applicable,android,Beta-Cash-Stack-Model_Trench2,Cash,Trench 2,"{""demo_score"": ""0.47550225812046554"", ""credo_s...",2025-11-13 23:27:24,NaT,2025-11
2,1075646,bf1bf72c-d11d-4afe-9fff-8355992f1e37,0.24793004286158293,2025-11-14 02:57:47.984144,2025-11-14 02:57:48.728304,Beta-Cash-Stack-Model,v1,Quick,M,not applicable,android,Beta-Cash-Stack-Model_Trench2,Cash,Trench 2,"{""demo_score"": ""0.544655771990251"", ""credo_sco...",2025-11-14 10:57:44,NaT,2025-11
3,1098638,060aa06e-be98-438d-bf5b-02c370ae4e27,0.9289152659392022,2025-11-14 05:08:33.358551,2025-11-14 05:08:34.133619,Beta-Cash-Stack-Model,v1,Quick,M,not applicable,android,Beta-Cash-Stack-Model_Trench2,Cash,Trench 2,"{""demo_score"": ""0.5289517650750601"", ""credo_sc...",2025-11-14 13:07:56,NaT,2025-11
4,1139306,9a391777-c368-478e-9845-dbdf39e9ae30,0.4650249339849845,2025-11-17 05:11:32.477939,2025-11-17 05:11:32.944708,Beta-Cash-Stack-Model,v1,Quick,F,not applicable,ios,Beta-Cash-Stack-Model_Trench2,Cash,Trench 2,"{""demo_score"": ""0.5113799845898461"", ""credo_sc...",2025-11-17 13:11:30,2025-11-17 14:17:40,2025-11


### Expand_df

In [88]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 18
Expanded columns: 22


,customerId,digitalLoanAccountId,Beta_cash_stack_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month,calc_demo_score,calc_credo_score,calc_trx_score,calc_app_score
0,1015055,4ead8628-cf9f-46db-bb95-797359b30b6b,0.046161475498969665,2025-10-28 04:50:45.773474,2025-10-28 04:50:46.803882,Beta-Cash-Stack-Model,v1,Quick,M,not applicable,android,Beta-Cash-Stack-Model_Trench2,Cash,Trench 2,"{""demo_score"": ""0.4435704467567995"", ""credo_sc...",2025-10-28 12:50:42,2025-10-31 00:23:39,2025-10,0.4435704467567995,0.1104,0.31738556123326134,0.27281398405801566
1,1049877,7ea50d03-6ec6-4bf7-817c-3d356192d595,0.8713590698122456,2025-11-13 15:27:28.704958,2025-11-13 15:27:29.532219,Beta-Cash-Stack-Model,v1,Quick,F,not applicable,android,Beta-Cash-Stack-Model_Trench2,Cash,Trench 2,"{""demo_score"": ""0.47550225812046554"", ""credo_s...",2025-11-13 23:27:24,NaT,2025-11,0.47550225812046554,0.1646,0.4149924594684216,0.6663503592319631
2,1075646,bf1bf72c-d11d-4afe-9fff-8355992f1e37,0.24793004286158293,2025-11-14 02:57:47.984144,2025-11-14 02:57:48.728304,Beta-Cash-Stack-Model,v1,Quick,M,not applicable,android,Beta-Cash-Stack-Model_Trench2,Cash,Trench 2,"{""demo_score"": ""0.544655771990251"", ""credo_sco...",2025-11-14 10:57:44,NaT,2025-11,0.544655771990251,0.1136,0.3814062240005537,0.41016983098594073
3,1098638,060aa06e-be98-438d-bf5b-02c370ae4e27,0.9289152659392022,2025-11-14 05:08:33.358551,2025-11-14 05:08:34.133619,Beta-Cash-Stack-Model,v1,Quick,M,not applicable,android,Beta-Cash-Stack-Model_Trench2,Cash,Trench 2,"{""demo_score"": ""0.5289517650750601"", ""credo_sc...",2025-11-14 13:07:56,NaT,2025-11,0.5289517650750601,0.6197,0.6263235997968442,0.6890014445826773
4,1139306,9a391777-c368-478e-9845-dbdf39e9ae30,0.4650249339849845,2025-11-17 05:11:32.477939,2025-11-17 05:11:32.944708,Beta-Cash-Stack-Model,v1,Quick,F,not applicable,ios,Beta-Cash-Stack-Model_Trench2,Cash,Trench 2,"{""demo_score"": ""0.5113799845898461"", ""credo_sc...",2025-11-17 13:11:30,2025-11-17 14:17:40,2025-11,0.5113799845898461,0.137,0.6017013967646913,NaN


In [89]:
df = expanded_df.drop(columns=['calcFeatures']).copy()

### run the psi calculation

In [90]:
df = df.copy()

# Define feature list
feature_list = ['Beta_cash_stack_score',  'calc_demo_score', 'calc_credo_score',     'calc_trx_score', 'calc_app_score']

# Define segment columns
segment_columns = ['new_loan_type', 'osType']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',   'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_stack_model_psi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_stack_model_csi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=64a33009-0899-4758-9131-22d4cc3a77f4>

In [91]:
psi_results

,modelDisplayName,Model_Name,modelVersionId,trenchCategory,Feature,Feature_Type,Segment_Column,Segment_Value,Month,Base_Month,Current_Month,PSI
0,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench2,v1,Trench 2,Beta_cash_stack_score,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000
1,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench2,v1,Trench 2,Beta_cash_stack_score,categorical,Overall,All,2025-10,2025-09,2025-10,0.150420
2,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench2,v1,Trench 2,Beta_cash_stack_score,categorical,Overall,All,2025-11,2025-09,2025-11,0.150420
3,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench2,v1,Trench 2,calc_demo_score,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000
4,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench2,v1,Trench 2,calc_demo_score,categorical,Overall,All,2025-10,2025-09,2025-10,0.174229
5,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench2,v1,Trench 2,calc_demo_score,categorical,Overall,All,2025-11,2025-09,2025-11,0.185316
6,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench2,v1,Trench 2,calc_credo_score,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000
7,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench2,v1,Trench 2,calc_credo_score,categorical,Overall,All,2025-10,2025-09,2025-10,0.178289
8,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench2,v1,Trench 2,calc_credo_score,categorical,Overall,All,2025-11,2025-09,2025-11,0.206199
9,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench2,v1,Trench 2,calc_trx_score,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000


### Trench 3

In [92]:
sq = r"""WITH parsed as (
  select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
--REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName = 'Beta-Cash-Stack-Model'),

latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)

select * from (
  select 
 r.customerId,
 r.digitalLoanAccountId,
 r.prediction Beta_cash_stack_score,
 r.start_time,
 r.end_time,
 r.modelDisplayName,
 r.modelVersionId,
 loanmaster.new_loan_type,
 loanmaster.gender,
   case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
        when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
        when lower(loanmaster.deviceType) like '%andro%' then 'android'
        else 'ios' end osType,
 'Beta-Cash-Stack-Model_Trench3' Model_Name,
 'Cash' as product,
 JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory") AS trenchCategory,
  r.calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,  
FROM latest_request r
left join model_run m
on r.digitalLoanAccountId = m.digitalLoanAccountId 
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by r.start_time desc) = 1
  )
 where trenchCategory = 'Trench 3'

;"""
dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,Beta_cash_stack_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,1386299,b0caf33d-83f3-4427-a8d8-e5eca82f6355,0.2886257354628351,2025-10-09 15:05:03.215975,2025-10-09 15:05:04.142213,Beta-Cash-Stack-Model,v1,Quick,F,not applicable,android,Beta-Cash-Stack-Model_Trench3,Cash,Trench 3,"{""demo_score"": ""0.40485963581973106"", ""credo_s...",2025-10-09 23:04:59,2025-10-10 07:12:46,2025-10
1,1465219,2f93d0d5-d161-4fbb-be7b-72c7da1b5307,0.2629594835015238,2025-10-25 07:10:54.932265,2025-10-25 07:10:55.668217,Beta-Cash-Stack-Model,v1,Quick,M,not applicable,android,Beta-Cash-Stack-Model_Trench3,Cash,Trench 3,"{""demo_score"": ""0.572585116558061"", ""credo_sco...",2025-10-25 15:10:51,NaT,2025-10
2,1481264,0c91c8e6-7fe3-4cbe-989b-5f7b909a9bb6,0.614258676725816,2025-11-05 03:09:25.559970,2025-11-05 03:09:26.502237,Beta-Cash-Stack-Model,v1,Quick,F,not applicable,android,Beta-Cash-Stack-Model_Trench3,Cash,Trench 3,"{""demo_score"": ""0.40911008935267046"", ""credo_s...",2025-11-05 11:09:22,NaT,2025-11
3,1502957,c36cb635-3f42-463f-a828-a4189f265970,0.6478110638835973,2025-10-24 04:30:56.184280,2025-10-24 04:30:57.073509,Beta-Cash-Stack-Model,v1,Quick,M,not applicable,android,Beta-Cash-Stack-Model_Trench3,Cash,Trench 3,"{""demo_score"": ""0.47698438254532394"", ""credo_s...",2025-10-24 12:30:52,NaT,2025-10
4,1564428,0f7f973b-8558-4480-8957-bc5742ffb998,0.3097051967548624,2025-10-22 10:52:28.646954,2025-10-22 10:52:29.349800,Beta-Cash-Stack-Model,v1,Quick,M,not applicable,ios,Beta-Cash-Stack-Model_Trench3,Cash,Trench 3,"{""demo_score"": ""0.3412514366272126"", ""credo_sc...",2025-10-22 18:52:25,NaT,2025-10


### Expand_df

In [93]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 18
Expanded columns: 22


,customerId,digitalLoanAccountId,Beta_cash_stack_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month,calc_demo_score,calc_credo_score,calc_trx_score,calc_app_score
0,1386299,b0caf33d-83f3-4427-a8d8-e5eca82f6355,0.2886257354628351,2025-10-09 15:05:03.215975,2025-10-09 15:05:04.142213,Beta-Cash-Stack-Model,v1,Quick,F,not applicable,android,Beta-Cash-Stack-Model_Trench3,Cash,Trench 3,"{""demo_score"": ""0.40485963581973106"", ""credo_s...",2025-10-09 23:04:59,2025-10-10 07:12:46,2025-10,0.40485963581973106,0.0698,0.4023404325122784,0.467487047687154
1,1465219,2f93d0d5-d161-4fbb-be7b-72c7da1b5307,0.2629594835015238,2025-10-25 07:10:54.932265,2025-10-25 07:10:55.668217,Beta-Cash-Stack-Model,v1,Quick,M,not applicable,android,Beta-Cash-Stack-Model_Trench3,Cash,Trench 3,"{""demo_score"": ""0.572585116558061"", ""credo_sco...",2025-10-25 15:10:51,NaT,2025-10,0.572585116558061,0.0755,0.43713484786837326,0.40351723186822464
2,1481264,0c91c8e6-7fe3-4cbe-989b-5f7b909a9bb6,0.614258676725816,2025-11-05 03:09:25.559970,2025-11-05 03:09:26.502237,Beta-Cash-Stack-Model,v1,Quick,F,not applicable,android,Beta-Cash-Stack-Model_Trench3,Cash,Trench 3,"{""demo_score"": ""0.40911008935267046"", ""credo_s...",2025-11-05 11:09:22,NaT,2025-11,0.40911008935267046,0.1133,0.6327586149070384,0.5036715900930235
3,1502957,c36cb635-3f42-463f-a828-a4189f265970,0.6478110638835973,2025-10-24 04:30:56.184280,2025-10-24 04:30:57.073509,Beta-Cash-Stack-Model,v1,Quick,M,not applicable,android,Beta-Cash-Stack-Model_Trench3,Cash,Trench 3,"{""demo_score"": ""0.47698438254532394"", ""credo_s...",2025-10-24 12:30:52,NaT,2025-10,0.47698438254532394,0.1508,0.6448802135926262,0.47690510996864377
4,1564428,0f7f973b-8558-4480-8957-bc5742ffb998,0.3097051967548624,2025-10-22 10:52:28.646954,2025-10-22 10:52:29.349800,Beta-Cash-Stack-Model,v1,Quick,M,not applicable,ios,Beta-Cash-Stack-Model_Trench3,Cash,Trench 3,"{""demo_score"": ""0.3412514366272126"", ""credo_sc...",2025-10-22 18:52:25,NaT,2025-10,0.3412514366272126,0.0811,0.5711641739916185,NaN


In [94]:
df = expanded_df.drop(columns=['calcFeatures']).copy()

### run the psi calculation

In [95]:
df = df.copy()

# Define feature list
feature_list = ['Beta_cash_stack_score', 'calc_demo_score', 'calc_credo_score',  'calc_trx_score', 'calc_app_score']

# Define segment columns
segment_columns = ['new_loan_type', 'osType']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',   'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_stack_model_psi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_stack_model_csi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=a0d42c27-ce4b-469d-9a51-20b4f85dba7b>

In [96]:
psi_results

,modelDisplayName,Model_Name,modelVersionId,trenchCategory,Feature,Feature_Type,Segment_Column,Segment_Value,Month,Base_Month,Current_Month,PSI
0,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench3,v1,Trench 3,Beta_cash_stack_score,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000
1,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench3,v1,Trench 3,Beta_cash_stack_score,categorical,Overall,All,2025-10,2025-09,2025-10,0.508651
2,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench3,v1,Trench 3,Beta_cash_stack_score,categorical,Overall,All,2025-11,2025-09,2025-11,0.508651
3,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench3,v1,Trench 3,calc_demo_score,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000
4,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench3,v1,Trench 3,calc_demo_score,categorical,Overall,All,2025-10,2025-09,2025-10,0.508651
5,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench3,v1,Trench 3,calc_demo_score,categorical,Overall,All,2025-11,2025-09,2025-11,0.479138
6,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench3,v1,Trench 3,calc_credo_score,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000
7,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench3,v1,Trench 3,calc_credo_score,categorical,Overall,All,2025-10,2025-09,2025-10,0.649614
8,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench3,v1,Trench 3,calc_credo_score,categorical,Overall,All,2025-11,2025-09,2025-11,0.554865
9,Beta-Cash-Stack-Model,Beta-Cash-Stack-Model_Trench3,v1,Trench 3,calc_trx_score,categorical,Overall,All,2025-09,2025-09,2025-09,0.000000


# SIL

## Alpha - CIC-SIL-Model

In [97]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cic_sil_model_psi;"""
job = client.query(sq)
job.result()  # Wait for job to complete

sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cic_sil_model_csi;"""
job = client.query(sq)
job.result()  # Wait for job to complete


In [98]:
sq = """ 
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,

    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName = 'Alpha - CIC-SIL-Model')
SELECT
  r.customerId,r.digitalLoanAccountId,prediction Alpha_cic_sil_score
    ,start_time,end_time,modelDisplayName,modelVersionId,
   loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
        when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
        when lower(loanmaster.deviceType) like '%andro%' then 'android'
        else 'ios' end osType,
 'Alpha - CIC-SIL-Model' Model_Name,
 'SIL' as product,
 'NA' trenchCategory,
  r.calcFeature calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,  
FROM cleaned r 
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by r.start_time desc) = 1
;
"""

dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,1361379,ac3852be-c7fa-447c-8060-6c0e7e3844c1,0.20537640725746512,2025-11-16 03:09:12.680639,2025-11-16 03:09:12.687160,Alpha - CIC-SIL-Model,v1,SIL-Instore,F,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1763251200000,""cic_Personal_Loans_...",2025-11-16 11:09:03,NaT,2025-11
1,1742321,d06324b8-4fc3-4f2c-9c64-4726bef2f6ad,0.12273705119029994,2025-11-15 10:51:28.346798,2025-11-15 10:51:28.352988,Alpha - CIC-SIL-Model,v1,SIL-Instore,M,Mall,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1763164800000,""cic_Personal_Loans_...",2025-11-15 18:51:16,2025-11-15 19:00:40,2025-11
2,1863838,4b39ca54-deff-4b76-84fc-5271e4474460,0.1963176138149603,2025-10-24 09:39:56.422441,2025-10-24 09:39:56.428838,Alpha - CIC-SIL-Model,v1,SIL-Instore,M,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1761264000000,""cic_Personal_Loans_...",2025-10-24 17:39:44,NaT,2025-10
3,1871448,24128d97-f631-4891-971f-438a3b0dafcb,0.12707932677276707,2025-09-29 10:23:12.020020,2025-09-29 10:23:12.027041,Alpha - CIC-SIL-Model,v1,SIL Competitor,M,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1759104000000,""cic_Personal_Loans_...",2025-09-29 18:23:03,2025-09-29 18:25:37,2025-09
4,2010073,7e87e454-079a-4621-af55-830f074a89d9,0.08796182675303948,2025-07-09 08:55:00.372299,2025-07-09 08:55:00.378773,Alpha - CIC-SIL-Model,v1,SIL-Instore,F,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1752019200000,""cic_Personal_Loans_...",2025-07-09 16:54:49,2025-07-09 17:04:55,2025-07


### Expand df

In [99]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 18
Expanded columns: 28


,customerId,digitalLoanAccountId,Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month,calc_run_date,calc_cic_Personal_Loans_granted_contracts_amt_24M,calc_cic_days_since_last_inquiry,calc_cic_cnt_active_contracts,calc_cic_vel_contract_nongranted_cnt_12on24,calc_cic_max_amt_granted_24M,calc_cic_zero_non_granted_ever_flag,calc_cic_tot_active_contracts_util,calc_cic_vel_contract_granted_amt_12on24,calc_cic_zero_granted_ever_flag
0,1361379,ac3852be-c7fa-447c-8060-6c0e7e3844c1,0.20537640725746512,2025-11-16 03:09:12.680639,2025-11-16 03:09:12.687160,Alpha - CIC-SIL-Model,v1,SIL-Instore,F,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1763251200000,""cic_Personal_Loans_...",2025-11-16 11:09:03,NaT,2025-11,1.763251e+12,NaN,17.0,NaN,1.00000,NaN,0,NaN,NaN,0
1,1742321,d06324b8-4fc3-4f2c-9c64-4726bef2f6ad,0.12273705119029994,2025-11-15 10:51:28.346798,2025-11-15 10:51:28.352988,Alpha - CIC-SIL-Model,v1,SIL-Instore,M,Mall,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1763164800000,""cic_Personal_Loans_...",2025-11-15 18:51:16,2025-11-15 19:00:40,2025-11,1.763165e+12,NaN,121.0,NaN,NaN,7500.0,1,NaN,1.000000,0
2,1863838,4b39ca54-deff-4b76-84fc-5271e4474460,0.1963176138149603,2025-10-24 09:39:56.422441,2025-10-24 09:39:56.428838,Alpha - CIC-SIL-Model,v1,SIL-Instore,M,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1761264000000,""cic_Personal_Loans_...",2025-10-24 17:39:44,NaT,2025-10,1.761264e+12,NaN,2.0,NaN,1.97619,NaN,0,NaN,NaN,0
3,1871448,24128d97-f631-4891-971f-438a3b0dafcb,0.12707932677276707,2025-09-29 10:23:12.020020,2025-09-29 10:23:12.027041,Alpha - CIC-SIL-Model,v1,SIL Competitor,M,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1759104000000,""cic_Personal_Loans_...",2025-09-29 18:23:03,2025-09-29 18:25:37,2025-09,1.759104e+12,NaN,189.0,1.0,NaN,6648.0,0,0.428099,1.000000,0
4,2010073,7e87e454-079a-4621-af55-830f074a89d9,0.08796182675303948,2025-07-09 08:55:00.372299,2025-07-09 08:55:00.378773,Alpha - CIC-SIL-Model,v1,SIL-Instore,F,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1752019200000,""cic_Personal_Loans_...",2025-07-09 16:54:49,2025-07-09 17:04:55,2025-07,1.752019e+12,112229.0,85.0,4.0,1.97619,60000.0,0,0.972706,1.411797,0


In [100]:
df = expanded_df.drop(columns=['calcFeatures']).copy()

In [101]:
df['Alpha_cic_sil_score'] = pd.to_numeric(df['Alpha_cic_sil_score'], errors='coerce')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81402 entries, 0 to 81401
Data columns (total 27 columns):
 #   Column                                             Non-Null Count  Dtype         
---  ------                                             --------------  -----         
 0   customerId                                         81402 non-null  object        
 1   digitalLoanAccountId                               81402 non-null  object        
 2   Alpha_cic_sil_score                                81402 non-null  float64       
 3   start_time                                         81402 non-null  datetime64[us]
 4   end_time                                           81402 non-null  datetime64[us]
 5   modelDisplayName                                   81402 non-null  object        
 6   modelVersionId                                     81402 non-null  object        
 7   new_loan_type                                      81329 non-null  object        
 8   gender          

### run the psi calculation

In [102]:
df = df.copy()

# Define feature list
feature_list = ['Alpha_cic_sil_score',  'calc_cic_Personal_Loans_granted_contracts_amt_24M',
       'calc_cic_days_since_last_inquiry', 'calc_cic_cnt_active_contracts',
       'calc_cic_vel_contract_nongranted_cnt_12on24',
       'calc_cic_max_amt_granted_24M', 'calc_cic_zero_non_granted_ever_flag',
       'calc_cic_tot_active_contracts_util',
       'calc_cic_vel_contract_granted_amt_12on24',
       'calc_cic_zero_granted_ever_flag']

# Define segment columns
segment_columns = ['new_loan_type','osType', 'loan_product_type']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',   'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cic_sil_model_psi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cic_sil_model_csi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=ac0968ab-7912-447b-863f-2230e2d6cc31>

In [103]:
psi_results

,modelDisplayName,Model_Name,modelVersionId,trenchCategory,Feature,Feature_Type,Segment_Column,Segment_Value,Month,Base_Month,Current_Month,PSI
0,Alpha - CIC-SIL-Model,Alpha - CIC-SIL-Model,v1,NA,Alpha_cic_sil_score,numerical,Overall,All,2025-03,2025-03,2025-03,0.000000
1,Alpha - CIC-SIL-Model,Alpha - CIC-SIL-Model,v1,NA,Alpha_cic_sil_score,numerical,Overall,All,2025-04,2025-03,2025-04,0.115764
2,Alpha - CIC-SIL-Model,Alpha - CIC-SIL-Model,v1,NA,Alpha_cic_sil_score,numerical,Overall,All,2025-05,2025-03,2025-05,0.049776
3,Alpha - CIC-SIL-Model,Alpha - CIC-SIL-Model,v1,NA,Alpha_cic_sil_score,numerical,Overall,All,2025-06,2025-03,2025-06,0.085705
4,Alpha - CIC-SIL-Model,Alpha - CIC-SIL-Model,v1,NA,Alpha_cic_sil_score,numerical,Overall,All,2025-07,2025-03,2025-07,0.093254
...,...,...,...,...,...,...,...,...,...,...,...,...
625,Alpha - CIC-SIL-Model,Alpha - CIC-SIL-Model,v1,NA,calc_cic_zero_granted_ever_flag,categorical,loan_product_type,Mall,2025-07,2025-03,2025-07,0.000000
626,Alpha - CIC-SIL-Model,Alpha - CIC-SIL-Model,v1,NA,calc_cic_zero_granted_ever_flag,categorical,loan_product_type,Mall,2025-08,2025-03,2025-08,0.000000
627,Alpha - CIC-SIL-Model,Alpha - CIC-SIL-Model,v1,NA,calc_cic_zero_granted_ever_flag,categorical,loan_product_type,Mall,2025-09,2025-03,2025-09,0.000000
628,Alpha - CIC-SIL-Model,Alpha - CIC-SIL-Model,v1,NA,calc_cic_zero_granted_ever_flag,categorical,loan_product_type,Mall,2025-10,2025-03,2025-10,0.000000


## Alpha - IncomeEstimationModel

In [104]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_income_estimation_model_psi;"""
job = client.query(sq)
job.result()  # Wait for job to complete

sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_income_estimation_model_csi;"""
job = client.query(sq)
job.result()  # Wait for job to complete



In [105]:
sq = """ 
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName = 'Alpha  - IncomeEstimationModel')
SELECT
  r.customerId,r.digitalLoanAccountId,prediction Alpha_Income_Estimated_score,start_time,end_time,modelDisplayName,modelVersionId,
  loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
  'Alpha  - IncomeEstimationModel' Model_Name,
  'SIL' as product,
  'NA' trenchCategory,
  r.calcFeature calcFeatures,
  JSON_VALUE(calcFeature, "$.inc_alpha_cic_credit_avg_credit_limit") AS inc_alpha_cic_credit_avg_credit_limit,
  JSON_VALUE(calcFeature, "$.inc_alpha_cic_max_active_contracts_amt") AS inc_alpha_cic_max_active_contracts_amt,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_age") AS inc_alpha_ln_age,
  JSON_VALUE(calcFeature, "$.inc_alpha_doc_type_rolled") AS inc_alpha_doc_type_rolled,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_brand") AS inc_alpha_ln_brand,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_city") AS inc_alpha_ln_city,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_cnt_dependents") AS inc_alpha_ln_cnt_dependents,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_education_level") AS inc_alpha_ln_education_level,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_employment_type_new") AS inc_alpha_ln_employment_type_new,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_gender") AS inc_alpha_ln_gender,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_industry_new") AS inc_alpha_ln_industry_new,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_loan_prod_type") AS inc_alpha_ln_loan_prod_type,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_marital_status_new") AS inc_alpha_ln_marital_status_new,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_nature_of_work_new") AS inc_alpha_ln_nature_of_work_new,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_osversion_bin") AS inc_alpha_ln_osversion_bin,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_purpose") AS inc_alpha_ln_purpose,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_source_of_funds_new") AS inc_alpha_ln_source_of_funds_new,
  JSON_VALUE(calcFeature, "$.inc_alpha_encoded_company_name_grouped") AS inc_alpha_encoded_company_name_grouped,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,
FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by r.start_time desc) = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,Alpha_Income_Estimated_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,inc_alpha_cic_credit_avg_credit_limit,inc_alpha_cic_max_active_contracts_amt,inc_alpha_ln_age,inc_alpha_doc_type_rolled,inc_alpha_ln_brand,inc_alpha_ln_city,inc_alpha_ln_cnt_dependents,inc_alpha_ln_education_level,inc_alpha_ln_employment_type_new,inc_alpha_ln_gender,inc_alpha_ln_industry_new,inc_alpha_ln_loan_prod_type,inc_alpha_ln_marital_status_new,inc_alpha_ln_nature_of_work_new,inc_alpha_ln_osversion_bin,inc_alpha_ln_purpose,inc_alpha_ln_source_of_funds_new,inc_alpha_encoded_company_name_grouped,appln_submit_datetime,disbursementDateTime,Application_month
0,3350084,9cc3fa32-3d62-4453-b5ee-5bedf1773cf0,28151.50062741188,2025-03-30 11:53:18.740713,2025-03-30 11:53:18.809926,Alpha - IncomeEstimationModel,v1,SIL-Instore,F,Mall,ios,Alpha - IncomeEstimationModel,SIL,NA,"{""inc_alpha_cic_credit_avg_credit_limit"":18000...",18000,75000.0,27,Other,Apple,CF1691,0,College Graduate,Employed,F,Call Center Agent/Tele Marketer,BNPL,Single,Call Center Agent/Tele Marketer,5-Pioneer,"Washing machine, dryer or both",1,352645.3476271578,2025-03-30 19:53:10,2025-03-30 20:09:52,2025-03
1,3354885,ec5ecea5-05d9-446a-a890-bb46333d8183,27771.991842566462,2025-04-02 06:01:15.415118,2025-04-02 06:01:15.488014,Alpha - IncomeEstimationModel,v1,SIL-Instore,M,Appliance,android,Alpha - IncomeEstimationModel,SIL,NA,"{""inc_alpha_cic_credit_avg_credit_limit"":0,""in...",0,0.0,29,Other,vivo,CF1377,0,College Undergraduate,Employed,M,Other Non Professional Services,BNPL,Single,Other Non Professional Services,2-Voyager,Mobile Phones,1,191064.2343503157,2025-04-02 14:01:05,2025-04-09 11:10:13,2025-04
2,3439875,5c8753ec-69b3-4615-b203-90710c880127,46448.76704685991,2025-09-19 04:37:08.581639,2025-09-19 04:37:08.665424,Alpha - IncomeEstimationModel,v1,SIL-Instore,M,Appliance,android,Alpha - IncomeEstimationModel,SIL,NA,"{""inc_alpha_cic_credit_avg_credit_limit"":0,""in...",0,916000.0,50,Driving License,realme,CF0712,1,College Undergraduate,Employed - Govt. Employee,M,Govt. Institution,BNPL,Single,Government - Uniformed Service,2-Voyager,Mobile Phones,Salary,224335.8707139521,2025-09-19 12:36:59,NaT,2025-09
3,3469557,3cbcc964-236a-41cb-9c3a-b120645d4766,8895.756184689211,2025-06-09 10:39:08.363735,2025-06-09 10:39:08.437638,Alpha - IncomeEstimationModel,v1,SIL-Instore,F,Appliance,android,Alpha - IncomeEstimationModel,SIL,NA,"{""inc_alpha_cic_credit_avg_credit_limit"":0,""in...",0,5000,47,ID Card,Infinix,CF1441,5,High School Graduate,Employed - Private Employee,F,Wholesale and Retail Trade,BNPL,Married,Broker/Dealer,2-Voyager,Mobile Phones,Salary,0.0,2025-05-31 14:14:05,NaT,2025-05
4,3577779,568cef39-1097-46a9-bd9a-45a98824ca7e,18114.58104803613,2025-07-25 08:35:50.554609,2025-07-25 08:35:50.629414,Alpha - IncomeEstimationModel,v1,SIL-Instore,M,Appliance,android,Alpha - IncomeEstimationModel,SIL,NA,"{""inc_alpha_cic_credit_avg_credit_limit"":0,""in...",0,0.0,33,Driving License,HONOR,CF0063,0,College Undergraduate,Employed - Govt. Employee,M,Govt. Institution,BNPL,Single,Government - Appointed Official,4-Innovator,Televisions,Salary,30086.7246045263,2025-07-25 16:35:42,NaT,2025-07


### Expand df


In [106]:
# Expand the calcFeatures column
expanded_df = dfd.copy()

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 36
Expanded columns: 36


,customerId,digitalLoanAccountId,Alpha_Income_Estimated_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,inc_alpha_cic_credit_avg_credit_limit,inc_alpha_cic_max_active_contracts_amt,inc_alpha_ln_age,inc_alpha_doc_type_rolled,inc_alpha_ln_brand,inc_alpha_ln_city,inc_alpha_ln_cnt_dependents,inc_alpha_ln_education_level,inc_alpha_ln_employment_type_new,inc_alpha_ln_gender,inc_alpha_ln_industry_new,inc_alpha_ln_loan_prod_type,inc_alpha_ln_marital_status_new,inc_alpha_ln_nature_of_work_new,inc_alpha_ln_osversion_bin,inc_alpha_ln_purpose,inc_alpha_ln_source_of_funds_new,inc_alpha_encoded_company_name_grouped,appln_submit_datetime,disbursementDateTime,Application_month
0,3350084,9cc3fa32-3d62-4453-b5ee-5bedf1773cf0,28151.50062741188,2025-03-30 11:53:18.740713,2025-03-30 11:53:18.809926,Alpha - IncomeEstimationModel,v1,SIL-Instore,F,Mall,ios,Alpha - IncomeEstimationModel,SIL,NA,"{""inc_alpha_cic_credit_avg_credit_limit"":18000...",18000,75000.0,27,Other,Apple,CF1691,0,College Graduate,Employed,F,Call Center Agent/Tele Marketer,BNPL,Single,Call Center Agent/Tele Marketer,5-Pioneer,"Washing machine, dryer or both",1,352645.3476271578,2025-03-30 19:53:10,2025-03-30 20:09:52,2025-03
1,3354885,ec5ecea5-05d9-446a-a890-bb46333d8183,27771.991842566462,2025-04-02 06:01:15.415118,2025-04-02 06:01:15.488014,Alpha - IncomeEstimationModel,v1,SIL-Instore,M,Appliance,android,Alpha - IncomeEstimationModel,SIL,NA,"{""inc_alpha_cic_credit_avg_credit_limit"":0,""in...",0,0.0,29,Other,vivo,CF1377,0,College Undergraduate,Employed,M,Other Non Professional Services,BNPL,Single,Other Non Professional Services,2-Voyager,Mobile Phones,1,191064.2343503157,2025-04-02 14:01:05,2025-04-09 11:10:13,2025-04
2,3439875,5c8753ec-69b3-4615-b203-90710c880127,46448.76704685991,2025-09-19 04:37:08.581639,2025-09-19 04:37:08.665424,Alpha - IncomeEstimationModel,v1,SIL-Instore,M,Appliance,android,Alpha - IncomeEstimationModel,SIL,NA,"{""inc_alpha_cic_credit_avg_credit_limit"":0,""in...",0,916000.0,50,Driving License,realme,CF0712,1,College Undergraduate,Employed - Govt. Employee,M,Govt. Institution,BNPL,Single,Government - Uniformed Service,2-Voyager,Mobile Phones,Salary,224335.8707139521,2025-09-19 12:36:59,NaT,2025-09
3,3469557,3cbcc964-236a-41cb-9c3a-b120645d4766,8895.756184689211,2025-06-09 10:39:08.363735,2025-06-09 10:39:08.437638,Alpha - IncomeEstimationModel,v1,SIL-Instore,F,Appliance,android,Alpha - IncomeEstimationModel,SIL,NA,"{""inc_alpha_cic_credit_avg_credit_limit"":0,""in...",0,5000,47,ID Card,Infinix,CF1441,5,High School Graduate,Employed - Private Employee,F,Wholesale and Retail Trade,BNPL,Married,Broker/Dealer,2-Voyager,Mobile Phones,Salary,0.0,2025-05-31 14:14:05,NaT,2025-05
4,3577779,568cef39-1097-46a9-bd9a-45a98824ca7e,18114.58104803613,2025-07-25 08:35:50.554609,2025-07-25 08:35:50.629414,Alpha - IncomeEstimationModel,v1,SIL-Instore,M,Appliance,android,Alpha - IncomeEstimationModel,SIL,NA,"{""inc_alpha_cic_credit_avg_credit_limit"":0,""in...",0,0.0,33,Driving License,HONOR,CF0063,0,College Undergraduate,Employed - Govt. Employee,M,Govt. Institution,BNPL,Single,Government - Appointed Official,4-Innovator,Televisions,Salary,30086.7246045263,2025-07-25 16:35:42,NaT,2025-07


In [107]:
df = expanded_df.drop(columns=['calcFeatures']).copy()

In [108]:
df['Alpha_Income_Estimated_score'] = pd.to_numeric(df['Alpha_Income_Estimated_score'], errors='coerce')
df.columns

Index(['customerId', 'digitalLoanAccountId', 'Alpha_Income_Estimated_score',
       'start_time', 'end_time', 'modelDisplayName', 'modelVersionId',
       'new_loan_type', 'gender', 'loan_product_type', 'osType', 'Model_Name',
       'product', 'trenchCategory', 'inc_alpha_cic_credit_avg_credit_limit',
       'inc_alpha_cic_max_active_contracts_amt', 'inc_alpha_ln_age',
       'inc_alpha_doc_type_rolled', 'inc_alpha_ln_brand', 'inc_alpha_ln_city',
       'inc_alpha_ln_cnt_dependents', 'inc_alpha_ln_education_level',
       'inc_alpha_ln_employment_type_new', 'inc_alpha_ln_gender',
       'inc_alpha_ln_industry_new', 'inc_alpha_ln_loan_prod_type',
       'inc_alpha_ln_marital_status_new', 'inc_alpha_ln_nature_of_work_new',
       'inc_alpha_ln_osversion_bin', 'inc_alpha_ln_purpose',
       'inc_alpha_ln_source_of_funds_new',
       'inc_alpha_encoded_company_name_grouped', 'appln_submit_datetime',
       'disbursementDateTime', 'Application_month'],
      dtype='object')

### run psi calculation

In [109]:
df = df.copy()

# Define feature list
feature_list = ['Alpha_Income_Estimated_score',   'inc_alpha_cic_credit_avg_credit_limit',
       'inc_alpha_cic_max_active_contracts_amt', 'inc_alpha_ln_age',
       'inc_alpha_doc_type_rolled', 'inc_alpha_ln_brand', 'inc_alpha_ln_city',
       'inc_alpha_ln_cnt_dependents', 'inc_alpha_ln_education_level',
       'inc_alpha_ln_employment_type_new', 'inc_alpha_ln_gender',
       'inc_alpha_ln_industry_new', 'inc_alpha_ln_loan_prod_type',
       'inc_alpha_ln_marital_status_new', 'inc_alpha_ln_nature_of_work_new',
       'inc_alpha_ln_osversion_bin', 'inc_alpha_ln_purpose',
       'inc_alpha_ln_source_of_funds_new',
       'inc_alpha_encoded_company_name_grouped',]

# Define segment columns
segment_columns = ['new_loan_type', 'osType', 'loan_product_type']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',   'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_income_estimation_model_psi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_income_estimation_model_csi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=5f503ec4-903e-497f-9f30-6b16e9945f56>

In [110]:
psi_results

,modelDisplayName,Model_Name,modelVersionId,trenchCategory,Feature,Feature_Type,Segment_Column,Segment_Value,Month,Base_Month,Current_Month,PSI
0,Alpha - IncomeEstimationModel,Alpha - IncomeEstimationModel,v1,NA,Alpha_Income_Estimated_score,numerical,Overall,All,2025-03,2025-03,2025-03,0.000000
1,Alpha - IncomeEstimationModel,Alpha - IncomeEstimationModel,v1,NA,Alpha_Income_Estimated_score,numerical,Overall,All,2025-04,2025-03,2025-04,0.128999
2,Alpha - IncomeEstimationModel,Alpha - IncomeEstimationModel,v1,NA,Alpha_Income_Estimated_score,numerical,Overall,All,2025-05,2025-03,2025-05,0.153427
3,Alpha - IncomeEstimationModel,Alpha - IncomeEstimationModel,v1,NA,Alpha_Income_Estimated_score,numerical,Overall,All,2025-06,2025-03,2025-06,0.378478
4,Alpha - IncomeEstimationModel,Alpha - IncomeEstimationModel,v1,NA,Alpha_Income_Estimated_score,numerical,Overall,All,2025-07,2025-03,2025-07,0.356047
...,...,...,...,...,...,...,...,...,...,...,...,...
1192,Alpha - IncomeEstimationModel,Alpha - IncomeEstimationModel,v1,NA,inc_alpha_encoded_company_name_grouped,categorical,loan_product_type,Appliance,2025-07,2025-03,2025-07,0.187582
1193,Alpha - IncomeEstimationModel,Alpha - IncomeEstimationModel,v1,NA,inc_alpha_encoded_company_name_grouped,categorical,loan_product_type,Appliance,2025-08,2025-03,2025-08,0.213297
1194,Alpha - IncomeEstimationModel,Alpha - IncomeEstimationModel,v1,NA,inc_alpha_encoded_company_name_grouped,categorical,loan_product_type,Appliance,2025-09,2025-03,2025-09,0.211354
1195,Alpha - IncomeEstimationModel,Alpha - IncomeEstimationModel,v1,NA,inc_alpha_encoded_company_name_grouped,categorical,loan_product_type,Appliance,2025-10,2025-03,2025-10,0.228473


## SIL Alpha - StackingModel

In [111]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_sil_stack_model_psi;"""
job = client.query(sq)
job.result()  # Wait for job to complete


sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_sil_stack_model_csi;"""
job = client.query(sq)
job.result()  # Wait for job to complete


In [112]:
sq = """ WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName = 'Alpha - StackingModel')
SELECT distinct
 r.customerId,r.digitalLoanAccountId,prediction Sil_Alpha_Stack_score,start_time,end_time,modelDisplayName,modelVersionId,
 loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'SIL Alpha - StackingModel' Model_Name,
 'SIL' as product,
 'NA' trenchCategory,
  r.calcFeature calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,  
FROM cleaned r 
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over(partition by r.digitalLoanAccountId order by r.start_time desc)=1
  ;"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,Sil_Alpha_Stack_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,2785572,538a12bb-462f-4f4f-8d2d-02c4d4048101,0.013275318507588315,2025-03-26 10:41:54.225164,2025-03-26 10:41:54.228681,Alpha - StackingModel,v1,SIL-Instore,M,Appliance,android,SIL Alpha - StackingModel,SIL,NA,"{""sb_demo_score"": null, ""s_cic_score"": 0.16284...",2025-03-26 18:41:39,NaT,2025-03
1,3350094,12370c03-e3f8-4fc2-9ba8-3617c15df74e,0.010592212216220547,2025-03-30 12:02:37.098667,2025-03-30 12:02:37.102159,Alpha - StackingModel,v1,SIL-Instore,F,Mall,android,SIL Alpha - StackingModel,SIL,NA,"{""sb_demo_score"": null, ""s_cic_score"": 0.14056...",2025-03-30 20:02:28,2025-03-30 20:34:49,2025-03
2,3342818,0e03b5b7-7201-4baf-ba27-b5df2844e574,0.010051546227182639,2025-03-27 02:18:08.320467,2025-03-27 02:18:08.323805,Alpha - StackingModel,v1,SIL-Instore,M,Appliance,android,SIL Alpha - StackingModel,SIL,NA,"{""sb_demo_score"": null, ""s_cic_score"": 0.11839...",2025-03-27 10:17:59,2025-03-27 10:25:36,2025-03
3,3346741,7a381a37-49dc-4811-b890-723db1a2a746,0.016745149713544932,2025-03-29 04:50:25.504160,2025-03-29 04:50:25.507739,Alpha - StackingModel,v1,SIL-Instore,F,Appliance,android,SIL Alpha - StackingModel,SIL,NA,"{""sb_demo_score"": null, ""s_cic_score"": 0.20395...",2025-03-29 12:50:15,2025-03-29 13:06:15,2025-03
4,3347304,32937f1c-53e7-4e34-b6ed-f9823b14c6f3,0.010309960448179863,2025-03-29 08:00:14.208879,2025-03-29 08:00:14.212537,Alpha - StackingModel,v1,SIL-Instore,F,Appliance,android,SIL Alpha - StackingModel,SIL,NA,"{""sb_demo_score"": null, ""s_cic_score"": 0.12556...",2025-03-29 16:00:03,2025-03-29 16:03:55,2025-03


### Expand df

In [113]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 18
Expanded columns: 22


,customerId,digitalLoanAccountId,Sil_Alpha_Stack_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month,calc_sb_demo_score,calc_s_cic_score,calc_s_credo_score,calc_s_apps_score
0,2785572,538a12bb-462f-4f4f-8d2d-02c4d4048101,0.013275318507588315,2025-03-26 10:41:54.225164,2025-03-26 10:41:54.228681,Alpha - StackingModel,v1,SIL-Instore,M,Appliance,android,SIL Alpha - StackingModel,SIL,NA,"{""sb_demo_score"": null, ""s_cic_score"": 0.16284...",2025-03-26 18:41:39,NaT,2025-03,NaN,0.162843,NaN,NaN
1,3350094,12370c03-e3f8-4fc2-9ba8-3617c15df74e,0.010592212216220547,2025-03-30 12:02:37.098667,2025-03-30 12:02:37.102159,Alpha - StackingModel,v1,SIL-Instore,F,Mall,android,SIL Alpha - StackingModel,SIL,NA,"{""sb_demo_score"": null, ""s_cic_score"": 0.14056...",2025-03-30 20:02:28,2025-03-30 20:34:49,2025-03,NaN,0.140566,NaN,NaN
2,3342818,0e03b5b7-7201-4baf-ba27-b5df2844e574,0.010051546227182639,2025-03-27 02:18:08.320467,2025-03-27 02:18:08.323805,Alpha - StackingModel,v1,SIL-Instore,M,Appliance,android,SIL Alpha - StackingModel,SIL,NA,"{""sb_demo_score"": null, ""s_cic_score"": 0.11839...",2025-03-27 10:17:59,2025-03-27 10:25:36,2025-03,NaN,0.118395,NaN,NaN
3,3346741,7a381a37-49dc-4811-b890-723db1a2a746,0.016745149713544932,2025-03-29 04:50:25.504160,2025-03-29 04:50:25.507739,Alpha - StackingModel,v1,SIL-Instore,F,Appliance,android,SIL Alpha - StackingModel,SIL,NA,"{""sb_demo_score"": null, ""s_cic_score"": 0.20395...",2025-03-29 12:50:15,2025-03-29 13:06:15,2025-03,NaN,0.203954,NaN,NaN
4,3347304,32937f1c-53e7-4e34-b6ed-f9823b14c6f3,0.010309960448179863,2025-03-29 08:00:14.208879,2025-03-29 08:00:14.212537,Alpha - StackingModel,v1,SIL-Instore,F,Appliance,android,SIL Alpha - StackingModel,SIL,NA,"{""sb_demo_score"": null, ""s_cic_score"": 0.12556...",2025-03-29 16:00:03,2025-03-29 16:03:55,2025-03,NaN,0.125561,NaN,NaN


In [114]:
df = expanded_df.drop(columns=['calcFeatures']).copy()
df = df.drop_duplicates(keep='first')

In [115]:
df['Sil_Alpha_Stack_score'] = pd.to_numeric(df['Sil_Alpha_Stack_score'], errors='coerce')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81402 entries, 0 to 81401
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             81402 non-null  object        
 1   digitalLoanAccountId   81402 non-null  object        
 2   Sil_Alpha_Stack_score  81402 non-null  float64       
 3   start_time             81402 non-null  datetime64[us]
 4   end_time               81402 non-null  datetime64[us]
 5   modelDisplayName       81402 non-null  object        
 6   modelVersionId         81402 non-null  object        
 7   new_loan_type          81329 non-null  object        
 8   gender                 81329 non-null  object        
 9   loan_product_type      81402 non-null  object        
 10  osType                 81402 non-null  object        
 11  Model_Name             81402 non-null  object        
 12  product                81402 non-null  object        
 13  t

### run psi calculation

In [116]:
df = df.copy()

# Define feature list
feature_list = ['Sil_Alpha_Stack_score',   'calc_sb_demo_score',
       'calc_s_cic_score', 'calc_s_credo_score', 'calc_s_apps_score']

# Define segment columns
segment_columns = ['new_loan_type','osType', 'loan_product_type']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',   'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_sil_stack_model_psi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_sil_stack_model_csi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=d401870d-2fa2-4daa-9230-c78f86c9e7ba>

In [117]:
psi_results

,modelDisplayName,Model_Name,modelVersionId,trenchCategory,Feature,Feature_Type,Segment_Column,Segment_Value,Month,Base_Month,Current_Month,PSI
0,Alpha - StackingModel,SIL Alpha - StackingModel,v1,NA,Sil_Alpha_Stack_score,numerical,Overall,All,2025-03,2025-03,2025-03,0.000000
1,Alpha - StackingModel,SIL Alpha - StackingModel,v1,NA,Sil_Alpha_Stack_score,numerical,Overall,All,2025-04,2025-03,2025-04,2.268342
2,Alpha - StackingModel,SIL Alpha - StackingModel,v1,NA,Sil_Alpha_Stack_score,numerical,Overall,All,2025-05,2025-03,2025-05,5.939222
3,Alpha - StackingModel,SIL Alpha - StackingModel,v1,NA,Sil_Alpha_Stack_score,numerical,Overall,All,2025-06,2025-03,2025-06,5.260444
4,Alpha - StackingModel,SIL Alpha - StackingModel,v1,NA,Sil_Alpha_Stack_score,numerical,Overall,All,2025-07,2025-03,2025-07,5.356686
...,...,...,...,...,...,...,...,...,...,...,...,...
310,Alpha - StackingModel,SIL Alpha - StackingModel,v1,NA,calc_s_apps_score,numerical,loan_product_type,Mall,2025-07,2025-03,2025-07,0.000000
311,Alpha - StackingModel,SIL Alpha - StackingModel,v1,NA,calc_s_apps_score,numerical,loan_product_type,Mall,2025-08,2025-03,2025-08,0.000000
312,Alpha - StackingModel,SIL Alpha - StackingModel,v1,NA,calc_s_apps_score,numerical,loan_product_type,Mall,2025-09,2025-03,2025-09,0.000000
313,Alpha - StackingModel,SIL Alpha - StackingModel,v1,NA,calc_s_apps_score,numerical,loan_product_type,Mall,2025-10,2025-03,2025-10,0.000000


## SIL Beta - AppsScoreModel

In [118]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_appscore_model_psi;"""
job = client.query(sq)
job.result()  # Wait for job to complete

sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_appscore_model_csi;"""
job = client.query(sq)
job.result()  # Wait for job to complete

In [119]:
sq = """ 
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,

    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName = 'Beta - AppsScoreModel')
SELECT
  r.customerId,r.digitalLoanAccountId,prediction,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'SIL Beta - AppsScoreModel' Model_Name,
 'SIL' as product,
  'NA' trenchCategory,
  safe_cast(JSON_VALUE(prediction_clean, "$.combined_score") AS float64) as sil_beta_app_score,
 calcFeature calcFeatures, 
    coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
    loanmaster.disbursementDateTime,
    format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,
 FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by start_time desc) = 1
;
"""
dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,sil_beta_app_score,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,1236079,4c0b90e4-383c-4fb9-83fd-106b3972ce5c,"{'cb_score': 0.43545740457781545, 'dl_score': ...",2025-10-15 06:14:56.797744,2025-10-15 06:14:57.317040,Beta - AppsScoreModel,v1,SIL Competitor,F,Appliance,android,SIL Beta - AppsScoreModel,SIL,NA,0.483320,"{""app_cnt_rated_for_3plus_ever"":40,""app_cnt_ed...",2025-10-15 14:14:53,NaT,2025-10
1,1279916,a3ad7810-ac4f-459c-b88e-02958773529c,"{'cb_score': 0.4433809464360384, 'dl_score': 0...",2025-07-06 05:05:26.872353,2025-07-06 05:05:27.310573,Beta - AppsScoreModel,v1,SIL Competitor,F,Appliance,android,SIL Beta - AppsScoreModel,SIL,NA,0.560750,"{""app_cnt_rated_for_3plus_ever"":31,""app_cnt_ed...",2025-07-06 13:05:24,NaT,2025-07
2,1508888,bd1c1259-f630-4712-a8df-fb211a9b7059,"{'cb_score': 0.34493482516754004, 'dl_score': ...",2025-06-21 10:15:30.792002,2025-06-21 10:15:31.244757,Beta - AppsScoreModel,v1,SIL Competitor,F,Appliance,android,SIL Beta - AppsScoreModel,SIL,NA,0.505210,"{""app_cnt_rated_for_3plus_ever"":20,""app_cnt_ed...",2025-06-21 18:15:29,NaT,2025-06
3,1976352,1caec6aa-3c9c-43a9-9939-57e81592f47c,"{'cb_score': 0.5601615539152716, 'dl_score': 0...",2025-04-16 08:27:06.570807,2025-04-16 08:27:07.008496,Beta - AppsScoreModel,v1,SIL-Instore,M,Appliance,android,SIL Beta - AppsScoreModel,SIL,NA,0.666355,"{""app_cnt_rated_for_3plus_ever"":36,""app_cnt_ed...",2025-04-16 16:27:03,NaT,2025-04
4,2082698,edc67e5d-b7f7-4a2c-a05f-93cb218acec8,"{'cb_score': 0.5666707533584865, 'dl_score': 0...",2025-07-20 06:44:33.478087,2025-07-20 06:44:33.924970,Beta - AppsScoreModel,v1,SIL Competitor,M,Appliance,android,SIL Beta - AppsScoreModel,SIL,NA,0.459619,"{""app_cnt_rated_for_3plus_ever"":28,""app_cnt_ed...",2025-07-20 14:44:29,NaT,2025-07


### Expand df

In [120]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 19
Expanded columns: 37


,customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,sil_beta_app_score,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month,calc_app_cnt_rated_for_3plus_ever,calc_app_cnt_education_ever,calc_app_cnt_business_ever,calc_app_cnt_music_and_audio_ever,calc_app_cnt_travel_and_local_ever,calc_app_cnt_finance_7d,calc_app_cnt_absence_tag_30d,calc_app_cnt_finance_30d,calc_app_cnt_competitors_30d,calc_app_cnt_absence_tag_90d,calc_app_cnt_finance_90d,calc_app_cnt_competitors_90d,calc_app_cnt_payday_90d,calc_app_avg_time_bw_installed_mins_30d,calc_app_median_time_bw_installed_mins_30d,calc_app_first_competitors_install_to_apply_days,calc_app_first_payday_install_to_apply_days,calc_app_vel_finance_30_over_365
0,1236079,4c0b90e4-383c-4fb9-83fd-106b3972ce5c,"{'cb_score': 0.43545740457781545, 'dl_score': ...",2025-10-15 06:14:56.797744,2025-10-15 06:14:57.317040,Beta - AppsScoreModel,v1,SIL Competitor,F,Appliance,android,SIL Beta - AppsScoreModel,SIL,NA,0.483320,"{""app_cnt_rated_for_3plus_ever"":40,""app_cnt_ed...",2025-10-15 14:14:53,NaT,2025-10,40,0,3,1,1,0,2,1,0,3,2,2,1,8625.400000,8716.375000,945.0,945.0,0.125000
1,1279916,a3ad7810-ac4f-459c-b88e-02958773529c,"{'cb_score': 0.4433809464360384, 'dl_score': 0...",2025-07-06 05:05:26.872353,2025-07-06 05:05:27.310573,Beta - AppsScoreModel,v1,SIL Competitor,F,Appliance,android,SIL Beta - AppsScoreModel,SIL,NA,0.560750,"{""app_cnt_rated_for_3plus_ever"":31,""app_cnt_ed...",2025-07-06 13:05:24,NaT,2025-07,31,0,1,1,3,0,1,0,0,3,2,1,1,14379.111111,13037.516667,214.3,212.4,0.000000
2,1508888,bd1c1259-f630-4712-a8df-fb211a9b7059,"{'cb_score': 0.34493482516754004, 'dl_score': ...",2025-06-21 10:15:30.792002,2025-06-21 10:15:31.244757,Beta - AppsScoreModel,v1,SIL Competitor,F,Appliance,android,SIL Beta - AppsScoreModel,SIL,NA,0.505210,"{""app_cnt_rated_for_3plus_ever"":20,""app_cnt_ed...",2025-06-21 18:15:29,NaT,2025-06,20,0,4,1,0,0,1,0,0,4,1,0,1,9484.258333,9484.258333,1189.9,1035.9,0.000000
3,1976352,1caec6aa-3c9c-43a9-9939-57e81592f47c,"{'cb_score': 0.5601615539152716, 'dl_score': 0...",2025-04-16 08:27:06.570807,2025-04-16 08:27:07.008496,Beta - AppsScoreModel,v1,SIL-Instore,M,Appliance,android,SIL Beta - AppsScoreModel,SIL,NA,0.666355,"{""app_cnt_rated_for_3plus_ever"":36,""app_cnt_ed...",2025-04-16 16:27:03,NaT,2025-04,36,0,2,0,1,9,18,14,6,20,14,6,11,1063.003153,300.633333,27.0,27.0,0.823529
4,2082698,edc67e5d-b7f7-4a2c-a05f-93cb218acec8,"{'cb_score': 0.5666707533584865, 'dl_score': 0...",2025-07-20 06:44:33.478087,2025-07-20 06:44:33.924970,Beta - AppsScoreModel,v1,SIL Competitor,M,Appliance,android,SIL Beta - AppsScoreModel,SIL,NA,0.459619,"{""app_cnt_rated_for_3plus_ever"":28,""app_cnt_ed...",2025-07-20 14:44:29,NaT,2025-07,28,0,5,0,0,1,5,3,0,11,8,3,4,2137.322222,1596.808333,85.8,86.6,0.375000


In [121]:
df = expanded_df.drop(columns=['calcFeatures']).copy()
df = df.drop_duplicates(keep='first')

In [122]:
df['sil_beta_app_score'] = pd.to_numeric(df['sil_beta_app_score'], errors='coerce')


### run psi calculation

In [123]:
df = df.copy()

# Define feature list
feature_list = ['sil_beta_app_score',
       'calc_app_cnt_rated_for_3plus_ever', 'calc_app_cnt_education_ever',
       'calc_app_cnt_business_ever', 'calc_app_cnt_music_and_audio_ever',
       'calc_app_cnt_travel_and_local_ever', 'calc_app_cnt_finance_7d',
       'calc_app_cnt_absence_tag_30d', 'calc_app_cnt_competitors_30d',
       'calc_app_cnt_finance_30d', 'calc_app_cnt_absence_tag_90d',
       'calc_app_cnt_finance_90d', 'calc_app_cnt_competitors_90d',
       'calc_app_cnt_payday_90d', 'calc_app_avg_time_bw_installed_mins_30d',
       'calc_app_median_time_bw_installed_mins_30d',
       'calc_app_first_competitors_install_to_apply_days',
       'calc_app_first_payday_install_to_apply_days',
       'calc_app_vel_finance_30_over_365']

# Define segment columns
segment_columns = ['new_loan_type', 'osType', 'loan_product_type']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',   'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_appscore_model_psi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_appscore_model_csi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=8a72428a-654b-4bc0-b533-94d7a4efd928>

In [124]:
psi_results

,modelDisplayName,Model_Name,modelVersionId,trenchCategory,Feature,Feature_Type,Segment_Column,Segment_Value,Month,Base_Month,Current_Month,PSI
0,Beta - AppsScoreModel,SIL Beta - AppsScoreModel,v1,NA,sil_beta_app_score,numerical,Overall,All,2025-03,2025-03,2025-03,0.000000
1,Beta - AppsScoreModel,SIL Beta - AppsScoreModel,v1,NA,sil_beta_app_score,numerical,Overall,All,2025-04,2025-03,2025-04,0.042601
2,Beta - AppsScoreModel,SIL Beta - AppsScoreModel,v1,NA,sil_beta_app_score,numerical,Overall,All,2025-05,2025-03,2025-05,0.023790
3,Beta - AppsScoreModel,SIL Beta - AppsScoreModel,v1,NA,sil_beta_app_score,numerical,Overall,All,2025-06,2025-03,2025-06,0.035022
4,Beta - AppsScoreModel,SIL Beta - AppsScoreModel,v1,NA,sil_beta_app_score,numerical,Overall,All,2025-07,2025-03,2025-07,0.031842
...,...,...,...,...,...,...,...,...,...,...,...,...
1021,Beta - AppsScoreModel,SIL Beta - AppsScoreModel,v1,NA,calc_app_vel_finance_30_over_365,numerical,loan_product_type,Mall,2025-07,2025-03,2025-07,0.507218
1022,Beta - AppsScoreModel,SIL Beta - AppsScoreModel,v1,NA,calc_app_vel_finance_30_over_365,numerical,loan_product_type,Mall,2025-08,2025-03,2025-08,0.544423
1023,Beta - AppsScoreModel,SIL Beta - AppsScoreModel,v1,NA,calc_app_vel_finance_30_over_365,numerical,loan_product_type,Mall,2025-09,2025-03,2025-09,0.458509
1024,Beta - AppsScoreModel,SIL Beta - AppsScoreModel,v1,NA,calc_app_vel_finance_30_over_365,numerical,loan_product_type,Mall,2025-10,2025-03,2025-10,0.426917


## SIL Beta - DemoScoreModel

In [125]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.beta_demo_score_model_psi;"""
job = client.query(sq)
job.result()  # Wait for job to complete


sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.beta_demo_score_model_csi;"""
job = client.query(sq)
job.result()  # Wait for job to complete


In [126]:
sq = """ WITH cleaned AS (
  SELECT
  customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature_cleaned
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName = 'Beta - DemoScoreModel')
SELECT
  r.customerId,  r.digitalLoanAccountId,start_time, prediction sil_beta_demo_score, modelDisplayName,modelVersionId,
 loanmaster.new_loan_type,
 loanmaster.gender,
      case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
  case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
 'SIL Beta - DemoScoreModel' Model_Name,
 'SIL' as product,
 'NA' trenchCategory,
  r.calcFeature_cleaned calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,
FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code

qualify row_number() over(partition by r.customerId, r.digitalLoanAccountId order by start_time desc) = 1;"""

dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,start_time,sil_beta_demo_score,modelDisplayName,modelVersionId,new_loan_type,gender,osType,loan_product_type,Model_Name,product,trenchCategory,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,1091255,e55b09ec-a687-44a4-9274-517dfb4afe72,2025-11-09 04:14:22.693009,0.0589409823,Beta - DemoScoreModel,v1,SIL-Instore,M,ios,Appliance,SIL Beta - DemoScoreModel,SIL,NA,"{""beta_de_ln_vas_opted_flag"": ""0"", ""beta_de_ln...",2025-11-09 12:14:19,2025-11-09 12:17:20,2025-11
1,1879654,4cf075c8-c81d-4759-9780-b0d9b883d1e1,2025-06-07 10:04:30.863592,0.1411435142,Beta - DemoScoreModel,v1,SIL-Instore,M,android,Appliance,SIL Beta - DemoScoreModel,SIL,NA,"{""beta_de_ln_vas_opted_flag"": ""1"", ""beta_de_ln...",2025-06-07 18:04:27,NaT,2025-06
2,2111705,be747c53-86f7-47ec-8a50-c6d6d1fc46f2,2025-09-05 06:31:03.968950,0.1734630861,Beta - DemoScoreModel,v1,SIL-Instore,M,android,Appliance,SIL Beta - DemoScoreModel,SIL,NA,"{""beta_de_ln_vas_opted_flag"": ""1"", ""beta_de_ln...",2025-09-05 14:30:52,NaT,2025-09
3,2112453,03db70c5-3273-467c-a3ca-1d0572f6ede9,2025-07-29 09:32:06.905992,0.1168238107,Beta - DemoScoreModel,v1,SIL Competitor,F,android,Appliance,SIL Beta - DemoScoreModel,SIL,NA,"{""beta_de_ln_vas_opted_flag"": ""0"", ""beta_de_ln...",2025-07-29 17:32:04,2025-07-29 17:35:30,2025-07
4,2145919,f843de3c-ab0d-4869-914e-5268bce8db31,2025-10-28 03:10:12.088136,0.15917279,Beta - DemoScoreModel,v1,SIL-Instore,M,android,Appliance,SIL Beta - DemoScoreModel,SIL,NA,"{""beta_de_ln_vas_opted_flag"": ""1"", ""beta_de_ln...",2025-10-28 11:10:09,NaT,2025-10


### Expand df

In [127]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 17
Expanded columns: 34


,customerId,digitalLoanAccountId,start_time,sil_beta_demo_score,modelDisplayName,modelVersionId,new_loan_type,gender,osType,loan_product_type,Model_Name,product,trenchCategory,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month,calc_beta_de_ln_vas_opted_flag,calc_beta_de_ln_doc_type_rolled,calc_beta_de_ln_marital_status,calc_beta_de_ln_age_bin,calc_beta_de_ln_province_bin,calc_beta_de_ln_ref2_type,calc_beta_de_ln_education_level,calc_beta_de_ln_ref1_type,calc_beta_de_ln_industry_new_bin,calc_beta_de_ln_appln_day_of_week,calc_beta_de_onb_name_email_match_score,calc_beta_de_ln_employment_type_new_bin,calc_beta_de_ln_telconame,calc_beta_de_time_bw_onb_loan_appln_mins,calc_beta_de_ln_source_of_funds_new_bin,calc_beta_de_ln_brand_bin,calc_beta_de_ln_email_primary_domain
0,1091255,e55b09ec-a687-44a4-9274-517dfb4afe72,2025-11-09 04:14:22.693009,0.0589409823,Beta - DemoScoreModel,v1,SIL-Instore,M,ios,Appliance,SIL Beta - DemoScoreModel,SIL,NA,"{""beta_de_ln_vas_opted_flag"": ""0"", ""beta_de_ln...",2025-11-09 12:14:19,2025-11-09 12:17:20,2025-11,0,Driving License,Married,"(32.0, 38.0]",others,Spouse,College Graduate,Friend,4,Sunday,100,Employed - Private Employee,Globe,2159719.0,Salary,apple,gmail.com
1,1879654,4cf075c8-c81d-4759-9780-b0d9b883d1e1,2025-06-07 10:04:30.863592,0.1411435142,Beta - DemoScoreModel,v1,SIL-Instore,M,android,Appliance,SIL Beta - DemoScoreModel,SIL,NA,"{""beta_de_ln_vas_opted_flag"": ""1"", ""beta_de_ln...",2025-06-07 18:04:27,NaT,2025-06,1,National ID,Single,"(32.0, 38.0]",metro manila,Friend,Technical/Vocational Graduate,Sibling,3,Saturday,62,Employed - Private Employee,Smart,1236964.0,Salary,tecno,gmail.com
2,2111705,be747c53-86f7-47ec-8a50-c6d6d1fc46f2,2025-09-05 06:31:03.968950,0.1734630861,Beta - DemoScoreModel,v1,SIL-Instore,M,android,Appliance,SIL Beta - DemoScoreModel,SIL,NA,"{""beta_de_ln_vas_opted_flag"": ""1"", ""beta_de_ln...",2025-09-05 14:30:52,NaT,2025-09,1,Driving License,Single,"(32.0, 38.0]",metro manila,Parent,College Graduate,Friend,3,Friday,100,Employed - Private Employee,DITO,1150939.0,Salary,samsung,gmail.com
3,2112453,03db70c5-3273-467c-a3ca-1d0572f6ede9,2025-07-29 09:32:06.905992,0.1168238107,Beta - DemoScoreModel,v1,SIL Competitor,F,android,Appliance,SIL Beta - DemoScoreModel,SIL,NA,"{""beta_de_ln_vas_opted_flag"": ""0"", ""beta_de_ln...",2025-07-29 17:32:04,2025-07-29 17:35:30,2025-07,0,Passport,Single,"(38.0, 46.0]",quezon,Sibling,Technical/Vocational Graduate,Sibling,2,Tuesday,100,New,Globe,1095903.0,Income from Business,oppo,gmail.com
4,2145919,f843de3c-ab0d-4869-914e-5268bce8db31,2025-10-28 03:10:12.088136,0.15917279,Beta - DemoScoreModel,v1,SIL-Instore,M,android,Appliance,SIL Beta - DemoScoreModel,SIL,NA,"{""beta_de_ln_vas_opted_flag"": ""1"", ""beta_de_ln...",2025-10-28 11:10:09,NaT,2025-10,1,National ID,Single,"(-inf, 27.0]",metro manila,Friend,High School Graduate,Parent,New,Tuesday,100,Employed - Private Employee,Smart,1197471.0,Salary,realme,gmail.com


In [128]:
df = expanded_df.drop(columns=['calcFeatures']).copy()
df = df.drop_duplicates(keep='first')

In [129]:
df['sil_beta_demo_score'] = pd.to_numeric(df['sil_beta_demo_score'], errors='coerce')


### run psi calculation

In [130]:
df = df.copy()

# Define feature list
feature_list = ['sil_beta_demo_score',
       'calc_beta_de_ln_vas_opted_flag',
       'calc_beta_de_ln_doc_type_rolled', 'calc_beta_de_ln_marital_status',
       'calc_beta_de_ln_age_bin', 'calc_beta_de_ln_province_bin',
       'calc_beta_de_ln_ref2_type', 'calc_beta_de_ln_education_level',
       'calc_beta_de_ln_ref1_type', 'calc_beta_de_ln_industry_new_bin',
       'calc_beta_de_ln_appln_day_of_week',
       'calc_beta_de_onb_name_email_match_score',
       'calc_beta_de_ln_employment_type_new_bin', 'calc_beta_de_ln_telconame',
       'calc_beta_de_time_bw_onb_loan_appln_mins',
       'calc_beta_de_ln_source_of_funds_new_bin', 'calc_beta_de_ln_brand_bin',
       'calc_beta_de_ln_email_primary_domain']

# Define segment columns
segment_columns = ['new_loan_type', 'osType', 'loan_product_type']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',   'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_demo_score_model_psi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_demo_score_model_csi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=6a4b0cdf-52b3-4c0e-9ffd-de62929df41b>

In [131]:
psi_results

,modelDisplayName,Model_Name,modelVersionId,trenchCategory,Feature,Feature_Type,Segment_Column,Segment_Value,Month,Base_Month,Current_Month,PSI
0,Beta - DemoScoreModel,SIL Beta - DemoScoreModel,v1,NA,sil_beta_demo_score,numerical,Overall,All,2025-03,2025-03,2025-03,0.000000
1,Beta - DemoScoreModel,SIL Beta - DemoScoreModel,v1,NA,sil_beta_demo_score,numerical,Overall,All,2025-04,2025-03,2025-04,0.034722
2,Beta - DemoScoreModel,SIL Beta - DemoScoreModel,v1,NA,sil_beta_demo_score,numerical,Overall,All,2025-05,2025-03,2025-05,0.022231
3,Beta - DemoScoreModel,SIL Beta - DemoScoreModel,v1,NA,sil_beta_demo_score,numerical,Overall,All,2025-06,2025-03,2025-06,0.460944
4,Beta - DemoScoreModel,SIL Beta - DemoScoreModel,v1,NA,sil_beta_demo_score,numerical,Overall,All,2025-07,2025-03,2025-07,0.537148
...,...,...,...,...,...,...,...,...,...,...,...,...
1129,Beta - DemoScoreModel,SIL Beta - DemoScoreModel,v1,NA,calc_beta_de_ln_email_primary_domain,categorical,loan_product_type,Mall,2025-07,2025-03,2025-07,0.034212
1130,Beta - DemoScoreModel,SIL Beta - DemoScoreModel,v1,NA,calc_beta_de_ln_email_primary_domain,categorical,loan_product_type,Mall,2025-08,2025-03,2025-08,0.029010
1131,Beta - DemoScoreModel,SIL Beta - DemoScoreModel,v1,NA,calc_beta_de_ln_email_primary_domain,categorical,loan_product_type,Mall,2025-09,2025-03,2025-09,0.036455
1132,Beta - DemoScoreModel,SIL Beta - DemoScoreModel,v1,NA,calc_beta_de_ln_email_primary_domain,categorical,loan_product_type,Mall,2025-10,2025-03,2025-10,0.018810


## SIL_Beta - StackScoreModel

In [132]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_stack_model_psi;"""
job = client.query(sq)
job.result()  # Wait for job to complete

sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_stack_model_csi;"""
job = client.query(sq)
job.result()  # Wait for job to complete


In [133]:
sq = """ WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName = 'Beta - StackScoreModel')
SELECT
  r.customerId,r.digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
 loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
 'SIL_Beta - StackScoreModel' Model_Name,
 'SIL' as product,
 'NA' trenchCategory,
  r.calcFeature calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,  
FROM cleaned r 
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId, r.digitalLoanAccountId order by start_time desc) = 1
;
"""
dfd= client.query(sq).to_dataframe()
dfd.head()


,customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,Model_Name,product,trenchCategory,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,1266402,66c3941d-33db-4712-bc76-9f68c2678dd2,0.058753918649740264,2025-10-01 02:26:38.595791,2025-10-01 02:26:38.599632,Beta - StackScoreModel,v1,SIL-Instore,F,Appliance,SIL_Beta - StackScoreModel,SIL,NA,"{""s_apps_score"": null, ""s_credo_score"": 0.0991...",2025-10-01 10:26:36,NaT,2025-10
1,1448883,f292782d-be35-433c-8aed-c43d415b8df9,0.3972757981528528,2025-10-11 08:50:27.752535,2025-10-11 08:50:27.756942,Beta - StackScoreModel,v1,SIL-Instore,F,Appliance,SIL_Beta - StackScoreModel,SIL,NA,"{""s_apps_score"": 0.7127478368867418, ""s_credo_...",2025-10-11 16:50:24,NaT,2025-10
2,2054362,63651091-b874-4027-8a18-23d8750a3fd8,0.07757967345270918,2025-06-06 02:59:44.193336,2025-06-06 02:59:44.197364,Beta - StackScoreModel,v1,SIL-Instore,F,Appliance,SIL_Beta - StackScoreModel,SIL,NA,"{""s_apps_score"": 0.447597457977133, ""s_credo_s...",2025-06-06 10:59:40,2025-06-06 11:04:54,2025-06
3,2087577,d047c09b-0cac-4327-94d4-becfdfa35c88,0.12574702900343615,2025-09-16 09:17:28.743855,2025-09-16 09:17:28.748066,Beta - StackScoreModel,v1,SIL Competitor,M,Appliance,SIL_Beta - StackScoreModel,SIL,NA,"{""s_apps_score"": 0.5235731600077855, ""s_credo_...",2025-09-16 17:17:24,2025-09-16 17:33:53,2025-09
4,2142063,b25f5235-8018-4e21-af0c-677292ec97ea,0.09388067513368545,2025-08-02 13:18:07.579325,2025-08-02 13:18:07.583333,Beta - StackScoreModel,v1,SIL Competitor,M,Mall,SIL_Beta - StackScoreModel,SIL,NA,"{""s_apps_score"": 0.5266375278565278, ""s_credo_...",2025-08-02 21:18:03,NaT,2025-08


### Expand df

In [134]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 17
Expanded columns: 20


,customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,Model_Name,product,trenchCategory,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month,calc_s_apps_score,calc_s_credo_score,calc_sb_demo_score
0,1266402,66c3941d-33db-4712-bc76-9f68c2678dd2,0.058753918649740264,2025-10-01 02:26:38.595791,2025-10-01 02:26:38.599632,Beta - StackScoreModel,v1,SIL-Instore,F,Appliance,SIL_Beta - StackScoreModel,SIL,NA,"{""s_apps_score"": null, ""s_credo_score"": 0.0991...",2025-10-01 10:26:36,NaT,2025-10,NaN,0.0991,0.062487
1,1448883,f292782d-be35-433c-8aed-c43d415b8df9,0.3972757981528528,2025-10-11 08:50:27.752535,2025-10-11 08:50:27.756942,Beta - StackScoreModel,v1,SIL-Instore,F,Appliance,SIL_Beta - StackScoreModel,SIL,NA,"{""s_apps_score"": 0.7127478368867418, ""s_credo_...",2025-10-11 16:50:24,NaT,2025-10,0.712748,0.2607,0.109952
2,2054362,63651091-b874-4027-8a18-23d8750a3fd8,0.07757967345270918,2025-06-06 02:59:44.193336,2025-06-06 02:59:44.197364,Beta - StackScoreModel,v1,SIL-Instore,F,Appliance,SIL_Beta - StackScoreModel,SIL,NA,"{""s_apps_score"": 0.447597457977133, ""s_credo_s...",2025-06-06 10:59:40,2025-06-06 11:04:54,2025-06,0.447597,0.1045,0.126578
3,2087577,d047c09b-0cac-4327-94d4-becfdfa35c88,0.12574702900343615,2025-09-16 09:17:28.743855,2025-09-16 09:17:28.748066,Beta - StackScoreModel,v1,SIL Competitor,M,Appliance,SIL_Beta - StackScoreModel,SIL,NA,"{""s_apps_score"": 0.5235731600077855, ""s_credo_...",2025-09-16 17:17:24,2025-09-16 17:33:53,2025-09,0.523573,0.0920,0.145219
4,2142063,b25f5235-8018-4e21-af0c-677292ec97ea,0.09388067513368545,2025-08-02 13:18:07.579325,2025-08-02 13:18:07.583333,Beta - StackScoreModel,v1,SIL Competitor,M,Mall,SIL_Beta - StackScoreModel,SIL,NA,"{""s_apps_score"": 0.5266375278565278, ""s_credo_...",2025-08-02 21:18:03,NaT,2025-08,0.526638,0.0999,0.089126


In [135]:
df = expanded_df.drop(columns=['calcFeatures']).copy()
df.rename(columns={'prediction': 'sil_beta_stack_score'}, inplace=True)
df = df.drop_duplicates(keep='first')

In [136]:
df['sil_beta_stack_score'] = pd.to_numeric(df['sil_beta_stack_score'], errors='coerce')


### run psi calculation

In [137]:
df.columns

Index(['customerId', 'digitalLoanAccountId', 'sil_beta_stack_score',
       'start_time', 'end_time', 'modelDisplayName', 'modelVersionId',
       'new_loan_type', 'gender', 'loan_product_type', 'Model_Name', 'product',
       'trenchCategory', 'appln_submit_datetime', 'disbursementDateTime',
       'Application_month', 'calc_s_apps_score', 'calc_s_credo_score',
       'calc_sb_demo_score'],
      dtype='object')

In [138]:
df = df.copy()

# Define feature list
feature_list = ['sil_beta_stack_score',
        'calc_s_apps_score', 'calc_s_credo_score', 'calc_sb_demo_score']

# Define segment columns
segment_columns = ['new_loan_type', 'osType', 'loan_product_type']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',   'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_stack_model_psi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_stack_model_csi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=b40b4d6e-0c62-457d-950b-8582ac5ed1a5>

In [139]:
psi_results

,modelDisplayName,Model_Name,modelVersionId,trenchCategory,Feature,Feature_Type,Segment_Column,Segment_Value,Month,Base_Month,Current_Month,PSI
0,Beta - StackScoreModel,SIL_Beta - StackScoreModel,v1,NA,sil_beta_stack_score,numerical,Overall,All,2025-03,2025-03,2025-03,0.000000
1,Beta - StackScoreModel,SIL_Beta - StackScoreModel,v1,NA,sil_beta_stack_score,numerical,Overall,All,2025-04,2025-03,2025-04,0.209108
2,Beta - StackScoreModel,SIL_Beta - StackScoreModel,v1,NA,sil_beta_stack_score,numerical,Overall,All,2025-05,2025-03,2025-05,0.232784
3,Beta - StackScoreModel,SIL_Beta - StackScoreModel,v1,NA,sil_beta_stack_score,numerical,Overall,All,2025-06,2025-03,2025-06,0.052853
4,Beta - StackScoreModel,SIL_Beta - StackScoreModel,v1,NA,sil_beta_stack_score,numerical,Overall,All,2025-07,2025-03,2025-07,0.050427
...,...,...,...,...,...,...,...,...,...,...,...,...
175,Beta - StackScoreModel,SIL_Beta - StackScoreModel,v1,NA,calc_sb_demo_score,numerical,loan_product_type,Mall,2025-07,2025-03,2025-07,0.418687
176,Beta - StackScoreModel,SIL_Beta - StackScoreModel,v1,NA,calc_sb_demo_score,numerical,loan_product_type,Mall,2025-08,2025-03,2025-08,0.431313
177,Beta - StackScoreModel,SIL_Beta - StackScoreModel,v1,NA,calc_sb_demo_score,numerical,loan_product_type,Mall,2025-09,2025-03,2025-09,0.459800
178,Beta - StackScoreModel,SIL_Beta - StackScoreModel,v1,NA,calc_sb_demo_score,numerical,loan_product_type,Mall,2025-10,2025-03,2025-10,0.605497


## Beta - IncomeEstimationModel

In [140]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_income_estimation_model_psi;"""        
job = client.query(sq)
job.result()  # Wait for job to complete

sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_income_estimation_model_csi;"""        
job = client.query(sq)
job.result()  # Wait for job to complete

In [141]:
sq = """ 
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,

    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName = 'Beta - IncomeEstimationModel')
SELECT
  r.customerId,r.digitalLoanAccountId,prediction sil_beta_income_estimation_score,start_time,end_time,modelDisplayName,modelVersionId,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_loan_type") AS inc_beta_ln_loan_type,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_education_level") AS inc_beta_ln_education_level,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_employment_type_new") AS inc_beta_ln_employment_type_new,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_industry_new") AS inc_beta_ln_industry_new,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_age") AS inc_beta_ln_age,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_brand") AS inc_beta_ln_brand,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_city") AS inc_beta_ln_city,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_purpose") AS inc_beta_ln_purpose,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_osversion_bin") AS inc_beta_ln_osversion_bin,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_postal_code") AS inc_beta_ln_postal_code,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_gender") AS inc_beta_ln_gender,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_doc_type_rolled") AS inc_beta_ln_doc_type_rolled,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_cnt_dependents") AS inc_beta_ln_cnt_dependents,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_source_of_funds_new") AS inc_beta_ln_source_of_funds_new,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_marital_status_new") AS inc_beta_ln_marital_status_new,
  JSON_VALUE(calcFeature, "$.inc_beta_encoded_company_name_grouped") AS inc_beta_encoded_company_name_grouped,
   loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
 'SIL Beta - IncomeEstimationModel' Model_Name,
 'SIL' as product,
 'NA' trenchCategory,
  r.calcFeature calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,  
FROM cleaned r 
left join risk_credit_mis.loan_master_table loanmaster 
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId, r.digitalLoanAccountId order by start_time desc) = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,sil_beta_income_estimation_score,start_time,end_time,modelDisplayName,modelVersionId,inc_beta_ln_loan_type,inc_beta_ln_education_level,inc_beta_ln_employment_type_new,inc_beta_ln_industry_new,inc_beta_ln_age,inc_beta_ln_brand,inc_beta_ln_city,inc_beta_ln_purpose,inc_beta_ln_osversion_bin,inc_beta_ln_postal_code,inc_beta_ln_gender,inc_beta_ln_doc_type_rolled,inc_beta_ln_cnt_dependents,inc_beta_ln_source_of_funds_new,inc_beta_ln_marital_status_new,inc_beta_encoded_company_name_grouped,new_loan_type,gender,loan_product_type,Model_Name,product,trenchCategory,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,2031572,b688d202-3cbd-40d1-8c98-710158d0136a,6869.148109244154,2025-07-16 09:53:26.027128,2025-07-16 09:53:26.100225,Beta - IncomeEstimationModel,v1,BNPL,High School Graduate,Employed - Private Employee,Fin Tech/Financial Services,22,POCO,CF0037,"Stereo set, Soundbar, Audio equipment",2-Voyager,4205,F,ID Card,2,Salary,Single,0.0,SIL-Instore,F,Appliance,SIL Beta - IncomeEstimationModel,SIL,NA,"{""inc_beta_ln_loan_type"":""BNPL"",""inc_beta_ln_e...",2025-07-16 17:41:34,NaT,2025-07
1,3087182,d3f53ed2-2d24-4fe4-b592-9571c3594a5b,9187.354147847203,2025-03-28 20:45:48.552874,2025-03-28 20:45:48.611677,Beta - IncomeEstimationModel,v1,BNPL,High School Graduate,Self-Employed/Private Practice,Other Professional services,34,Infinix,CF0208,"Stereo set, Soundbar, Audio equipment",4-Innovator,1940,M,Other,2,2,Single,0.0,SIL-Instore,M,Appliance,SIL Beta - IncomeEstimationModel,SIL,NA,"{""inc_beta_ln_loan_type"":""BNPL"",""inc_beta_ln_e...",2025-03-26 17:55:41,2025-03-31 14:19:57,2025-03
2,3201489,04364a3a-7b84-451c-92ad-1d8b0d9ce290,26994.508829444403,2025-06-07 04:39:30.308179,2025-06-07 04:39:30.361013,Beta - IncomeEstimationModel,v1,BNPL,College Graduate,Employed - Private Employee,Fin Tech/Financial Services,23,Apple,CF0375,Refrigerators,5-Pioneer,1440,F,ID Card,0,Salary,Single,397753.1527432982,SIL-Instore,F,Appliance,SIL Beta - IncomeEstimationModel,SIL,NA,"{""inc_beta_ln_loan_type"":""BNPL"",""inc_beta_ln_e...",2025-06-07 12:39:27,NaT,2025-06
3,3247020,b89280a3-abf7-4359-8e11-d30a1a1d4afe,7773.44279583617,2025-11-11 08:42:45.380769,2025-11-11 08:42:45.443738,Beta - IncomeEstimationModel,v1,SILCOM,Technical/Vocational Graduate,Employed - Private Employee,Car/Boat/Plane/Used Truck/AutoParts Manufactur...,26,vivo,CF0288,Refrigerators,2-Voyager,4027,M,Driving License,0,Salary,Single,0.0,SIL Competitor,M,Appliance,SIL Beta - IncomeEstimationModel,SIL,NA,"{""inc_beta_ln_loan_type"":""SILCOM"",""inc_beta_ln...",2025-11-11 16:42:39,2025-11-11 17:02:30,2025-11
4,3337888,c5a1280d-166b-4738-8989-f9c49365bb50,29990.77512107438,2025-03-26 04:54:04.953249,2025-03-26 04:54:05.017533,Beta - IncomeEstimationModel,v1,BNPL,College Undergraduate,Self-Employed/Private Practice,Skilled Worker,39,Apple,CF1279,Televisions,5-Pioneer,4118,F,Other,4,2,Single,30892.5062448281,SIL-Instore,F,Appliance,SIL Beta - IncomeEstimationModel,SIL,NA,"{""inc_beta_ln_loan_type"":""BNPL"",""inc_beta_ln_e...",2025-03-26 12:54:02,2025-03-26 12:59:22,2025-03


### Expand df


In [142]:
# Expand the calcFeatures column
expanded_df = dfd.copy()

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
expanded_df.head()


Original columns: 33
Expanded columns: 33


,customerId,digitalLoanAccountId,sil_beta_income_estimation_score,start_time,end_time,modelDisplayName,modelVersionId,inc_beta_ln_loan_type,inc_beta_ln_education_level,inc_beta_ln_employment_type_new,inc_beta_ln_industry_new,inc_beta_ln_age,inc_beta_ln_brand,inc_beta_ln_city,inc_beta_ln_purpose,inc_beta_ln_osversion_bin,inc_beta_ln_postal_code,inc_beta_ln_gender,inc_beta_ln_doc_type_rolled,inc_beta_ln_cnt_dependents,inc_beta_ln_source_of_funds_new,inc_beta_ln_marital_status_new,inc_beta_encoded_company_name_grouped,new_loan_type,gender,loan_product_type,Model_Name,product,trenchCategory,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,2031572,b688d202-3cbd-40d1-8c98-710158d0136a,6869.148109244154,2025-07-16 09:53:26.027128,2025-07-16 09:53:26.100225,Beta - IncomeEstimationModel,v1,BNPL,High School Graduate,Employed - Private Employee,Fin Tech/Financial Services,22,POCO,CF0037,"Stereo set, Soundbar, Audio equipment",2-Voyager,4205,F,ID Card,2,Salary,Single,0.0,SIL-Instore,F,Appliance,SIL Beta - IncomeEstimationModel,SIL,NA,"{""inc_beta_ln_loan_type"":""BNPL"",""inc_beta_ln_e...",2025-07-16 17:41:34,NaT,2025-07
1,3087182,d3f53ed2-2d24-4fe4-b592-9571c3594a5b,9187.354147847203,2025-03-28 20:45:48.552874,2025-03-28 20:45:48.611677,Beta - IncomeEstimationModel,v1,BNPL,High School Graduate,Self-Employed/Private Practice,Other Professional services,34,Infinix,CF0208,"Stereo set, Soundbar, Audio equipment",4-Innovator,1940,M,Other,2,2,Single,0.0,SIL-Instore,M,Appliance,SIL Beta - IncomeEstimationModel,SIL,NA,"{""inc_beta_ln_loan_type"":""BNPL"",""inc_beta_ln_e...",2025-03-26 17:55:41,2025-03-31 14:19:57,2025-03
2,3201489,04364a3a-7b84-451c-92ad-1d8b0d9ce290,26994.508829444403,2025-06-07 04:39:30.308179,2025-06-07 04:39:30.361013,Beta - IncomeEstimationModel,v1,BNPL,College Graduate,Employed - Private Employee,Fin Tech/Financial Services,23,Apple,CF0375,Refrigerators,5-Pioneer,1440,F,ID Card,0,Salary,Single,397753.1527432982,SIL-Instore,F,Appliance,SIL Beta - IncomeEstimationModel,SIL,NA,"{""inc_beta_ln_loan_type"":""BNPL"",""inc_beta_ln_e...",2025-06-07 12:39:27,NaT,2025-06
3,3247020,b89280a3-abf7-4359-8e11-d30a1a1d4afe,7773.44279583617,2025-11-11 08:42:45.380769,2025-11-11 08:42:45.443738,Beta - IncomeEstimationModel,v1,SILCOM,Technical/Vocational Graduate,Employed - Private Employee,Car/Boat/Plane/Used Truck/AutoParts Manufactur...,26,vivo,CF0288,Refrigerators,2-Voyager,4027,M,Driving License,0,Salary,Single,0.0,SIL Competitor,M,Appliance,SIL Beta - IncomeEstimationModel,SIL,NA,"{""inc_beta_ln_loan_type"":""SILCOM"",""inc_beta_ln...",2025-11-11 16:42:39,2025-11-11 17:02:30,2025-11
4,3337888,c5a1280d-166b-4738-8989-f9c49365bb50,29990.77512107438,2025-03-26 04:54:04.953249,2025-03-26 04:54:05.017533,Beta - IncomeEstimationModel,v1,BNPL,College Undergraduate,Self-Employed/Private Practice,Skilled Worker,39,Apple,CF1279,Televisions,5-Pioneer,4118,F,Other,4,2,Single,30892.5062448281,SIL-Instore,F,Appliance,SIL Beta - IncomeEstimationModel,SIL,NA,"{""inc_beta_ln_loan_type"":""BNPL"",""inc_beta_ln_e...",2025-03-26 12:54:02,2025-03-26 12:59:22,2025-03


In [143]:
df = expanded_df.drop(columns=['calcFeatures']).copy()

In [144]:
df['sil_beta_income_estimation_score'] = pd.to_numeric(df['sil_beta_income_estimation_score'], errors='coerce')
df.columns

Index(['customerId', 'digitalLoanAccountId',
       'sil_beta_income_estimation_score', 'start_time', 'end_time',
       'modelDisplayName', 'modelVersionId', 'inc_beta_ln_loan_type',
       'inc_beta_ln_education_level', 'inc_beta_ln_employment_type_new',
       'inc_beta_ln_industry_new', 'inc_beta_ln_age', 'inc_beta_ln_brand',
       'inc_beta_ln_city', 'inc_beta_ln_purpose', 'inc_beta_ln_osversion_bin',
       'inc_beta_ln_postal_code', 'inc_beta_ln_gender',
       'inc_beta_ln_doc_type_rolled', 'inc_beta_ln_cnt_dependents',
       'inc_beta_ln_source_of_funds_new', 'inc_beta_ln_marital_status_new',
       'inc_beta_encoded_company_name_grouped', 'new_loan_type', 'gender',
       'loan_product_type', 'Model_Name', 'product', 'trenchCategory',
       'appln_submit_datetime', 'disbursementDateTime', 'Application_month'],
      dtype='object')

### run psi calculation

In [145]:
df = df.copy()

# Define feature list
feature_list = ['sil_beta_income_estimation_score',  'inc_beta_ln_loan_type',
       'inc_beta_ln_education_level', 'inc_beta_ln_employment_type_new',
       'inc_beta_ln_industry_new', 'inc_beta_ln_age', 'inc_beta_ln_brand',
       'inc_beta_ln_city', 'inc_beta_ln_purpose', 'inc_beta_ln_osversion_bin',
       'inc_beta_ln_postal_code', 'inc_beta_ln_gender',
       'inc_beta_ln_doc_type_rolled', 'inc_beta_ln_cnt_dependents',
       'inc_beta_ln_source_of_funds_new', 'inc_beta_ln_marital_status_new',
       'inc_beta_encoded_company_name_grouped',]

# Define segment columns
segment_columns = ['new_loan_type', 'osType', 'loan_product_type']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[[ 'modelDisplayName', 'Model_Name',   'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[[ 'modelDisplayName', 'Model_Name',  'modelVersionId', 'trenchCategory','Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
       'Base_Month', 'Current_Month', 'Bin', 'Bin_Range', 'Bin_Min', 'Bin_Max', 'Base_Percentage',
       'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_income_estimation_model_psi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_income_estimation_model_csi"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=79401406-51cd-42c5-8fa6-d39ec9826c31>

In [146]:
bin_psi_results

,modelDisplayName,Model_Name,modelVersionId,trenchCategory,Feature,Feature_Type,Segment_Column,Segment_Value,Month,Base_Month,Current_Month,Bin,Bin_Range,Bin_Min,Bin_Max,Base_Percentage,Actual_Percentage,Bin_PSI
0,Beta - IncomeEstimationModel,SIL Beta - IncomeEstimationModel,v1,NA,sil_beta_income_estimation_score,numerical,Overall,All,2025-03,2025-03,2025-03,Bin_10,"(29863.253939662456, inf)",2.986325e+04,inf,10.144928,10.144928,0.000000
1,Beta - IncomeEstimationModel,SIL Beta - IncomeEstimationModel,v1,NA,sil_beta_income_estimation_score,numerical,Overall,All,2025-03,2025-03,2025-03,Bin_4,"[8957.18, 9067.55]",8.957178e+03,9.067554e+03,10.144928,10.144928,0.000000
2,Beta - IncomeEstimationModel,SIL Beta - IncomeEstimationModel,v1,NA,sil_beta_income_estimation_score,numerical,Overall,All,2025-03,2025-03,2025-03,Bin_1,"(-inf, 8539.5031693278)",-inf,8.539503e+03,10.144928,10.144928,0.000000
3,Beta - IncomeEstimationModel,SIL Beta - IncomeEstimationModel,v1,NA,sil_beta_income_estimation_score,numerical,Overall,All,2025-03,2025-03,2025-03,Bin_7,"[9357.10, 25685.69]",9.357096e+03,2.568569e+04,10.144928,10.144928,0.000000
4,Beta - IncomeEstimationModel,SIL Beta - IncomeEstimationModel,v1,NA,sil_beta_income_estimation_score,numerical,Overall,All,2025-03,2025-03,2025-03,Bin_6,"[9183.84, 9357.10]",9.183843e+03,9.357096e+03,9.903382,9.903382,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4402,Beta - IncomeEstimationModel,SIL Beta - IncomeEstimationModel,v1,NA,inc_beta_encoded_company_name_grouped,categorical,loan_product_type,Mall,2025-11,2025-03,2025-11,2949351.4323653369,2949351.4323653369,NaN,NaN,0.000000,0.148699,0.003744
4403,Beta - IncomeEstimationModel,SIL Beta - IncomeEstimationModel,v1,NA,inc_beta_encoded_company_name_grouped,categorical,loan_product_type,Mall,2025-11,2025-03,2025-11,30086.7246045263,30086.7246045263,NaN,NaN,2.222222,0.111524,0.063153
4404,Beta - IncomeEstimationModel,SIL Beta - IncomeEstimationModel,v1,NA,inc_beta_encoded_company_name_grouped,categorical,loan_product_type,Mall,2025-11,2025-03,2025-11,30892.5062448281,30892.5062448281,NaN,NaN,15.555556,12.602230,0.006218
4405,Beta - IncomeEstimationModel,SIL Beta - IncomeEstimationModel,v1,NA,inc_beta_encoded_company_name_grouped,categorical,loan_product_type,Mall,2025-11,2025-03,2025-11,480455.9091384736,480455.9091384736,NaN,NaN,2.222222,0.074349,0.072974


# End